Genera, entrena y actualiza los parámetros día a día: Se alimenta desde Nexus Main

In [869]:
# v241230: Los pkls, empiezan a guardarse en GCP

In [870]:
# TO DO

# sys.exit("Buscar CONTINUAR AQUI [241022]")"

# 241021_1130: Asegurar agilidad e incrementalidad en los parámetros 1-4
# Train/test en todo lo que sea ajuste de curvas o RLS en gral

# OK:Cambia LTCH por TIPO_MEDIO (+ agregado)}

# palabras clave de búsqueda:  'SMALL BATCH', 'ELIMINAR', 'REVISAR'

# Librerías

In [871]:
import os
import random
import itertools
import pandas as pd
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import matplotlib.dates as mdates # Permite particionar las fechas que se muestran en el eje x (ejemplo, cada 6 meses)
from sklearn import linear_model
import datetime as dt
import sys
from sklearn.model_selection import train_test_split
import time

#import logging
#logging.getLogger('prophet').setLevel(logging.DEBUG)

import tqdm

#from prophet import Prophet
#from prophet.plot import add_changepoints_to_plot
#from sklearn.model_selection import train_test_split

import logging
logger = logging.getLogger('cmdstanpy')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)
logging.getLogger('NeuralProphet').setLevel(logging.ERROR)
logging.getLogger('NeuralProphet').setLevel(logging.DEBUG)

from matplotlib.dates import (
    MonthLocator,
    num2date,
    AutoDateLocator,
    AutoDateFormatter)

from neuralprophet import NeuralProphet, set_log_level
from neuralprophet import save as model_save
from neuralprophet import load as model_load

# Disable logging messages unless there is an error
set_log_level("ERROR")

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

import inspect # importacion

time_0 = time.time()

# Parámetos

In [872]:
version_rapida = False
primera_del_dia = True

actualizar_todo = False # mantener en False (va a buscar toda la data desde el comienzo, y no solamente las tuplas faltantes)
no_entrenar_nada = False

In [873]:
ejecutar = {1: {'Actualizar_tuplas': False},  2: {'Actualizar_tuplas': False, 'Entrenar': True, 'Consolidar': True}, 3: {'Actualizar_tuplas': False, 'Entrenar': True, 'Evolucion_ECM': True, 'Predict': True, 'Shares_canal_fuente': True, 'Consolidar': True}, 4: {'Entrenar': True, 'Consolidar': True}} # parametro temporal

# Actualizar_tuplas: No el remanente, toda la data (ir a buscar nuevamente toda la data con el request)
# mientras (prueba)
#ejecutar = {1: False,  2: {'Actualizar_tuplas': False, 'Entrenar': False}, 3: {'Reiniciar_data_historica': True, 'Train': True, 'Evolucion_ECM': True, 'Predict': True, 'Shares_canal_fuente': True, 'Consolidar': True}} # parametro temporal

if no_entrenar_nada:
    ejecutar[2]['Entrenar'] = False
    ejecutar[3]['Entrenar'] = False
    
if version_rapida:
    ejecutar[2]['Consolidar'] = False
    ejecutar[3]['Evolucion_ECM'] = False
    ejecutar[3]['Shares_canal_fuente'] = False

if primera_del_dia:
    ejecutar[3]['Consolidar'] = True
else:
    ejecutar[3]['Consolidar'] = False

if actualizar_todo:
    ejecutar[1]['Actualizar_tuplas'] = True
    ejecutar[2]['Actualizar_tuplas'] = True
    ejecutar[3]['Actualizar_tuplas'] = True
# Para el funcionamiento general, mantener en False:

# 3 - Reiniciar_data_historica: Solo en true cuando se quiera actualizar toooda la data historica, y no solo los días más recientes (los que faltan)

In [ ]:
print('SMALL BATCH TEMPORAL (ELIMINAR)')
#no_entrenar_nada = True
#ejecutar[3]['Predict'] = False
#ejecutar[3]['Consolidar'] = False
#ejecutar[2]['Entrenar'] = False

#for i in [1, 2, 3]:
#    for c in ejecutar[i]:
#        ejecutar[i][c] = False

ejecutar[2]['Consolidar'] = False
ejecutar[3]['Evolucion_ECM'] = False
ejecutar[3]['Shares_canal_fuente'] = False
ejecutar[2]['Entrenar'] = False
ejecutar[3]['Entrenar'] = False
ejecutar[3]['Consolidar'] = False
ejecutar[3]['Predict'] = False
ejecutar[4]['Entrenar'] = True

In [875]:
mode = 'PC'

In [876]:
paises = ['MX'] #['MX', 'CL', 'BR', 'UY', 'PE', 'AR', 'CO'] #['MX', 'CL', 'PE', 'BR', 'UY', 'AR', 'CO'] #['UY', 'CL', 'MX', 'CO', 'BR', 'AR', 'PE']
#metricas = ['VISITAS_ORGANICO', 'TP_ORGANICO', 'TC_ORGANICO', 'TP_PAGO', 'TC_PAGO'] 
modos_proyeccion = ['LIBRE', 'PRESUPUESTO']
tabla_data = 'sod-corp-plp-beta.ETL_main_2023.Nexus_Main_230829_0_v10'

campo_last_touch = 'TIPO_MEDIO'

# Funciones Transversales

In [877]:
carpeta_modulos = '../../Modulos/'  #/home/data/python/performance_automation/Modulos
if mode == 'VM': # Esto tiene que ir en funcion generar dag
    carpeta_modulos = '/home/data/python/performance_automation/Modulos/'

In [878]:
sys.path.insert(0, carpeta_modulos)
import funciones_transversales as tf
client = tf.generar_cliente()

In [ ]:
dir_histogramas = '../Data/histogramas/'
carpeta_input = '../Inputs/'
dir_data = '../Data/'
actualizar = False

df_dias_eventos = tf.generar_calendario_eventos_new(actualizar = actualizar, carpeta_input = carpeta_input)
df_dias_eventos.head()

# Tuplas

In [ ]:
#dias_proyeccion = [dt.datetime.today().date(), dt.datetime.today().replace(year = dt.datetime.today().year + 1, month = 12, day = 31).date()] # declarar arriba, al inicio del código
dias_proyeccion = [dt.datetime.today().replace(year = dt.datetime.today().year, month = 1, day = 1).date(), dt.datetime.today().replace(year = dt.datetime.today().year, month = 12, day = 31).date()] # por ahora (defecto)...comienzos 2025
dias_proyeccion # Desde hoy, hasta el último día del próximo año (por defecto)

In [881]:
#if 'PE' in paises:
#    sys.exit('En date PE (cualquiera), faltan correcciones para APP SODIMAC PE [ver iron 2.0]')

In [ ]:
df_paises = pd.DataFrame({'PAIS': paises})
df_paises['AUX'] = 'aux'
df_modos_proyeccion = pd.DataFrame({'MODO_PROYECCION': modos_proyeccion})
df_modos_proyeccion['AUX'] = 'aux'

df_tuplas = pd.merge(df_paises, df_modos_proyeccion, on = 'AUX')
df_tuplas = df_tuplas.drop(columns = ['AUX'])
df_tuplas

In [883]:
duplicaciones_formulacion = ['F-SF', 'F-LT', 'F', 'LT', 'TOTALES', 'F1', 'G1'] # ['F-SF-LT', 'F-SF', 'F-LT', 'F', 'LT', 'TOTALES', 'F1', 'G1'] # Parametro no dinámico (cambia menos)
duplicaciones_con_datos = ['F-SF', 'F-LT', 'F', 'LT', 'TOTALES', 'F1', 'G1'] # Parametro no dinámico (cambia menos)

In [884]:
# validacion
if len(set(duplicaciones_con_datos) - set(duplicaciones_formulacion)) != 0  :
    print('Duplicaciones con datos que no estan en duplicaciones formulacion', set(duplicaciones_con_datos) - set(duplicaciones_formulacion))
    sys.exit('Hay duplicaciones en duplicaciones_con_datos que no estan en duplicaciones_formulacion')

In [ ]:
# Obtención de niveles
all_duplicaciones = duplicaciones_formulacion
niveles = {}
for duplicacion in all_duplicaciones:
    if duplicacion == 'TOTALES':
        nivel = 0
    elif duplicacion in ['F1', 'G1']:
        nivel = 'FCOM1'
    else:
        nivel = len(duplicacion.split('-'))
    
    if nivel not in niveles:
        niveles[nivel] = []
    niveles[nivel].append(duplicacion)

max_nivel = max([nivel for nivel in niveles if type(nivel) == int]) # Type = int >> Sin casos FCOM
max_nivel

duplicaciones_sin_formulacion = list(set(duplicaciones_formulacion) - set(duplicaciones_con_datos))

if len(niveles[max_nivel]) == 1:
    grano = niveles[max_nivel]
else:
    grano = ''
    for duplicacion in niveles[max_nivel]:
        if ('LT' in duplicacion) and (grano == ''):
            grano = duplicacion
        elif ('LT' in duplicacion): # Si hay más de dos granos potenciales con LT
            sys.exit('Problema en declaración de duplicaciones, hay más de un grano')

grano

In [ ]:
niveles

In [887]:
# crea una funcion que pida los argumentos substring y string y que indique si el substring esta contenido en el string
# por ejemplo substring = hola si está contenido en el string = hokala, ya que, quitando letras de string, puedo formar el substring

# crea una funcion que pida los argumentos substring y string y que indique si el substring está contenido en el string

def es_substring(substring, string):
    for letra in string:
        if letra == substring[0]:
            substring = substring[1:]
            if len(substring) == 0:
                return True
    return False

In [ ]:
dic_representantes = {}

for duplicacion in duplicaciones_sin_formulacion:
    
    for nivel in niveles: # Se determina el nivel
        if duplicacion in niveles[nivel]:
            break
        
    for duplicacion_existente in duplicaciones_con_datos:
        # si duplicacion existente esta contenido en duplicacion (substring de cualquier forma)
        if es_substring(duplicacion_existente, duplicacion) and ('LT' in duplicacion_existente) and (duplicacion_existente in niveles[nivel - 1]):
            representante = duplicacion_existente
            break
    
    dic_representantes[duplicacion] = representante

for duplicacion in duplicaciones_con_datos: # Las duplicaciones que existen, son representadas por si mismas
    dic_representantes[duplicacion] = duplicacion
    
dic_representantes  
        

In [ ]:
niveles

# 1. Obtención de los niveles de duplicidad (parámeto tipo 4, gamma)

## 1.0. Tuplas

In [ ]:
# Se obtienen todos los casos en un df: Duplicación + apertura orgánico - pago si corresponde
df_tuplas_duplicidades = pd.DataFrame(columns = ['NIVEL', 'DUPLICACION', 'REPRESENTANTE'])
for nivel in niveles:
    for duplicacion in niveles[nivel]:
        df_tuplas_duplicidades = pd.concat([df_tuplas_duplicidades, pd.DataFrame({'NIVEL': [nivel], 'DUPLICACION': [duplicacion], 'REPRESENTANTE': [dic_representantes[duplicacion]]})])

df_tuplas_duplicidades['AUX'] = 'aux'
df_naturaleza_medio = pd.DataFrame({'NATURALEZA_MEDIO': ['ORGANICO', 'PAGO', '']})
df_naturaleza_medio['AUX'] = 'aux'

df_tuplas_duplicidades = df_tuplas_duplicidades.merge(df_naturaleza_medio, on = 'AUX')
df_tuplas_duplicidades = df_tuplas_duplicidades.drop(columns = ['AUX'])

df_tuplas_duplicidades = df_tuplas_duplicidades[(df_tuplas_duplicidades['DUPLICACION'].str.contains('LT') & df_tuplas_duplicidades['NATURALEZA_MEDIO'].isin(['ORGANICO', 'PAGO'])) | (~df_tuplas_duplicidades['DUPLICACION'].str.contains('LT') & df_tuplas_duplicidades['NATURALEZA_MEDIO'].isin(['']))]
df_tuplas_duplicidades = df_tuplas_duplicidades.reset_index(drop = True)
df_tuplas_duplicidades

In [ ]:
df_tuplas['AUX'] = 'aux'
df_tuplas_duplicidades['AUX'] = 'aux'
df_tuplas_duplicidades = df_tuplas_duplicidades.merge(df_tuplas, on = 'AUX', how = 'left')
df_tuplas_duplicidades = df_tuplas_duplicidades.drop(columns = ['AUX'])

co_sin_fcom = True

paises_no_andes = ['BR', 'MX', 'AR', 'UY']
paises_andes = ['CL', 'PE']
if co_sin_fcom:
    paises_no_andes.append('CO')

df_tuplas_duplicidades = df_tuplas_duplicidades[~((df_tuplas_duplicidades['PAIS'].isin(paises_no_andes)) & (df_tuplas_duplicidades['NIVEL'] == 'FCOM1'))]
df_tuplas_duplicidades # Tuplas que son requeridas (inter df_tuplas_duplicidades con PAIS-MODO_PROYECCION)


## 1.1. Max date (depende del representante)

In [892]:
# 1. Obtener max_date de los datos de la tabla v10 con duplicacion = representante
df_representantes = df_tuplas_duplicidades[['REPRESENTANTE']].drop_duplicates().reset_index(drop = True)
df_representantes['DIA_DESDE'], df_representantes['DIA_HASTA'] = '', ''

for i in range(len(df_representantes)):
    representante = df_representantes.loc[i, 'REPRESENTANTE']
    print(representante)
    
    query_max_date = f'select max(DATE) as DATE from `{tabla_data}_{representante}` where FUENTE_DATOS = "REAL"'

    df_max_date = tf.request_GCP_vnew(
        nombre_tabla = "",
        specific_query = query_max_date,
        client = client,
        output = False, permitir_fallos = False)

    df_max_date['DATE'] = pd.to_datetime(df_max_date['DATE'])
    dia_hasta = df_max_date['DATE'][0].date()
    
    dia_desde = (dia_hasta + dt.timedelta(days = 1)).replace(year = dia_hasta.year - 1)#.strftime('%Y-%m-%d')
    df_representantes['DIA_DESDE'][i], df_representantes['DIA_HASTA'][i] = dia_desde, dia_hasta

df_representantes

TOTALES


,REPRESENTANTE,DIA_DESDE,DIA_HASTA
0,F-SF,2024-01-23,2025-01-22
1,F-LT,2024-01-23,2025-01-22
2,F,2024-01-23,2025-01-22
3,LT,2024-01-23,2025-01-22
4,TOTALES,2024-01-23,2025-01-22


## 1.2. Datos del último año

In [893]:
df_representantes_pais = df_tuplas_duplicidades[['PAIS', 'REPRESENTANTE']].drop_duplicates().reset_index(drop = True)
df_representantes_pais

,PAIS,REPRESENTANTE
0,MX,F-SF
1,MX,F-LT
2,MX,F
3,MX,LT
4,MX,TOTALES


In [894]:
df_factores_PE = pd.DataFrame(columns = ['PAIS', 'CANAL', 'FUENTE', 'DATE', 'FACTOR'])
if 'PE' in paises:
    df_factores_PE = tf.lectura_segura_Gsheets('Factor_correccion_app_PE', 'Factores')
    df_factores_PE['DATE'] = pd.to_datetime(df_factores_PE['DATE'])
    df_factores_PE['FACTOR'] = df_factores_PE['FACTOR'].astype(float)
df_factores_PE

,PAIS,CANAL,FUENTE,DATE,FACTOR


In [895]:
"""
def request_df(tf, tabla_data, client, pais, nat_medio):
    
    dic_str_nat_medio = {'ORGANICO': '= "Organico"', 'PAGO': '!= "Organico"'}
    dic_str_nat_medio = {'ORGANICO': '!= "Pago"', 'PAGO': '= "Pago"'} # Cambio 240624...los medios propios forman parte del orgánico

    str_nat_medio = dic_str_nat_medio[nat_medio]

    str_fuente = ''
    if pais == 'CO':
        str_fuente = 'FUENTE = "SODIMAC" and'
                                                                                                                                                                                                                                                                                                                  
    query = f'SELECT DATE, SUM(VENTA_COLOCADA) AS VENTA_COLOCADA, SUM(ORDENES) AS ORDENES, SUM(VISITAS) AS VISITAS FROM `{tabla_data}` where PAIS = "{pais}" and NATURALEZA_MEDIO {str_nat_medio} and DUPLICACION = "LT" and {str_fuente} FUENTE_DATOS = "REAL" group by 1 order by 1' # main request para entrenamiento
        
    df =  tf.request_GCP_vnew(
            nombre_tabla = "",
            specific_query = query,
            client = client,
            output = True, permitir_fallos = False)

    df['DATE'] = pd.to_datetime(df['DATE'])
    
    return df
"""

def request_df_duplicidades(tf, client, tabla_data, duplicacion, paises, dia_desde, dia_hasta, diccionario_dimensiones, dia_0, df_factores_PE, co_sin_fcom = True):
    
    # CO sin FCOM
    str_co = ''
    if co_sin_fcom:
        str_co = 'and not (PAIS = "CO" and FUENTE != "SODIMAC")'
    
    # otras dimensiones (que dependen de la duplicacion)
    
    str_dimensiones, str_numeros = 'PAIS, DATE, CANAL, FUENTE, ', '1, 2, 3, 4, '
    for i, elemento in enumerate(duplicacion.split('-')):
        if elemento == 'TOTALES':
            break
        str_dimensiones += diccionario_dimensiones[elemento] + ', '
        str_numeros += f'{i + 5}, '
    
    str_dimensiones, str_numeros = str_dimensiones[:-2], str_numeros[:-2]
    
    """
    ejecutar_PE = False
    if 'PE' in paises:
        ejecutar_PE = True
    """
        
    #paises = list(set(paises) - {'PE'}) # paises sin PE, mientras que está parchada APP SODIMAC PE

    # paises          
    paises = list_to_gcp(paises)      
    
    # query   
    # dia_0 se ocupa por defecto, en caso de que no exista df_representantes y quiera actualizarse por completo  
    """  
    if ejecutar_PE:
        
        query = f'SELECT {str_dimensiones}, SUM(VENTA_COLOCADA) AS VENTA_COLOCADA, SUM(ORDENES) AS ORDENES, SUM(VISITAS) AS VISITAS FROM `{tabla_data}_{duplicacion}` where PAIS in {paises} {str_co} and FUENTE_DATOS = "REAL" and DATE >= "{dia_0}" and DATE >= "{dia_desde}" and DATE <= "{dia_hasta}" group by {str_numeros} order by 1, 2' # main request para entrenamiento
        print(query)
        
        dia_hasta = pd.to_datetime(str(dia_hasta)[:10]).date()
        dia_desde_definitivo = max(pd.to_datetime(str(dia_0)[:10]), pd.to_datetime(str(dia_desde)[:10])).date()
        print('desde, hasta', dia_desde_definitivo, dia_hasta)
        
        if ((dt.datetime(2024, 5, 8).date() >= dia_desde_definitivo) and (dt.datetime(2024, 5, 8).date() <= dia_hasta)) or ((dt.datetime(2024, 6, 8).date() >= dia_desde_definitivo) and (dt.datetime(2024, 6, 8).date() <= dia_hasta)):
            
        #sys.exit('desarrollar request para PE')
    """
                                                                                                                                                                                                                                                                    
    query = f'SELECT {str_dimensiones}, SUM(VENTA_COLOCADA) AS VENTA_COLOCADA, SUM(ORDENES) AS ORDENES, SUM(VISITAS) AS VISITAS FROM `{tabla_data}_{duplicacion}` where PAIS in {paises} {str_co} and FUENTE_DATOS = "REAL" and DATE >= "{dia_0}" and DATE >= "{dia_desde}" and DATE <= "{dia_hasta}" group by {str_numeros} order by 1, 2' # main request para entrenamiento
    
    # request y df
    df =  tf.request_GCP_vnew(
            nombre_tabla = "",
            specific_query = query,
            client = client,
            output = True, permitir_fallos = False)
    
    df['DATE'], df_factores_PE['DATE'] = pd.to_datetime(df['DATE']), pd.to_datetime(df_factores_PE['DATE'])
    
    # Corrección con factores PE cuando corresponde
    df = df.merge(df_factores_PE, on = ['PAIS', 'DATE', 'CANAL', 'FUENTE'], how = 'left')
    df['FACTOR'] = df['FACTOR'].fillna(1)
    df['VENTA_COLOCADA'] = df['VENTA_COLOCADA'] * df['FACTOR']
    df = df.drop(columns = ['FACTOR'])
    return df

def list_to_gcp(lista):
    s = '('
    for i in lista:
        s += f'"{i}",'
    s = s[:-1] + ')'
    return s

In [896]:
diccionario_dimensiones = {'TOTALES': '', 'F': 'FAMILIA', 'LT': campo_last_touch, 'SF': 'SUBFAMILIA', 'F1': 'F1', 'G1': 'G1'}  

In [897]:
# nuevo módulo
dir_data_back = f'{dir_data}Back/'
print(dir_data_back) # Donde almacenar

df_representantes_base = pd.DataFrame(columns = ['DUPLICACION', 'PAIS', 'DATE'])
if 'P1_df_all_data.pkl' in os.listdir(dir_data_back):
    df_representantes_base = tf.parquet_act(f'{dir_data_back}P1_df_all_data')
df_representantes_base

../Data/Back/


,DUPLICACION,PAIS,DATE,CANAL,FUENTE,FAMILIA,SUBFAMILIA,VENTA_COLOCADA,ORDENES,VISITAS,TIPO_MEDIO,F1,G1
0,F-SF,CL,2023-12-03,APP,ES,27 - Organizacion,10 - Bolsas De Compra,0.0,0.0,0.0,None,None,None
1,F-SF,CL,2023-12-05,APP,SIS,27 - Organizacion,NO_CLACOMIZADA,0.0,0.0,873.0,None,None,None
2,F-SF,CL,2023-12-06,APP,SIS,27 - Organizacion,NO_CLACOMIZADA,0.0,0.0,791.0,None,None,None
3,F-SF,CL,2023-12-07,APP,SIS,27 - Organizacion,NO_CLACOMIZADA,8990.0,1.0,712.0,None,None,None
4,F-SF,CL,2023-12-08,APP,SIS,27 - Organizacion,NO_CLACOMIZADA,89520.0,4.0,1165.0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8249444,G1,PE,2024-07-15,WEB,SODIMAC,None,None,195778.6,791.0,92552.0,None,None,G21 - Hogar
8249445,G1,PE,2024-07-15,WEB,SODIMAC,None,None,35376.4,139.0,44804.0,None,None,G02 - Jardineria
8249446,G1,PE,2024-07-15,WEB,SODIMAC,None,None,170239.6,285.0,57559.0,None,None,G14 - Electrodomesticos
8249447,G1,PE,2024-07-15,WEB,SODIMAC,None,None,0.0,0.0,2058.0,None,None,"G05 - Artesanias, manualidades y costura"


In [898]:
df_representantes['AUX'] = 'aux'
df_representantes = df_representantes.merge(df_paises, on = 'AUX')
df_representantes = df_representantes.drop(columns = ['AUX'])
df_representantes = df_representantes[['PAIS', 'REPRESENTANTE', 'DIA_DESDE', 'DIA_HASTA']]
df_representantes = df_representantes[~((df_representantes['PAIS'].isin(paises_no_andes)) & (df_representantes['REPRESENTANTE'].isin(['F1', 'G1'])))].reset_index(drop = True)
df_representantes

,PAIS,REPRESENTANTE,DIA_DESDE,DIA_HASTA
0,MX,F-SF,2024-01-23,2025-01-22
1,MX,F-LT,2024-01-23,2025-01-22
2,MX,F,2024-01-23,2025-01-22
3,MX,LT,2024-01-23,2025-01-22
4,MX,TOTALES,2024-01-23,2025-01-22


In [899]:
for i in range(len(df_representantes)):
    representante, dia_desde, dia_hasta, pais = df_representantes.loc[i, 'REPRESENTANTE'], df_representantes.loc[i, 'DIA_DESDE'], df_representantes.loc[i, 'DIA_HASTA'], df_representantes.loc[i, 'PAIS']
    print(representante, dia_desde, dia_hasta, pais)
    
    df_dates = pd.date_range(start = dia_desde, end = dia_hasta, freq = 'D')
    df_dates = pd.DataFrame({'DATE': df_dates})
    df_dates['AUX'] = 'aux'
    
    df_pais = pd.DataFrame({'PAIS': [pais]})
    df_pais['AUX'] = 'aux'
    
    df_req = df_dates.merge(df_pais, on = 'AUX')
    df_req = df_req.drop(columns = ['AUX'])
    df_req['DUPLICACION'] = representante

    if not ejecutar[1]['Actualizar_tuplas']:
        df_existente = df_representantes_base[['DUPLICACION', 'PAIS', 'DATE']].drop_duplicates().reset_index(drop = True)
        df_existente['EXISTE'] = True
        
        df_req = df_req.merge(df_existente, on = ['DUPLICACION', 'PAIS', 'DATE'], how = 'left')
        df_req['EXISTE'] = df_req['EXISTE'].fillna(False)
    else:
        df_req['EXISTE'] = False
        
    df_req = df_req[~df_req['EXISTE']]

    if len(df_req) == 0: # Están todos los datos actualizados
        continue
        
    min_date, max_date, paises_ejecucion = str(df_req['DATE'].min())[:10], str(df_req['DATE'].max())[:10], df_req['PAIS'].unique()
    df_ejecucion = request_df_duplicidades(tf, client, tabla_data, representante, paises_ejecucion, min_date, max_date, diccionario_dimensiones, dia_desde, df_factores_PE)
    df_ejecucion['DUPLICACION'] = representante
    df_representantes_base = pd.concat([df_representantes_base, df_ejecucion])
    df_representantes_base['DATE'] = pd.to_datetime(df_representantes_base['DATE'])
    df_representantes_base = df_representantes_base.drop_duplicates(subset = list(set(df_representantes_base.columns) - {'VENTA_COLOCADA', 'ORDENES', 'VISITAS'}), keep = 'last').reset_index(drop = True) # Se mantienen con prioridad los nuevos datos

# Correccion sodimac web en CL y PE
# Se reemplaza sodimac web por sis web para los días que existen, y los periodos donde sodimac web estuvo migrado

df_representantes_base_web_sis_andes = df_representantes_base[(df_representantes_base['PAIS'].isin(paises_andes)) & (df_representantes_base['CANAL'] == 'WEB') & (df_representantes_base['FUENTE'] == 'SIS')]

filtro_migracion_CL = (df_representantes_base_web_sis_andes['PAIS'] == 'CL') & (df_representantes_base_web_sis_andes['DATE'] >= '2022-06-14') & (df_representantes_base_web_sis_andes['DATE'] <= '2024-06-26')
filtro_migracion_PE = (df_representantes_base_web_sis_andes['PAIS'] == 'PE') & (df_representantes_base_web_sis_andes['DATE'] >= '2023-01-23') & (df_representantes_base_web_sis_andes['DATE'] <= '2024-07-15')
df_representantes_base_web_sis_andes = df_representantes_base_web_sis_andes[filtro_migracion_CL | filtro_migracion_PE]
df_representantes_base_web_sis_andes['FUENTE'] = 'SODIMAC'

#df_representantes_base[((df_representantes_base['CANAL'] == 'WEB') & (df_representantes_base['FUENTE'] == 'SODIMAC'))]

filtro_migracion_CL = (df_representantes_base['PAIS'] == 'CL') & (df_representantes_base['DATE'] >= '2022-06-14') & (df_representantes_base['DATE'] <= '2024-06-26')
filtro_migracion_PE = (df_representantes_base['PAIS'] == 'PE') & (df_representantes_base['DATE'] >= '2023-01-23') & (df_representantes_base['DATE'] <= '2024-07-15')
df_sodimac_web = df_representantes_base[((df_representantes_base['CANAL'] == 'WEB') & (df_representantes_base['FUENTE'] == 'SODIMAC') & (filtro_migracion_CL | filtro_migracion_PE))]
df_sodimac_web = df_sodimac_web[['PAIS', 'DATE']].drop_duplicates().reset_index(drop = True) # combinaciones pais-día que estaban originalmente
df_sodimac_web = df_sodimac_web.merge(df_representantes_base_web_sis_andes, on = ['PAIS', 'DATE'], how = 'left')

df_representantes_base = df_representantes_base[~((df_representantes_base['CANAL'] == 'WEB') & (df_representantes_base['FUENTE'] == 'SODIMAC') & (filtro_migracion_CL | filtro_migracion_PE))]
df_representantes_base = pd.concat([df_representantes_base, df_representantes_base_web_sis_andes]).reset_index(drop = True)

tf.parquet_act(f'{dir_data_back}P1_df_all_data', variable = df_representantes_base, mode = 'save') # Respaldo de la nueva data

print('REVISAR: Se está haciendo request en F1 y G1')

F-SF 2024-01-23 2025-01-22 MX
F-LT 2024-01-23 2025-01-22 MX
F 2024-01-23 2025-01-22 MX
LT 2024-01-23 2025-01-22 MX
TOTALES 2024-01-23 2025-01-22 MX
REVISAR: Se está haciendo request en F1 y G1


In [900]:
campos_grano = []
for d in grano.split('-'):
    campos_grano.append(diccionario_dimensiones[d])
campos_grano

['FAMILIA', 'TIPO_MEDIO']

In [901]:
#df_representantes_base = tf.ickle_act(f'{dir_data_back}P1_df_all_data')
df_representantes_base = tf.parquet_act(f'{dir_data_back}P1_df_all_data')
df_representantes_base

,DUPLICACION,PAIS,DATE,CANAL,FUENTE,FAMILIA,SUBFAMILIA,VENTA_COLOCADA,ORDENES,VISITAS,TIPO_MEDIO,F1,G1
0,F-SF,CL,2023-12-03,APP,ES,27 - Organizacion,10 - Bolsas De Compra,0.0,0.0,0.0,None,None,None
1,F-SF,CL,2023-12-05,APP,SIS,27 - Organizacion,NO_CLACOMIZADA,0.0,0.0,873.0,None,None,None
2,F-SF,CL,2023-12-06,APP,SIS,27 - Organizacion,NO_CLACOMIZADA,0.0,0.0,791.0,None,None,None
3,F-SF,CL,2023-12-07,APP,SIS,27 - Organizacion,NO_CLACOMIZADA,8990.0,1.0,712.0,None,None,None
4,F-SF,CL,2023-12-08,APP,SIS,27 - Organizacion,NO_CLACOMIZADA,89520.0,4.0,1165.0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8249444,G1,PE,2024-07-15,WEB,SODIMAC,None,None,195778.6,791.0,92552.0,None,None,G21 - Hogar
8249445,G1,PE,2024-07-15,WEB,SODIMAC,None,None,35376.4,139.0,44804.0,None,None,G02 - Jardineria
8249446,G1,PE,2024-07-15,WEB,SODIMAC,None,None,170239.6,285.0,57559.0,None,None,G14 - Electrodomesticos
8249447,G1,PE,2024-07-15,WEB,SODIMAC,None,None,0.0,0.0,2058.0,None,None,"G05 - Artesanias, manualidades y costura"


In [902]:
df_representantes

,PAIS,REPRESENTANTE,DIA_DESDE,DIA_HASTA
0,MX,F-SF,2024-01-23,2025-01-22
1,MX,F-LT,2024-01-23,2025-01-22
2,MX,F,2024-01-23,2025-01-22
3,MX,LT,2024-01-23,2025-01-22
4,MX,TOTALES,2024-01-23,2025-01-22


In [903]:
df_representantes['DF_REPRESENTATIVO'] = ""

for i in range(len(df_representantes)):
    representante, dia_desde, dia_hasta, pais = df_representantes.loc[i, 'REPRESENTANTE'], df_representantes.loc[i, 'DIA_DESDE'], df_representantes.loc[i, 'DIA_HASTA'], df_representantes.loc[i, 'PAIS']
    dia_desde, dia_hasta = pd.to_datetime(dia_desde), pd.to_datetime(dia_hasta)
    
    df_representativo = df_representantes_base[(df_representantes_base['DUPLICACION'] == representante) & (df_representantes_base['DATE'] >= dia_desde) & (df_representantes_base['DATE'] <= dia_hasta) & (df_representantes_base['PAIS'] == pais)].reset_index(drop = True)
    lista_dimensiones = []
    for d in representante.split('-'):
        if d == "TOTALES":
            break
        lista_dimensiones.append(diccionario_dimensiones[d])
    df_representativo = df_representativo[['PAIS', 'DATE', 'CANAL', 'FUENTE'] + lista_dimensiones + ['VENTA_COLOCADA', 'ORDENES', 'VISITAS']].reset_index(drop = True)
    df_representantes['DF_REPRESENTATIVO'][i] = df_representativo

df_representantes_duplicidades = df_representantes[['REPRESENTANTE', 'DIA_DESDE']]
df_representantes

,PAIS,REPRESENTANTE,DIA_DESDE,DIA_HASTA,DF_REPRESENTATIVO
0,MX,F-SF,2024-01-23,2025-01-22,PAIS DATE CANAL FUENTE ...
1,MX,F-LT,2024-01-23,2025-01-22,PAIS DATE CANAL FUENTE ...
2,MX,F,2024-01-23,2025-01-22,PAIS DATE CANAL FUENTE ...
3,MX,LT,2024-01-23,2025-01-22,PAIS DATE CANAL FUENTE ...
4,MX,TOTALES,2024-01-23,2025-01-22,PAIS DATE CANAL FUENTE VENTA_COLO...


## 1.3. Asignación de día histórico a días futuros

In [904]:
sys.path.insert(0, '')
from dias_comparacion import dias_comparacion 

In [905]:
dia_desde, dia_hasta = df_representantes['DIA_DESDE'][0], df_representantes['DIA_HASTA'][0]
df_dias_predecir = dias_comparacion(paises, dias_proyeccion, df_dias_eventos, [dia_desde, dia_hasta]) # Asigna a cada día de la proyección, uno que ya exista (histórico)
df_dias_predecir

100%|██████████| 378/378 [00:01<00:00, 254.96it/s]


,DATE,DIA_SEMANA,PAIS,EVENTO,DATE_COMPARACION
0,2025-01-01,Wednesday,MX,,2025-01-01
1,2025-01-02,Thursday,MX,,2025-01-02
2,2025-01-03,Friday,MX,,2025-01-03
3,2025-01-04,Saturday,MX,,2025-01-04
4,2025-01-05,Sunday,MX,,2025-01-05
...,...,...,...,...,...
373,2025-12-27,Saturday,MX,,2024-12-28
374,2025-12-28,Sunday,MX,,2024-12-29
375,2025-12-29,Monday,MX,,2024-12-30
376,2025-12-30,Tuesday,MX,,2025-01-07


## 1.4. Parámetros de duplicidad

In [906]:
# Búsqueda de arcos

arcos = []

for nivel in niveles:
    if type(nivel) == str:
        continue
    for duplicacion in niveles[nivel]:
        #print('A', nivel, duplicacion)
        if nivel - 1 not in niveles:
            continue
        for duplicacion_agregada in niveles[nivel - 1]:
            if not es_substring(duplicacion_agregada, duplicacion) and (duplicacion_agregada != "TOTALES"):
                continue
            print(nivel, duplicacion, duplicacion_agregada)
            arcos.append([duplicacion, duplicacion_agregada])

arcos

2 F-SF F
2 F-LT F
2 F-LT LT
1 F TOTALES
1 LT TOTALES


[['F-SF', 'F'],
 ['F-LT', 'F'],
 ['F-LT', 'LT'],
 ['F', 'TOTALES'],
 ['LT', 'TOTALES']]

In [907]:
# Representantes
df_duplicacion_representante = df_tuplas_duplicidades[['DUPLICACION', 'REPRESENTANTE']].drop_duplicates()
df_duplicacion_representante

# crea un diccionario {duplicacion: representante} en una linea
dic_duplicacion_representante = df_duplicacion_representante.set_index('DUPLICACION')['REPRESENTANTE'].to_dict()
dic_duplicacion_representante

{'F-SF': 'F-SF', 'F-LT': 'F-LT', 'F': 'F', 'LT': 'LT', 'TOTALES': 'TOTALES'}

In [908]:
df_representantes

,PAIS,REPRESENTANTE,DIA_DESDE,DIA_HASTA,DF_REPRESENTATIVO
0,MX,F-SF,2024-01-23,2025-01-22,PAIS DATE CANAL FUENTE ...
1,MX,F-LT,2024-01-23,2025-01-22,PAIS DATE CANAL FUENTE ...
2,MX,F,2024-01-23,2025-01-22,PAIS DATE CANAL FUENTE ...
3,MX,LT,2024-01-23,2025-01-22,PAIS DATE CANAL FUENTE ...
4,MX,TOTALES,2024-01-23,2025-01-22,PAIS DATE CANAL FUENTE VENTA_COLO...


In [909]:
dic_dfs_hist = {}
for duplicacion in df_representantes['REPRESENTANTE'].unique():
    df = pd.DataFrame()
    df_representantes_duplicacion = df_representantes[df_representantes['REPRESENTANTE'] == duplicacion].reset_index(drop = True)
    for i in range(len(df_representantes_duplicacion)):
        df = pd.concat([df, df_representantes_duplicacion['DF_REPRESENTATIVO'][i]])
    dic_dfs_hist[duplicacion] = df.reset_index(drop = True)

In [910]:
df_factores_duplicidad = pd.DataFrame()

metricas = ['VENTA_COLOCADA', 'ORDENES', 'VISITAS']

for desagregado, agregado in arcos:
    print(desagregado, agregado)
    representante_desagregado = dic_duplicacion_representante[desagregado]
    representante_agregado = dic_duplicacion_representante[agregado]
    
    df_desagregado = dic_dfs_hist[representante_desagregado] # dfs históricos para ambos casos
    df_agregado = dic_dfs_hist[representante_agregado]
    
    campos_agregacion = ['PAIS', 'DATE', 'CANAL', 'FUENTE'] # campos agregado: intersección de las dimensiones
    for c in agregado.split('-'):
        if c == 'TOTALES':
            continue
        campos_agregacion.append(diccionario_dimensiones[c])
    
    df_desagregado = df_desagregado[campos_agregacion + metricas].groupby(campos_agregacion).sum().reset_index() # se suman las métricas
    
    for m in metricas:
        df_desagregado = df_desagregado.rename(columns = {m: f'{m}_DESAGREGADO'})
        df_agregado = df_agregado.rename(columns = {m: f'{m}_AGREGADO'})
        
    df_parametro = df_agregado.merge(df_desagregado, on = campos_agregacion, how = 'outer') # se unen los dfs
    df_parametro = df_parametro.fillna(0)
    
    for m in metricas:
        if m == 'VENTA_COLOCADA':
            df_parametro[f'f_{m}'] = 1
        else:
            df_parametro[f'f_{m}'] = df_parametro[f'{m}_AGREGADO'] / df_parametro[f'{m}_DESAGREGADO'] # se calcula la fracción
            
        df_parametro[f'f_{m}'] = df_parametro[f'f_{m}'].fillna(1) # 0 / 0 por ejemplo
        df_parametro[f'f_{m}'] = np.where(df_parametro[f'f_{m}'] > 1, 1, df_parametro[f'f_{m}']) # X > 0 / 0 por ejemplo, o casos que tienen problemas de validación de consistencia en Nexus
        
        #df_parametro = df_parametro.drop(columns = [f'{m}_AGREGADO', f'{m}_DESAGREGADO']) # bloqueo, cambio 240925

    df_parametro['AGREGADO'], df_parametro['DESAGREGADO'] = agregado, desagregado    
    df_factores_duplicidad = pd.concat([df_factores_duplicidad, df_parametro])    

F-SF F
F-LT F
F-LT LT
F TOTALES
LT TOTALES


Se asocian finalmente, los factores históricos a los días futuros

In [911]:
df_factores_duplicidad = df_factores_duplicidad.rename(columns = {'DATE': 'DATE_COMPARACION'})
df_dias_predecir_resumen = df_dias_predecir[['PAIS', 'DATE', 'DATE_COMPARACION', 'EVENTO']]
df_dias_predecir_resumen['DATE_COMPARACION'], df_factores_duplicidad['DATE_COMPARACION'] = pd.to_datetime(df_dias_predecir_resumen['DATE_COMPARACION']), pd.to_datetime(df_factores_duplicidad['DATE_COMPARACION'])
df_dias_predecir_resumen = df_dias_predecir_resumen.merge(df_factores_duplicidad, on = ['PAIS', 'DATE_COMPARACION'], how = 'left')
df_dias_predecir_resumen

,PAIS,DATE,DATE_COMPARACION,EVENTO,CANAL,FUENTE,FAMILIA,VENTA_COLOCADA_AGREGADO,ORDENES_AGREGADO,VISITAS_AGREGADO,VENTA_COLOCADA_DESAGREGADO,ORDENES_DESAGREGADO,VISITAS_DESAGREGADO,f_VENTA_COLOCADA,f_ORDENES,f_VISITAS,AGREGADO,DESAGREGADO,TIPO_MEDIO
0,MX,2025-01-01,2025-01-01,,APP,SODIMAC,01 - Madera Y Tableros,109.80,1.0,151.0,109.80,1.0,246.0,1,1.000000,0.613821,F,F-SF,NaN
1,MX,2025-01-01,2025-01-01,,APP,SODIMAC,02 - Puertas/Ventanas/Molduras,1178.00,1.0,428.0,1178.00,1.0,866.0,1,1.000000,0.494226,F,F-SF,NaN
2,MX,2025-01-01,2025-01-01,,APP,SODIMAC,03 - Fierro/Hierro/Acero,814.00,1.0,96.0,814.00,1.0,118.0,1,1.000000,0.813559,F,F-SF,NaN
3,MX,2025-01-01,2025-01-01,,APP,SODIMAC,04 - Tabiqueria/Techumbre/Aislacion,1316.00,1.0,268.0,1316.00,1.0,372.0,1,1.000000,0.720430,F,F-SF,NaN
4,MX,2025-01-01,2025-01-01,,APP,SODIMAC,05 - Obra Gruesa,1404.00,2.0,137.0,1404.00,3.0,200.0,1,0.666667,0.685000,F,F-SF,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49065,MX,2025-12-31,2024-12-31,Festividad,WEB,SODIMAC,NaN,34536.00,8.0,7780.0,34536.00,10.0,16781.0,1,0.800000,0.463620,LT,F-LT,Typed
49066,MX,2025-12-31,2024-12-31,Festividad,APP,SODIMAC,NaN,349682.90,51.0,3356.0,349682.90,65.0,11378.0,1,0.784615,0.294955,TOTALES,F,NaN
49067,MX,2025-12-31,2024-12-31,Festividad,WEB,SODIMAC,NaN,203995.87,53.0,19650.0,203995.87,66.0,41580.0,1,0.803030,0.472583,TOTALES,F,NaN
49068,MX,2025-12-31,2024-12-31,Festividad,APP,SODIMAC,NaN,349682.90,51.0,3356.0,349682.90,51.0,3444.0,1,1.000000,0.974448,TOTALES,LT,NaN


Ordenamiento de campos

In [912]:
orden_campos = ['AGREGADO', 'DESAGREGADO', 'PAIS', 'DATE', 'CANAL', 'FUENTE']

for c in diccionario_dimensiones.values():
    if c in df_dias_predecir_resumen.columns:
        orden_campos.append(c)

for c in metricas:
    orden_campos += [f'{c}_AGREGADO', f'{c}_DESAGREGADO', f'f_{c}']

orden_campos += ['DATE_COMPARACION', 'EVENTO']

df_dias_predecir_resumen = df_dias_predecir_resumen.fillna("")
df_dias_predecir_resumen = df_dias_predecir_resumen[orden_campos]

tf.parquet_act(f'{dir_data}Parametros/P1', variable = df_dias_predecir_resumen, mode = 'save')
df_dias_predecir_resumen # Output final del parámetro gamma

,AGREGADO,DESAGREGADO,PAIS,DATE,CANAL,FUENTE,FAMILIA,TIPO_MEDIO,VENTA_COLOCADA_AGREGADO,VENTA_COLOCADA_DESAGREGADO,f_VENTA_COLOCADA,ORDENES_AGREGADO,ORDENES_DESAGREGADO,f_ORDENES,VISITAS_AGREGADO,VISITAS_DESAGREGADO,f_VISITAS,DATE_COMPARACION,EVENTO
0,F,F-SF,MX,2025-01-01,APP,SODIMAC,01 - Madera Y Tableros,,109.80,109.80,1,1.0,1.0,1.000000,151.0,246.0,0.613821,2025-01-01,
1,F,F-SF,MX,2025-01-01,APP,SODIMAC,02 - Puertas/Ventanas/Molduras,,1178.00,1178.00,1,1.0,1.0,1.000000,428.0,866.0,0.494226,2025-01-01,
2,F,F-SF,MX,2025-01-01,APP,SODIMAC,03 - Fierro/Hierro/Acero,,814.00,814.00,1,1.0,1.0,1.000000,96.0,118.0,0.813559,2025-01-01,
3,F,F-SF,MX,2025-01-01,APP,SODIMAC,04 - Tabiqueria/Techumbre/Aislacion,,1316.00,1316.00,1,1.0,1.0,1.000000,268.0,372.0,0.720430,2025-01-01,
4,F,F-SF,MX,2025-01-01,APP,SODIMAC,05 - Obra Gruesa,,1404.00,1404.00,1,2.0,3.0,0.666667,137.0,200.0,0.685000,2025-01-01,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49065,LT,F-LT,MX,2025-12-31,WEB,SODIMAC,,Typed,34536.00,34536.00,1,8.0,10.0,0.800000,7780.0,16781.0,0.463620,2024-12-31,Festividad
49066,TOTALES,F,MX,2025-12-31,APP,SODIMAC,,,349682.90,349682.90,1,51.0,65.0,0.784615,3356.0,11378.0,0.294955,2024-12-31,Festividad
49067,TOTALES,F,MX,2025-12-31,WEB,SODIMAC,,,203995.87,203995.87,1,53.0,66.0,0.803030,19650.0,41580.0,0.472583,2024-12-31,Festividad
49068,TOTALES,LT,MX,2025-12-31,APP,SODIMAC,,,349682.90,349682.90,1,51.0,51.0,1.000000,3356.0,3444.0,0.974448,2024-12-31,Festividad


# 2. Tasa de conversión y ticket promedio colocado pago (TCP & TPP) [1b]

In [913]:
# Se requiere: M = {TCP, TPP}
# M (alpha, beta), 
# donde alpha = (c1, c2, ..., cn, lt): c1 = familia, c2 = subfamilia, ....., lt = last touch channel
# beta = (pais, periodo, canal, fuente)

# Sugerencia: 
# En general, obtener con neural prophet
# Obtener por defecto una proyección con RLS acotada min =0 0 y max = valor razonable, por ejemplo f * max_hist, donde f es un factor
# Si hay un solo dato, recta horizontal
# Si no existen datos, tomar x defecto un nivel + agregado y ajustar x duplicidad

## 2.1. Tuplas

In [914]:
#df_tuplas_duplicidades_combinaciones_pago = df_tuplas_duplicidades[['DUPLICACION', 'REPRESENTANTE', 'NATURALEZA_MEDIO', 'PAIS']] # a nivel pago, no se discrimina entre libre y presupueso, con tendencias no decrecientes
#df_tuplas_duplicidades_combinaciones_pago = df_tuplas_duplicidades_combinaciones_pago[(df_tuplas_duplicidades_combinaciones_pago['NATURALEZA_MEDIO'] == 'PAGO') & (df_tuplas_duplicidades_combinaciones_pago['DUPLICACION'] == grano)].drop_duplicates().reset_index(drop = True)
#df_tuplas_duplicidades_combinaciones_pago

In [915]:
def request_pago(tf, client, tabla_data, duplicacion, paises, dia_desde, dia_hasta, diccionario_dimensiones, df_factores_PE, co_sin_fcom = True):
    
    # CO sin FCOM
    str_co = ''
    if co_sin_fcom:
        str_co = 'and not (PAIS = "CO" and FUENTE != "SODIMAC")'
    
    # otras dimensiones (que dependen de la duplicacion)
    
    campos_desagregados = []
    str_dimensiones, str_numeros = 'PAIS, DATE, CANAL, FUENTE, ', '1, 2, 3, 4, '
    for i, elemento in enumerate(duplicacion.split('-')):
        if elemento == 'TOTALES':
            break
        str_dimensiones += diccionario_dimensiones[elemento] + ', '
        str_numeros += f'{i + 5}, '
        campos_desagregados.append(diccionario_dimensiones[elemento])
    
    str_dimensiones, str_numeros = str_dimensiones[:-2], str_numeros[:-2]
    
    # paises          
    paises = list_to_gcp(paises)    
    
    # query                                                                                                                                                                                                                                                                                  
    query = f'SELECT {str_dimensiones}, SUM(VENTA_COLOCADA) AS VENTA_COLOCADA, SUM(ORDENES) AS ORDENES, SUM(VISITAS) AS VISITAS, SUM(INVERSION) AS INVERSION FROM `{tabla_data}_{duplicacion}` where PAIS in {paises} {str_co} and FUENTE_DATOS = "REAL" and DATE >= "{dia_desde}" and DATE <= "{dia_hasta}" and NATURALEZA_MEDIO = "Pago" group by {str_numeros} order by 1, 2' # main request para entrenamiento
    
    # request y df
    df =  tf.request_GCP_vnew(
            nombre_tabla = "",
            specific_query = query,
            client = client,
            output = True, permitir_fallos = False)

    df['DATE'] = pd.to_datetime(df['DATE'])

    # Ajuste error de marcaje APP PE SODIMAC
    df = df.merge(df_factores_PE, on = ['PAIS', 'DATE', 'CANAL', 'FUENTE'], how = 'left')
    df['FACTOR'] = df['FACTOR'].fillna(1)
    df['VENTA_COLOCADA'] = df['VENTA_COLOCADA'] * df['FACTOR']
    df = df.drop(columns = ['FACTOR'])
    return df
    

In [916]:
dia_desde, dia_hasta = str(dt.datetime(2021, 4, 5).date()), str(dt.datetime.today().date()) # desde el 2021-04-05, porque UY no tiene data ni el 3 ni el 4 de mayo
dia_desde, dia_hasta 

('2021-04-05', '2025-01-24')

In [917]:
# cambios...idem a par 1 para extracción 

In [918]:
df_representantes

,PAIS,REPRESENTANTE,DIA_DESDE,DIA_HASTA,DF_REPRESENTATIVO
0,MX,F-SF,2024-01-23,2025-01-22,PAIS DATE CANAL FUENTE ...
1,MX,F-LT,2024-01-23,2025-01-22,PAIS DATE CANAL FUENTE ...
2,MX,F,2024-01-23,2025-01-22,PAIS DATE CANAL FUENTE ...
3,MX,LT,2024-01-23,2025-01-22,PAIS DATE CANAL FUENTE ...
4,MX,TOTALES,2024-01-23,2025-01-22,PAIS DATE CANAL FUENTE VENTA_COLO...


In [919]:
df_representantes_grano = df_representantes[df_representantes['REPRESENTANTE'] == grano][['PAIS', 'REPRESENTANTE', 'DIA_HASTA']].reset_index(drop = True)
df_representantes_grano['DIA_DESDE'] = dia_desde
df_representantes_grano = df_representantes_grano[['PAIS', 'REPRESENTANTE', 'DIA_DESDE', 'DIA_HASTA']]
df_representantes_grano

,PAIS,REPRESENTANTE,DIA_DESDE,DIA_HASTA
0,MX,F-LT,2021-04-05,2025-01-22


In [920]:
df_representantes_base = pd.DataFrame(columns = ['PAIS', 'DATE'])
if 'P2_pars2.pkl' in os.listdir(dir_data_back):
    df_representantes_base = tf.parquet_act(f'{dir_data_back}P2_pars2')
    df_representantes_base['DATE'] = pd.to_datetime(df_representantes_base['DATE'])

for i in range(len(df_representantes_grano)):
        
    representante, dia_desde, dia_hasta, pais = df_representantes_grano.loc[i, 'REPRESENTANTE'], df_representantes_grano.loc[i, 'DIA_DESDE'], df_representantes_grano.loc[i, 'DIA_HASTA'], df_representantes_grano.loc[i, 'PAIS']
    print(representante, dia_desde, dia_hasta, pais)
    
    df_dates = pd.date_range(start = dia_desde, end = dia_hasta, freq = 'D')
    df_dates = pd.DataFrame({'DATE': df_dates})
    df_dates['AUX'] = 'aux'
    
    df_pais = pd.DataFrame({'PAIS': [pais]})
    df_pais['AUX'] = 'aux'
    
    df_req = df_dates.merge(df_pais, on = 'AUX')
    df_req = df_req.drop(columns = ['AUX'])
    df_req['DUPLICACION'] = representante
    
    if not ejecutar[2]['Actualizar_tuplas']:
        df_existente = df_representantes_base[['PAIS', 'DATE']].drop_duplicates().reset_index(drop = True)
        df_existente['EXISTE'] = True
        
        df_req = df_req.merge(df_existente, on = ['PAIS', 'DATE'], how = 'left')
        df_req['EXISTE'] = df_req['EXISTE'].fillna(False)
    else:
        df_req['EXISTE'] = False
        
    df_req = df_req[~df_req['EXISTE']]
    
    if len(df_req) == 0: # Están todos los datos actualizados
        continue
    
    min_date, max_date, paises_ejecucion = str(df_req['DATE'].min())[:10], str(df_req['DATE'].max())[:10], df_req['PAIS'].unique()
    df_ejecucion = request_pago(tf, client, tabla_data, duplicacion = grano, paises = paises_ejecucion, dia_desde = min_date, dia_hasta = max_date, diccionario_dimensiones = diccionario_dimensiones, df_factores_PE = df_factores_PE)
    df_representantes_base = pd.concat([df_representantes_base, df_ejecucion])
    df_representantes_base['DATE'] = pd.to_datetime(df_representantes_base['DATE'])
    df_representantes_base = df_representantes_base.drop_duplicates(subset = list(set(df_representantes_base.columns) - {'VENTA_COLOCADA', 'ORDENES', 'VISITAS', 'INVERSION'}), keep = 'last').reset_index(drop = True) # Se mantienen con prioridad los nuevos datos

tf.parquet_act(f'{dir_data_back}P2_pars2', variable = df_representantes_base, mode = 'save') # Respaldo de la nueva data

F-LT 2021-04-05 2025-01-22 MX


In [921]:
df = tf.parquet_act(f'{dir_data_back}P2_pars2') # Toda la data Pago requerida
df['DATE'] = pd.to_datetime(df['DATE'])

df = df.drop_duplicates().reset_index(drop = True)
df_combinaciones = df[['PAIS', 'CANAL', 'FUENTE'] + campos_grano].drop_duplicates().reset_index(drop = True)
df_combinaciones = df_combinaciones.fillna('None')
df_combinaciones = df_combinaciones[df_combinaciones['PAIS'].isin(paises)].reset_index(drop = True)
df_combinaciones.head() # Tuplas requeridas

,PAIS,CANAL,FUENTE,FAMILIA,TIPO_MEDIO
0,MX,WEB,SODIMAC,06 - Plomeria / Gasfiteria,SEM
1,MX,WEB,SODIMAC,16 - Electrohogar,SEM
2,MX,WEB,SODIMAC,22 - Aire Libre,SEM
3,MX,WEB,SODIMAC,20 - Aseo,SEM
4,MX,WEB,SODIMAC,04 - Tabiqueria/Techumbre/Aislacion,Videos


### 2.1.2 Ejecución de tuplas

In [922]:
# Se crea el nombre con el ID

df_combinaciones['NOMBRE'] = ''
for c in ['PAIS', 'CANAL', 'FUENTE'] + campos_grano:
    df_combinaciones['NOMBRE'] += df_combinaciones[c].astype(str) + '_'
df_combinaciones['NOMBRE'] = df_combinaciones['NOMBRE'].apply(lambda x: x[:-1])
df_combinaciones.head()

# Se le agregan las métricas TC y TP a la apertura

df_metricas = pd.DataFrame({'METRICA': ['TC', 'TP']})
df_metricas['AUX'] = 'aux'
df_combinaciones['AUX'] = 'aux'
df_combinaciones = df_combinaciones.merge(df_metricas, on = 'AUX', how = 'left')
df_combinaciones = df_combinaciones.drop(columns = 'AUX')
df_combinaciones

,PAIS,CANAL,FUENTE,FAMILIA,TIPO_MEDIO,NOMBRE,METRICA
0,MX,WEB,SODIMAC,06 - Plomeria / Gasfiteria,SEM,MX_WEB_SODIMAC_06 - Plomeria / Gasfiteria_SEM,TC
1,MX,WEB,SODIMAC,06 - Plomeria / Gasfiteria,SEM,MX_WEB_SODIMAC_06 - Plomeria / Gasfiteria_SEM,TP
2,MX,WEB,SODIMAC,16 - Electrohogar,SEM,MX_WEB_SODIMAC_16 - Electrohogar_SEM,TC
3,MX,WEB,SODIMAC,16 - Electrohogar,SEM,MX_WEB_SODIMAC_16 - Electrohogar_SEM,TP
4,MX,WEB,SODIMAC,22 - Aire Libre,SEM,MX_WEB_SODIMAC_22 - Aire Libre_SEM,TC
...,...,...,...,...,...,...,...
639,MX,WEB,SODIMAC,25 - Promociones - Soporte Tecnico,Prospecting,MX_WEB_SODIMAC_25 - Promociones - Soporte Tecn...,TP
640,MX,WEB,SODIMAC,25 - Promociones - Soporte Tecnico,RRSS - Pago,MX_WEB_SODIMAC_25 - Promociones - Soporte Tecn...,TC
641,MX,WEB,SODIMAC,25 - Promociones - Soporte Tecnico,RRSS - Pago,MX_WEB_SODIMAC_25 - Promociones - Soporte Tecn...,TP
642,MX,WEB,SODIMAC,25 - Promociones - Soporte Tecnico,Display,MX_WEB_SODIMAC_25 - Promociones - Soporte Tecn...,TC


In [923]:
#print('SMALL BATCH')
#df_combinaciones = df_combinaciones[(df_combinaciones.FAMILIA.isin([ '14 - Pisos'])) & (df_combinaciones.CANAL == 'WEB')].reset_index(drop = True)
df_combinaciones.head(7) # Solicitado

,PAIS,CANAL,FUENTE,FAMILIA,TIPO_MEDIO,NOMBRE,METRICA
0,MX,WEB,SODIMAC,06 - Plomeria / Gasfiteria,SEM,MX_WEB_SODIMAC_06 - Plomeria / Gasfiteria_SEM,TC
1,MX,WEB,SODIMAC,06 - Plomeria / Gasfiteria,SEM,MX_WEB_SODIMAC_06 - Plomeria / Gasfiteria_SEM,TP
2,MX,WEB,SODIMAC,16 - Electrohogar,SEM,MX_WEB_SODIMAC_16 - Electrohogar_SEM,TC
3,MX,WEB,SODIMAC,16 - Electrohogar,SEM,MX_WEB_SODIMAC_16 - Electrohogar_SEM,TP
4,MX,WEB,SODIMAC,22 - Aire Libre,SEM,MX_WEB_SODIMAC_22 - Aire Libre_SEM,TC
5,MX,WEB,SODIMAC,22 - Aire Libre,SEM,MX_WEB_SODIMAC_22 - Aire Libre_SEM,TP
6,MX,WEB,SODIMAC,20 - Aseo,SEM,MX_WEB_SODIMAC_20 - Aseo_SEM,TC


In [924]:
if 'P2_df_combinaciones_pago.pkl' in os.listdir(dir_data_back): # Se obtiene el respaldo de iteraciones anteriores (si existe)
    df_combinaciones_resultados = tf.parquet_act(f'{dir_data_back}P2_df_combinaciones_pago')
    df_combinaciones = df_combinaciones.merge(df_combinaciones_resultados[['NOMBRE', 'METRICA', 'N_EPOCHS_ACTUALES', 'BEST_ECM', 'DIC_BEST_PARAMS', 'NUEVA_PROYECCION']], on = ['NOMBRE', 'METRICA'], how = 'left')
    df_combinaciones['N_EPOCHS_ACTUALES'] = df_combinaciones['N_EPOCHS_ACTUALES'].fillna(0)
    df_combinaciones['BEST_ECM'] = df_combinaciones['BEST_ECM'].fillna("")
    df_combinaciones['DIC_BEST_PARAMS'] = df_combinaciones['DIC_BEST_PARAMS'].fillna("")
    df_combinaciones['NUEVA_PROYECCION'] = df_combinaciones['NUEVA_PROYECCION'].fillna(False)

else:
    df_combinaciones['N_EPOCHS_ACTUALES'] = 0
    df_combinaciones['BEST_ECM'] = [{} for _ in range(len(df_combinaciones))] # Diccionario vacío en todos los casos (pd no interpreta bien = {})
    df_combinaciones['DIC_BEST_PARAMS'] = [{} for _ in range(len(df_combinaciones))] # Diccionario vacío en todos los casos (pd no interpreta bien = {})
    df_combinaciones['NUEVA_PROYECCION'] = False
    
    print('Aqui faltan algunas cosas por defecto')
    
df_combinaciones

,PAIS,CANAL,FUENTE,FAMILIA,TIPO_MEDIO,NOMBRE,METRICA,N_EPOCHS_ACTUALES,BEST_ECM,DIC_BEST_PARAMS,NUEVA_PROYECCION
0,MX,WEB,SODIMAC,06 - Plomeria / Gasfiteria,SEM,MX_WEB_SODIMAC_06 - Plomeria / Gasfiteria_SEM,TC,45.0,"{'': 2.492288495823834e-08, 'Hard Sell': 0.0}","{'': {'logistica_0': [56.827301551424114, 192....",False
1,MX,WEB,SODIMAC,06 - Plomeria / Gasfiteria,SEM,MX_WEB_SODIMAC_06 - Plomeria / Gasfiteria_SEM,TP,45.0,"{'': 162560.51422998752, 'Hard Sell': 8.123270...","{'': {'logistica_0': [-37.40034076738345, 192....",False
2,MX,WEB,SODIMAC,16 - Electrohogar,SEM,MX_WEB_SODIMAC_16 - Electrohogar_SEM,TC,45.0,"{'': 1.0577386612646813e-07, 'Hard Sell': 0.0}","{'': {'logistica_0': [7.499293536586776e-07, -...",False
3,MX,WEB,SODIMAC,16 - Electrohogar,SEM,MX_WEB_SODIMAC_16 - Electrohogar_SEM,TP,45.0,"{'': 562609.507064632, 'Hard Sell': 52.5009450...","{'': {'logistica_0': [-70.209471220846, 202.86...",True
4,MX,WEB,SODIMAC,22 - Aire Libre,SEM,MX_WEB_SODIMAC_22 - Aire Libre_SEM,TC,45.0,"{'': 2.420664401998811e-07, 'Hard Sell': 3.715...","{'': {'logistica_0': [1.655352644806558e-07, 1...",False
...,...,...,...,...,...,...,...,...,...,...,...
639,MX,WEB,SODIMAC,25 - Promociones - Soporte Tecnico,Prospecting,MX_WEB_SODIMAC_25 - Promociones - Soporte Tecn...,TP,45.0,"{'': 0.0, 'Hard Sell': 0.0}","{'': {'logistica_0': [0.0, 705.0, 0.0, 0.0]}, ...",False
640,MX,WEB,SODIMAC,25 - Promociones - Soporte Tecnico,RRSS - Pago,MX_WEB_SODIMAC_25 - Promociones - Soporte Tecn...,TC,45.0,"{'': 0.0, 'Hard Sell': 0.0}","{'': {'logistica_0': [0.0, 710.0, 0.0, 0.0]}, ...",False
641,MX,WEB,SODIMAC,25 - Promociones - Soporte Tecnico,RRSS - Pago,MX_WEB_SODIMAC_25 - Promociones - Soporte Tecn...,TP,45.0,"{'': 0.0, 'Hard Sell': 0.0}","{'': {'logistica_0': [0.0, 710.0, 0.0, 0.0]}, ...",False
642,MX,WEB,SODIMAC,25 - Promociones - Soporte Tecnico,Display,MX_WEB_SODIMAC_25 - Promociones - Soporte Tecn...,TC,45.0,"{'Hard Sell': 0.0, '': 0.0}","{'Hard Sell': {'logistica_0': [0.0, 689.0, 0.0...",False


## 2.2 Train & Predict

In [925]:
def inicializar_data(df, df_tupla, df_dias_eventos = pd.DataFrame(), campos_merge = ['PAIS', 'CANAL', 'FUENTE'] + campos_grano):
    
    # Filtra la ata de la tupal correspondiente, agrupa para no tener duplicados y calcula TC & TP
    
    df_tupla['ASIGNAR'] = True
    df_data = df.merge(df_tupla, on = campos_merge, how = 'left')
    df_data['ASIGNAR'] = df_data['ASIGNAR'].fillna(False)
    df_data = df_data[df_data['ASIGNAR']].reset_index(drop = True)
        
    df_data = df_data.drop(columns = ['ASIGNAR'])
    df_tupla = df_tupla.drop(columns = 'ASIGNAR')
        
    df_data = df_data[campos_merge + ['DATE'] + ['VENTA_COLOCADA', 'ORDENES', 'VISITAS']].groupby(campos_merge + ['DATE']).sum().reset_index()
    
    df_data['TP'] = df_data['VENTA_COLOCADA'] / df_data['ORDENES']
    df_data['TC'] = df_data['ORDENES'] / df_data['VISITAS']
    df_data = df_data.drop(columns = ['VENTA_COLOCADA', 'ORDENES', 'VISITAS'])
    
    if len(df_dias_eventos) > 0:
        df_data = df_data.merge(df_dias_eventos, on = ['DATE', 'PAIS'], how = 'left')
        df_data['EVENTO'] = df_data['EVENTO'].fillna('')
    
    return df_data

Cantidad de iteraciones

In [926]:
# hoy menos el 2024-11-14
delta_days_iters = (dt.datetime.today() - dt.datetime(2024, 11, 18)).days
if no_entrenar_nada:
    delta_days_iters = 0
delta_days_iters

67

In [927]:
total_min_epochs = int(0.8 * delta_days_iters) # 250 Parámetro  # 2.5
n_epochs = 5 #50 # Parámetro: cuantos epochs se ejecutan por iteracion (o cuantos random sets train / test se prueban en RLS)
paso = 100 # Parámetro. Paso de guardado

In [928]:
df_combinaciones['N_ITERACIONES_FALTANTES'] = total_min_epochs - df_combinaciones['N_EPOCHS_ACTUALES']
df_combinaciones['N_ITERACIONES_FALTANTES'] = np.where(df_combinaciones['N_ITERACIONES_FALTANTES'] < 0, 0, df_combinaciones['N_ITERACIONES_FALTANTES'])
df_combinaciones['N_ITERS'] = (df_combinaciones['N_ITERACIONES_FALTANTES'] - 1) // n_epochs + 1
iteraciones_totales = int(df_combinaciones['N_ITERS'].sum())

df_combinaciones = df_combinaciones.drop(columns = ['N_ITERACIONES_FALTANTES', 'N_ITERS'])
df_combinaciones = df_combinaciones.fillna("")
#display(df_combinaciones)
print(f'\n\n\n Se ejecutarán un total de {iteraciones_totales} iteraciones')




 Se ejecutarán un total de 1288 iteraciones


Mover estas funciones

In [929]:
from scipy.optimize import curve_fit # importación
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression


def estimar_valores_iniciales(df_train, b0_v, b1_v, n_v, n_iters):
    
    n, A, k, x, b0, b1, c = sp.symbols('n A k x b0 b1 c')

    # Definir las funciones f(x) y h(x)
    f_x = A * x ** n + k
    h_x = b0 + b1 * x

    # Definir la expresión para la diferencia cuadrática entre f(x) y h(x)
    difference_squared = (f_x - h_x) ** 2

    # Calcular la integral desde 0 hasta c sin simplificación automática
    Z = sp.integrate(difference_squared, (x, 0, c), manual = True)

    # Mostrar Z para ver su forma
    #display(Z)
    
    # Funciones

    def A_diff(A, n, k, c, b0, b1): # A simpy le cuesta la resolver la integral polínomica...función desde chat gpt
        """
        Calcula la integral de Z_A respecto a los límites [0, c].
        
        Parámetros:
        A: Coeficiente A de la expresión.
        n: Exponente n de la expresión.
        k: Constante k de la expresión.
        c: Límite superior de integración.
        b0: Coeficiente constante b0.
        b1: Coeficiente lineal b1.
        
        Retorna:
        El valor de la integral evaluada.
        """
        # Calcula cada término de la integral resuelta
        term1 = (2 * A * c**(2 * n + 1)) / (2 * n + 1)
        term2 = -2 * b0 * c**(n + 1) / (n + 1)
        term3 = -2 * b1 * c**(n + 2) / (n + 2)
        term4 = 2 * k * c**(n + 1) / (n + 1)
        
        # Suma todos los términos para obtener el valor de la integral
        integral_value = term1 + term2 + term3 + term4
        return integral_value

    # Definir la función para la derivada de Z con respecto a k
    def k_diff(A, n, k, c, b0, b1):
        """
        Calcula la integral de Z_k respecto a los límites [0, c].
        
        Parámetros:
        A: Coeficiente A de la expresión.
        n: Exponente n de la expresión.
        k: Constante k de la expresión.
        c: Límite superior de integración (debe ser > 0).
        b0: Coeficiente constante b0.
        b1: Coeficiente lineal b1.
        
        Retorna:
        El valor de la integral evaluada.
        """
        # Verificamos si n es igual a -1 para manejar el caso especial
        if n == -1:
            integral_value = 2 * A * sp.log(c) - 2 * b0 * c - b1 * c**2 + 2 * k * c
        else:
            integral_value = (2 * A * c**(n + 1)) / (n + 1) - 2 * b0 * c - b1 * c**2 + 2 * k * c
        
        return integral_value


    A_v, k_v, c_v = 1, 1, df_train['X'].max() # Valores iniciales

    for i in range(n_iters):
        last_result = (A_v, k_v)
        
        # A
        Z_A_derivative = sp.diff(Z, A)
        resultado_A = A_diff(A = A, n = n, k = k, c = c, b0 = b0, b1 = b1) # Ejemplo de uso (resultado de derivar Z con respecto a A)
        A_solution = sp.solve(resultado_A, A)[0] # Resolver la ecuación Z_A_derivative = 0 para A
        A_solution_eval = A_solution.evalf(subs = {n: n_v, k: k_v, c: c_v, b0: b0_v, b1: b1_v})
        A_v = float(A_solution_eval) # Nuevo A_v


        Z_k_derivative = sp.diff(Z, k)
        resultado_k = k_diff(A = A, n = n, k = k, c = c, b0 = b0, b1 = b1) # Ejemplo de uso (resultado de derivar Z con respecto a k)
        k_solution = sp.solve(resultado_k, k)[0] # Resolver la ecuación Z_A_derivative = 0 para k
        k_solution_eval = k_solution.evalf(subs = {n: n_v, A: A_v, c: c_v, b0: b0_v, b1: b1_v})
        k_v = float(k_solution_eval)

        # Criterio de salida
        if np.allclose(last_result, (A_v, k_v)):
            break

    return A_v, k_v

def funcion_inicial_guess(df_train, x_data, y_data, n_iters):
    rlm = LinearRegression()
    rlm.fit(x_data.reshape(-1, 1), y_data)
    # dame los parametros
    b0_v, b1_v = rlm.intercept_, rlm.coef_[0]
    
    n_0 = 0.5
    A_0, k_0 = estimar_valores_iniciales(df_train, b0_v, b1_v, n_0, n_iters)
    # Utilizamos curve_fit para encontrar los mejores parámetros
    initial_guess = [A_0, n_0, k_0]  # Valores iniciales para A, n y k
    return initial_guess

def curve_fitting_general(df, n_cross_validation, d_0, dia_predecir_desde, dia_predecir_hasta, funcion, best_ecm, inicial_guess, n_iters_actual, info_eventos, title = "", bounds = [], maxfev = 1000, test_size = 0.2, metricas_x = [], metrica_y = "", mostrar_grafico = True, alpha =  0.5, lista_filtrar_metrica_positiva = [], funcion_inicial_guess = [False, None, 0], output = True, hacer_prediccion = True):
    
    """
    Realiza el ajuste de una curva a partir de un conjunto de datos históricos mediante validación cruzada, y genera predicciones.

    **Inputs:**

    - **df** (`pd.DataFrame`): El DataFrame que contiene los datos históricos.
    - **n_cross_validation** (`int`): Número de iteraciones para la validación cruzada.
    - **d_0** (`pd.Timestamp` o `str`): Fecha inicial desde la cual se empieza a contar.
    - **dia_predecir_desde** (`pd.Timestamp` o `str`): Fecha desde la cual se desea empezar a predecir.
    - **dia_predecir_hasta** (`pd.Timestamp` o `str`): Fecha hasta la cual se desea realizar la predicción.
    - **funcion** (`callable`): Función matemática que se ajustará a los datos.
    - **best_ecm** (`float`): Valor del mejor Error Cuadrático Medio (ECM) que se ha encontrado.
    - **inicial_guess** (`list`): Lista con los valores iniciales de los parámetros a ajustar.
    - **n_iters_actual** (`int`): Número de iteraciones ejecutadas hasta el momento.
    - **title** (`str`, opcional): Título para el gráfico que se mostrará.
    - **bounds** (`list`, opcional): Límites de los parámetros para el ajuste de la curva. Si no se proporcionan, se asumen infinitos.
    - **maxfev** (`int`, opcional, default: 1000): Número máximo de evaluaciones de la función permitidas durante el ajuste.
    - **test_size** (`float`, opcional, default: 0.2): Proporción del conjunto de datos que se utilizará para pruebas durante la validación cruzada.
    - **metricas_x** (`list`): Lista de métricas que se usarán como variables independientes en el modelo.
    - **metrica_y** (`str`): Nombre de la métrica dependiente que se desea predecir.
    - **mostrar_grafico** (`bool`, opcional, default: True): Si es `True`, se muestra un gráfico con el ajuste de la curva.
    - **alpha** (`float`, opcional, default: 0.5): Factor de ajuste para establecer los límites superior e inferior de las predicciones.
    - **lista_filtrar_metrica_positiva** (`list`, opcional): Lista de métricas para las cuales se filtrarán los casos con valores positivos (por ejemplo, para eliminar casos sin inversión).
    - **funcion_inicial_guess** (`list`, opcional): Lista con un booleano indicando si se debe calcular una conjetura inicial, la función que lo hará, n_iters_inicial_guess (Número de iteraciones para calcular una conjetura inicial)
    - **output** (`bool`, opcional, default: True): Si es `True`, imprime los resultados en la consola.
    - **hacer_prediccion** (`bool`, opcional, default: True): Si es `True`, genera las predicciones con los mejores parámetros obtenidos del ajuste.

    **Returns:**
    - **df_prediccion** (`pd.DataFrame`): DataFrame que contiene las predicciones generadas en el rango de fechas especificado.
    - **best_params** (`list`): Lista de los mejores parámetros ajustados por la función.
    - **best_ecm** (`float`): Mejor valor del Error Cuadrático Medio (ECM) encontrado durante el ajuste.
    - **n_iters_actual** (`int`): Número total de iteraciones ejecutadas (incluyendo las de validación cruzada).
    """
    
    nueva_proyeccion = False
    
    [df_data_metrica_evento, evento] = info_eventos
    
    #display('df_data_metrica_evento 0', df_data_metrica_evento)
    
    con_inicial_guess = True
    if all(np.isnan(elemento) for elemento in inicial_guess): # Si todos son nan
        con_inicial_guess = False
    
    if bounds == []:
        bounds = ([-float('inf')] * len(inicial_guess), [float('inf')] * len(inicial_guess))
    
    if metrica_y == "": # df solo con los campos date y metrica
        metrica_y = list(set(df.columns) - {'DATE'})[0]
    
    # crea df_dates, desde d_0 a d_f(desde que existe la data hasta el último día de predicción)
    df_dates = pd.date_range(d_0, dia_predecir_desde - dt.timedelta(days = 1))
    df_dates = pd.DataFrame({'DATE': df_dates})
    df = df_dates.merge(df, how = 'left', on = 'DATE')
    df = df.merge(df_data_metrica_evento, how = 'left', on = 'DATE')
    df['EVENTO'] = df['EVENTO'].fillna("")
    df[metrica_y] = np.where((df[metrica_y].isna()) & (df['EVENTO'] == evento), 0, df[metrica_y])
    df = df[df[metrica_y].notna()].reset_index(drop = True)
    
    df[metrica_y] = np.where(df[metrica_y] < -999999999, 0, df[metrica_y]) # Corrección de valores extremos
    df[metrica_y] = np.where(df[metrica_y] > 999999999, 0, df[metrica_y]) # Corrección de valores extremos
    
    df['DATE'] = pd.to_datetime(df['DATE'])
    d_0 = pd.to_datetime(d_0)
    
    df['t'] = (df['DATE'] - d_0).dt.days
    
    y_min, y_max = (1 - alpha) * df[metrica_y].min(), (1 + alpha) *  df[metrica_y].max()
    
    for m in lista_filtrar_metrica_positiva: # df = df[df['INVERSION'] > 0] # Evita casos sin inversión que podrían tener tráfico
        df = df[df[m] > 0]
        
    if funcion_inicial_guess[0] and (not con_inicial_guess): # funcion_inicial_guess = [realizar o no, funcion, n_iters_inicial_guess]
        
        dic_arrays = {}
        for m in metricas_x:
            dic_arrays[m] = np.array(df[m])
        X = np.vstack([dic_arrays[m] for m in metricas_x])
        Y = np.array(df[metrica_y]) # Valores observados de Y
        inicial_guess = funcion_inicial_guess[1](df, X, Y, funcion_inicial_guess[2])
        con_inicial_guess = True
    
    # Entrenar
    
    best_params = inicial_guess # Por defecto, si nunca entra al guess, se mantienen
    for _ in range(n_cross_validation):
        
        try:
            df_train, df_test = train_test_split(df, test_size = test_size)
        except:
            break # Si hay muy pocos casos, queda en nan y posteriormente = 0
        
        df_train = df_train.sort_values('DATE').reset_index(drop = True)
        df_test = df_test.sort_values('DATE').reset_index(drop = True)
        

        dic_arrays = {}
        for m in metricas_x:
            dic_arrays[m] = np.array(df_train[m])

        X = np.vstack([dic_arrays[m] for m in metricas_x])
        if len(metricas_x) == 1:
            X = X[0]
        Y = np.array(df_train[metrica_y])  # Valores observados de z

        # Ajustar la función
        
        if con_inicial_guess:
            p0 = inicial_guess
        else:
            p0 = None
        
        maxfev_aux = maxfev
        n_intentos = 4
        
        #print('metricas_x', metricas_x)
        #df_train.to_csv('df_train_rev.csv', index = False, sep = ';', decimal = ',')
        
        for i in range(n_intentos):
            #print('intento', i)
            if i == n_intentos - 1:
                #print('Salida. Óptimo No encontrado', evento, X.shape, Y.shape, maxfev_aux)
                popt = inicial_guess
                break
            try:
                popt, pcov = curve_fit(funcion, X, Y, p0 = p0, bounds = bounds, maxfev = maxfev_aux)
                break
            except RuntimeError:
                maxfev_aux = maxfev_aux * 2
               
        # Resultados del ajuste
        lista_x = []
        for m in metricas_x:
            lista_x.append(m)
        
        df_test = df_test[df_test['EVENTO'] == evento].reset_index(drop = True)
        
        df_test[f'{metrica_y}_PREDICCION'] = df_test.apply(lambda x: funcion(*[x[m] for m in lista_x], *popt), axis = 1)
        df_test[f'{metrica_y}_PREDICCION'] = np.where(df_test[f'{metrica_y}_PREDICCION'] < y_min, y_min, df_test[f'{metrica_y}_PREDICCION']) # Correcciones con y_min e y_max
        df_test[f'{metrica_y}_PREDICCION'] = np.where(df_test[f'{metrica_y}_PREDICCION'] > y_max, y_max, df_test[f'{metrica_y}_PREDICCION'])

        ecm = np.mean((df_test[metrica_y] - df_test[f'{metrica_y}_PREDICCION']) ** 2)
        
        # Se evalúa si es mejor
        if ecm < best_ecm:
            nueva_proyeccion = True
            best_ecm = ecm
            best_params = popt
    
    if output:
        # Uso de inspect para obtener los nombres de los argumentos
        arg_names = list(inspect.signature(funcion).parameters.keys())
        print('Mejores parámetros:', end = ' ')
        for i, p in enumerate(best_params):
            print(f"{arg_names[i + len(lista_x)]}: {round(p, 3)}", end = ' | ')

        print(f'Mejor ECM: {round(best_ecm, 7)}')
        
    df_prediccion = pd.DataFrame()
    if hacer_prediccion:
        
        # Df completo al final, para predicción
        df_all_dates = pd.date_range(d_0, dia_predecir_hasta)
        df_all_dates = pd.DataFrame({'DATE': df_all_dates})
        df_prediccion = df_all_dates.merge(df, how = 'left', on = 'DATE')
        df_prediccion['t'] = (df_prediccion['DATE'] - d_0).dt.days
        
        lista_x = []
        for m in metricas_x:
            lista_x.append(m)
        
        if 'EVENTO' in df_prediccion.columns:
            df_prediccion = df_prediccion.drop(columns = ['EVENTO'])
        
        df_prediccion = df_prediccion.merge(df_data_metrica_evento, how = 'left', on = 'DATE')
        df_prediccion['EVENTO'] = df_prediccion['EVENTO'].fillna("")
        df_prediccion = df_prediccion[df_prediccion['EVENTO'] == evento].reset_index(drop = True)
        df_prediccion[f'{metrica_y}_PREDICCION'] = df_prediccion.apply(lambda x: funcion(*[x[m] for m in lista_x], *best_params), axis = 1)
        df_prediccion[f'{metrica_y}_PREDICCION'] = np.where(df_prediccion[f'{metrica_y}_PREDICCION'] < y_min, y_min, df_prediccion[f'{metrica_y}_PREDICCION']) # Correcciones con y_min e y_max
        df_prediccion[f'{metrica_y}_PREDICCION'] = np.where(df_prediccion[f'{metrica_y}_PREDICCION'] > y_max, y_max, df_prediccion[f'{metrica_y}_PREDICCION'])

    if mostrar_grafico:
        for m in metricas_x:
            dic_arrays[m] = np.array(df[m])
        X = np.vstack([dic_arrays[m] for m in metricas_x])
        if len(metricas_x) == 1:
            X = X[0]
        Y = np.array(df[metrica_y]) # Valores observados de Y
        X, Y = X.round(4), Y.round(4)

        plt.scatter(X, Y, label = 'Datos originales')
        plt.plot(X, funcion(X, *best_params).round(4), color = 'red', label = 'Ajuste de la curva')
        plt.xlabel('x')
        plt.ylabel('y')
        plt.legend()
        plt.title(title)
        plt.show()
    
    n_iters_actual += n_cross_validation
    return df_prediccion, best_params, best_ecm, n_iters_actual, nueva_proyeccion

In [930]:
def plotear_all(df_all, metrica_y, mostrar_grafico = True, title = ""):
    
    if not mostrar_grafico:
        return None
    
    print('T', title)
    colores = ['salmon', 'goldenrod', 'cadetblue', 'indigo', 'gray', 'cyan']
    

    plt.figure(figsize = (10, 5))
    for i, evento in enumerate(df_all['EVENTO'].unique()):
        
        df_evento = df_all[df_all['EVENTO'] == evento].reset_index(drop = True)
        d_0, d_f = df_evento['DATE'].min(), df_evento['DATE'].max()
        df_dates = pd.date_range(d_0, d_f)
        df_dates = pd.DataFrame({'DATE': df_dates})
        df_evento = df_dates.merge(df_evento, how = 'left', on = 'DATE')
        if evento == "":
            evento = "Sin evento"
        plt.plot(df_evento['DATE'], df_evento[f'{metrica_y}_PREDICCION'], color = colores[i], label = evento, lw = 2)
        plt.plot(df_evento['DATE'], df_evento[metrica_y], 'o', color = colores[i], markersize = 0.5)
    
    plt.title(title)  
    plt.xlabel('Date')
    plt.ylabel(metrica_y)
    plt.legend()
    
    plt.show()
        
    return None

In [931]:
# Generaliza y define las funciones
def logistics_function(x, k, x0, y_inf, y_sup):
    return y_inf + (y_sup - y_inf) / (1 + np.exp(-k * (x - x0)))

dict_funciones = {'logistica_0': logistics_function}

dia_predecir_desde = dt.datetime(2024, 9, 1).date() # Parámetros
dia_predecir_hasta = dt.datetime(2025, 12, 31).date()
test_size = 0.2 # Parámetro
alpha = 0.3
maxfev = 5000
mostrar_grafico = False

d_0 = dt.datetime(2021, 4, 5).date() # Parámetro

In [932]:
#dates_extremos = [df['DATE'].min(), dias_proyeccion[1]] 
t_max = 1.5 # tiempo máximo de ejecución en horas
# La función depende de la complejidad
nombre_funcion = 'logistica_0' # Parámetro
funcion = dict_funciones[nombre_funcion]
df_all_new = pd.DataFrame()
d_0 = dt.datetime(2021, 1, 1).date()

t0 = time.time()

for i in tqdm.tqdm(range(iteraciones_totales)):
    
    if not ejecutar[2]['Entrenar']:
        break
    
    #display(df_combinaciones.head(3))
    
    if ((i + 1) % paso == 0): # Guardar
        print('Guardar')
        df_combinaciones_pago_base = pd.DataFrame()
        if 'P2_df_combinaciones_pago.pkl' in os.listdir(dir_data_back):
            df_combinaciones_pago_base = tf.parquet_act(f'{dir_data_back}P2_df_combinaciones_pago')
        
        df_combinaciones_pago_base = pd.concat([df_combinaciones, df_combinaciones_pago_base], axis = 0) # Se priorizan los datos nuevos
        df_combinaciones_pago_base = df_combinaciones_pago_base.drop_duplicates(subset = ['NOMBRE', 'METRICA']).reset_index(drop = True)
        tf.parquet_act(f'{dir_data_back}P2_df_combinaciones_pago', df_combinaciones_pago_base, 'save')
        
        df_all_new_base = pd.DataFrame()
        if 'P2_df_all_new_par2.pkl' in os.listdir(dir_data_back):
            df_all_new_base = tf.parquet_act(f'{dir_data_back}P2_df_all_new_par2')
        
        df_all_new_base = pd.concat([df_all_new, df_all_new_base], axis = 0)
        df_all_new_base['DATE'] = pd.to_datetime(df_all_new_base['DATE'])
        df_all_new_base = df_all_new_base.drop_duplicates(subset = ['NOMBRE', 'METRICA', 'DATE']).reset_index(drop = True)
        tf.parquet_act(f'{dir_data_back}P2_df_all_new_par2', df_all_new_base, 'save')
        df_all_new = pd.DataFrame()
        
        if (time.time() - t0) > 3600 * t_max: 
            print(f'Salida por t_max: {t_max} hrs')
            break
            
    df_combinaciones = df_combinaciones.sort_values(by = ['N_EPOCHS_ACTUALES', 'NOMBRE']).reset_index(drop = True) # Ordenamiento
    df_tupla_metrica = df_combinaciones.iloc[0:1].reset_index(drop = True) # Selección de tupla
    n_epochs_actuales = df_tupla_metrica['N_EPOCHS_ACTUALES'][0] 
    dic_best_ecm, dic_best_params = df_tupla_metrica['BEST_ECM'][0], df_tupla_metrica['DIC_BEST_PARAMS'][0]
    
    if dic_best_ecm == "":
        dic_best_ecm = {}
    if dic_best_params == "":
        dic_best_params = {}
    
    df_data = inicializar_data(df, df_tupla_metrica, df_dias_eventos) # datos históricos asociados a la tupla en particular
    df_data = df_data.fillna(0) # Nuevo
    
    #display(df_data)

    metrica = df_tupla_metrica['METRICA'][0]
    df_data_metrica = df_data[['DATE', metrica, 'EVENTO']]
    df_data_metrica = df_data_metrica[df_data_metrica[metrica].notna()].reset_index(drop = True) # sin denominador, la métrica se considera nula (X/0) y simplemente es un dato no existente en el entrenamiento
    df_data_metrica = df_data_metrica[df_data_metrica[metrica] < 10 ** 20].reset_index(drop = True) # filtra casos con infinitos (que eventualmente podrían aparecer)
    nombre = df_tupla_metrica['NOMBRE'][0]
    

    pais = df_tupla_metrica['PAIS'][0]
    df_dias_eventos_pais = df_dias_eventos[df_dias_eventos['PAIS'] == pais][['DATE', 'EVENTO']]

    num_pars = len(inspect.signature(logistics_function).parameters) - 1 # Ese - 1 corresponde al x # Uso de inspect para contar los argumentos

    df_all = pd.DataFrame()
    nueva_proyeccion_all = False

    x0_0 = (df_data_metrica['DATE'].max().date() - d_0).days / 2
    
    for evento in df_data_metrica['EVENTO'].unique():
        df_data_metrica_evento = df_data_metrica[df_data_metrica['EVENTO'] == evento].reset_index(drop = True)[['DATE', metrica]]
        
        # Mejor ecm
        if evento not in dic_best_ecm.keys():
            best_ecm = float('inf')
        else:
            best_ecm = dic_best_ecm[evento]
        
        if best_ecm == None:
            best_ecm = float('inf')
        
        # Mejor parámetro
        
        y_sup_0 = df_data_metrica_evento[metrica].max()
        y_min_0 = 0
        if len(df_data_metrica_evento[df_data_metrica_evento[metrica] > 0]) > 0:
            y_min_0 = df_data_metrica_evento[df_data_metrica_evento[metrica] > 0][metrica].min()
     
        if evento not in dic_best_params.keys(): # Si no existe el evento
            inicial_guess = [0, x0_0, y_min_0, y_sup_0] #[[np.nan] * num_pars ] # cambio el 241202
        elif (dic_best_params[evento] == None) or (nombre_funcion not in dic_best_params[evento].keys()): # O no existe la función dentro del evento
            inicial_guess = [0, x0_0, y_min_0, y_sup_0] # [np.nan] * num_pars #Entonces Inicial guess = None
        else:
            inicial_guess = dic_best_params[evento][nombre_funcion] # cambio el 241202
        
        df_prediccion, best_params, best_ecm, n_iters_actual, nueva_proyeccion = curve_fitting_general(df = df_data_metrica_evento, n_cross_validation = n_epochs, 
                                                                            d_0 = d_0, 
                                                                            dia_predecir_desde = dia_predecir_desde, 
                                                                            dia_predecir_hasta = dia_predecir_hasta, 
                                                                            funcion = funcion, best_ecm = best_ecm, 
                                                                            inicial_guess = inicial_guess, 
                                                                            n_iters_actual = n_epochs_actuales, info_eventos = [df_dias_eventos_pais, evento], test_size = test_size, 
                                                                            metricas_x = ['t'], metrica_y = metrica, mostrar_grafico = False, alpha = alpha, 
                                                                            output = False, maxfev = maxfev,
                                                                            hacer_prediccion = True)
        
        nueva_proyeccion_all = nueva_proyeccion_all or nueva_proyeccion
        ## Guardar 
        # Mejor ecm
        dic_best_ecm[evento] = best_ecm
        
        # Mejor parámetro
        if evento not in dic_best_params.keys():
            dic_best_params[evento] = {}
        if dic_best_params[evento] == None:
            dic_best_params[evento] = {}
        dic_best_params[evento][nombre_funcion] = best_params
            
        df_all = pd.concat([df_all, df_prediccion], axis = 0)
    
    df_combinaciones['BEST_ECM'][0], df_combinaciones['DIC_BEST_PARAMS'][0] = dic_best_ecm, dic_best_params
    df_combinaciones['N_EPOCHS_ACTUALES'][0] = n_iters_actual
    df_combinaciones['NUEVA_PROYECCION'][0] = nueva_proyeccion_all
    
    plotear_all(df_all, metrica, mostrar_grafico = mostrar_grafico, title = nombre.replace("_", " "))
        
    df_all = df_all[['DATE', 'EVENTO', metrica, f'{metrica}_PREDICCION']].rename(columns = {metrica: 'REAL', f'{metrica}_PREDICCION': 'PREDICCION'})
    df_all['NOMBRE'] = nombre
    df_all['METRICA'] = metrica
    df_all = df_all[['NOMBRE', 'METRICA', 'DATE', 'EVENTO', 'REAL', 'PREDICCION']]
    df_all_new = pd.concat([df_all, df_all_new], axis = 0)

if 'DATE' in df_all_new.columns:
    df_all_new['DATE'] = pd.to_datetime(df_all_new['DATE'])
df_all_new = df_all_new.drop_duplicates(subset = ['NOMBRE', 'METRICA', 'DATE']).reset_index(drop = True)
    
# Guardado final
df_combinaciones_pago_base = pd.DataFrame()
if 'P2_df_combinaciones_pago.pkl' in os.listdir(dir_data_back):
    df_combinaciones_pago_base = tf.parquet_act(f'{dir_data_back}P2_df_combinaciones_pago')  

df_combinaciones_pago_base = pd.concat([df_combinaciones, df_combinaciones_pago_base], axis = 0) # Se priorizan los datos nuevos
df_combinaciones_pago_base = df_combinaciones_pago_base.drop_duplicates(subset = ['NOMBRE', 'METRICA']).reset_index(drop = True)
tf.parquet_act(f'{dir_data_back}P2_df_combinaciones_pago', df_combinaciones_pago_base, 'save')

df_all_new_base = pd.DataFrame()
if 'P2_df_all_new_par2.pkl' in os.listdir(dir_data_back):
    df_all_new_base = tf.parquet_act(f'{dir_data_back}P2_df_all_new_par2')

df_all_new_base = pd.concat([df_all_new, df_all_new_base], axis = 0)
df_all_new_base['DATE'] = pd.to_datetime(df_all_new_base['DATE'])
df_all_new_base = df_all_new_base.drop_duplicates(subset = ['NOMBRE', 'METRICA', 'DATE']).reset_index(drop = True)
tf.parquet_act(f'{dir_data_back}P2_df_all_new_par2', df_all_new_base, 'save')

  0%|          | 0/1288 [00:00<?, ?it/s]


In [933]:
print('[OK] Falta guardar el df_all')
print('[OK] Falta limpiar arrba todo lo que esté comentado')
print('[OK] Falta cambiar nombre a los pkls...deben empezar con PX')
#sys.exit('Continuar aquí post proyecciones')

[OK] Falta guardar el df_all
[OK] Falta limpiar arrba todo lo que esté comentado
[OK] Falta cambiar nombre a los pkls...deben empezar con PX


## 2.3 Consolidar parámetro

In [934]:
if ejecutar[2]['Consolidar']:
    df_pars2 = tf.parquet_act(f'{dir_data_back}P2_df_all_new_par2')
    df_pars2[['PAIS', 'CANAL', 'FUENTE'] + campos_grano] = df_pars2['NOMBRE'].str.split('_', expand = True)
    df_pars2['VALOR'] = np.where(df_pars2['REAL'].isna(), df_pars2['PREDICCION'], df_pars2['REAL'])
    tf.parquet_act(f'{dir_data}Parametros/P2', df_pars2, 'save')

# 3. Proyecciones orgánicas

In [935]:
#PE_LIBRE_TP_PAGO

In [936]:
# hasta ahora (seún lo desarrollado en el cuadrerno)

# PARAMETRO CUADERNO | PARAMETRO PYTHON
#       4          |           1
#       1b         |           2
#       1a + 2     |           3
#       3          |           4
#       5          |           5

In [937]:
# Nat medio pago igual está disponible solo para generar TC y TP por defecto en el archivo de calibración

## 3.1. Tuplas

In [938]:
df_tuplas = df_tuplas[['PAIS', 'MODO_PROYECCION']]
df_tuplas['AUX'] = 'aux'
df_nat_medio = pd.DataFrame({'NATURALEZA_MEDIO': ['ORGANICO', 'PAGO']})
df_nat_medio['AUX'] = 'aux'
df_tuplas = df_tuplas.merge(df_nat_medio, on = 'AUX', how = 'left')
df_metricas = pd.DataFrame({'METRICA': ['VISITAS', 'TC', 'TP']})

df_metricas['AUX'] = 'aux'
df_tuplas = df_tuplas.merge(df_metricas, on = 'AUX', how = 'left')
df_tuplas = df_tuplas.drop(columns = 'AUX')
df_tuplas['NOMBRE'] = df_tuplas['PAIS'] + '_' + df_tuplas['MODO_PROYECCION'] + '_' + df_tuplas['METRICA'] + '_' + df_tuplas['NATURALEZA_MEDIO']
df_tuplas = df_tuplas[['NOMBRE', 'PAIS', 'MODO_PROYECCION', 'METRICA', 'NATURALEZA_MEDIO']]
df_tuplas = df_tuplas.drop_duplicates().reset_index(drop = True) # solo para asegurarse
df_tuplas = df_tuplas[~((df_tuplas['METRICA'] == 'VISITAS') & (df_tuplas['NATURALEZA_MEDIO'] == 'PAGO'))].reset_index(drop = True) # Se excluyen las visitas pago
df_tuplas

,NOMBRE,PAIS,MODO_PROYECCION,METRICA,NATURALEZA_MEDIO
0,MX_LIBRE_VISITAS_ORGANICO,MX,LIBRE,VISITAS,ORGANICO
1,MX_LIBRE_TC_ORGANICO,MX,LIBRE,TC,ORGANICO
2,MX_LIBRE_TP_ORGANICO,MX,LIBRE,TP,ORGANICO
3,MX_LIBRE_TC_PAGO,MX,LIBRE,TC,PAGO
4,MX_LIBRE_TP_PAGO,MX,LIBRE,TP,PAGO
5,MX_PRESUPUESTO_VISITAS_ORGANICO,MX,PRESUPUESTO,VISITAS,ORGANICO
6,MX_PRESUPUESTO_TC_ORGANICO,MX,PRESUPUESTO,TC,ORGANICO
7,MX_PRESUPUESTO_TP_ORGANICO,MX,PRESUPUESTO,TP,ORGANICO
8,MX_PRESUPUESTO_TC_PAGO,MX,PRESUPUESTO,TC,PAGO
9,MX_PRESUPUESTO_TP_PAGO,MX,PRESUPUESTO,TP,PAGO


In [939]:
if 'P3_df_combinaciones.pkl' in os.listdir(dir_data_back):
    df_combinaciones_resultados = tf.parquet_act(f'{dir_data_back}P3_df_combinaciones')
    df_tuplas = df_tuplas[['PAIS', 'MODO_PROYECCION', 'METRICA', 'NATURALEZA_MEDIO', 'NOMBRE']]
    df_tuplas = df_tuplas.merge(df_combinaciones_resultados[['NOMBRE', 'N_EPOCHS_ACTUALES']], on = ['NOMBRE'], how = 'left')
    df_tuplas['N_EPOCHS_ACTUALES'] = df_tuplas['N_EPOCHS_ACTUALES'].fillna(0)
    #sys.exit('Hacer lógica de df_tuplas_proyeccion_t hasta el final')
else:
    df_tuplas['N_EPOCHS_ACTUALES'] = 0

df_tuplas['N_EPOCHS_ACTUALES'] = df_tuplas['N_EPOCHS_ACTUALES'].astype(int)   

In [940]:
df_tuplas

,PAIS,MODO_PROYECCION,METRICA,NATURALEZA_MEDIO,NOMBRE,N_EPOCHS_ACTUALES
0,MX,LIBRE,VISITAS,ORGANICO,MX_LIBRE_VISITAS_ORGANICO,40800
1,MX,LIBRE,TC,ORGANICO,MX_LIBRE_TC_ORGANICO,40750
2,MX,LIBRE,TP,ORGANICO,MX_LIBRE_TP_ORGANICO,40750
3,MX,LIBRE,TC,PAGO,MX_LIBRE_TC_PAGO,40750
4,MX,LIBRE,TP,PAGO,MX_LIBRE_TP_PAGO,40800
5,MX,PRESUPUESTO,VISITAS,ORGANICO,MX_PRESUPUESTO_VISITAS_ORGANICO,40650
6,MX,PRESUPUESTO,TC,ORGANICO,MX_PRESUPUESTO_TC_ORGANICO,40750
7,MX,PRESUPUESTO,TP,ORGANICO,MX_PRESUPUESTO_TP_ORGANICO,40750
8,MX,PRESUPUESTO,TC,PAGO,MX_PRESUPUESTO_TC_PAGO,40600
9,MX,PRESUPUESTO,TP,PAGO,MX_PRESUPUESTO_TP_PAGO,40600


Cantidad de iteraciones

In [941]:
delta_days_iters

67

In [942]:
total_min_epochs_prophet = 1000 + 200 * delta_days_iters # parametros (2000 + 600 * x)
n_epochs_prophet = 200 # parametros
iteraciones = 2 # cada cuantas iteraciones (de n_epochs) se cambia la tupla

In [943]:
df_tuplas['N_ITERACIONES_FALTANTES'] = total_min_epochs_prophet - df_tuplas['N_EPOCHS_ACTUALES']
df_tuplas['N_ITERACIONES_FALTANTES'] = np.where(df_tuplas['N_ITERACIONES_FALTANTES'] < 0, 0, df_tuplas['N_ITERACIONES_FALTANTES'])
df_tuplas['N_ITERS'] = (df_tuplas['N_ITERACIONES_FALTANTES'] - 1) // (n_epochs_prophet * iteraciones) + 1
iteraciones_totales = df_tuplas['N_ITERS'].sum()

display(df_tuplas)
df_tuplas = df_tuplas.drop(columns = ['N_ITERACIONES_FALTANTES', 'N_ITERS'])

print(f'\n\n\n Se ejecutarán un total de {iteraciones_totales} iteraciones')

,PAIS,MODO_PROYECCION,METRICA,NATURALEZA_MEDIO,NOMBRE,N_EPOCHS_ACTUALES,N_ITERACIONES_FALTANTES,N_ITERS
0,MX,LIBRE,VISITAS,ORGANICO,MX_LIBRE_VISITAS_ORGANICO,40800,0,0
1,MX,LIBRE,TC,ORGANICO,MX_LIBRE_TC_ORGANICO,40750,0,0
2,MX,LIBRE,TP,ORGANICO,MX_LIBRE_TP_ORGANICO,40750,0,0
3,MX,LIBRE,TC,PAGO,MX_LIBRE_TC_PAGO,40750,0,0
4,MX,LIBRE,TP,PAGO,MX_LIBRE_TP_PAGO,40800,0,0
5,MX,PRESUPUESTO,VISITAS,ORGANICO,MX_PRESUPUESTO_VISITAS_ORGANICO,40650,0,0
6,MX,PRESUPUESTO,TC,ORGANICO,MX_PRESUPUESTO_TC_ORGANICO,40750,0,0
7,MX,PRESUPUESTO,TP,ORGANICO,MX_PRESUPUESTO_TP_ORGANICO,40750,0,0
8,MX,PRESUPUESTO,TC,PAGO,MX_PRESUPUESTO_TC_PAGO,40600,0,0
9,MX,PRESUPUESTO,TP,PAGO,MX_PRESUPUESTO_TP_PAGO,40600,0,0





 Se ejecutarán un total de 0 iteraciones


In [944]:
print(total_min_epochs_prophet)
print(df_tuplas['N_EPOCHS_ACTUALES'].min())

14400
40600


In [945]:
df_tuplas

,PAIS,MODO_PROYECCION,METRICA,NATURALEZA_MEDIO,NOMBRE,N_EPOCHS_ACTUALES
0,MX,LIBRE,VISITAS,ORGANICO,MX_LIBRE_VISITAS_ORGANICO,40800
1,MX,LIBRE,TC,ORGANICO,MX_LIBRE_TC_ORGANICO,40750
2,MX,LIBRE,TP,ORGANICO,MX_LIBRE_TP_ORGANICO,40750
3,MX,LIBRE,TC,PAGO,MX_LIBRE_TC_PAGO,40750
4,MX,LIBRE,TP,PAGO,MX_LIBRE_TP_PAGO,40800
5,MX,PRESUPUESTO,VISITAS,ORGANICO,MX_PRESUPUESTO_VISITAS_ORGANICO,40650
6,MX,PRESUPUESTO,TC,ORGANICO,MX_PRESUPUESTO_TC_ORGANICO,40750
7,MX,PRESUPUESTO,TP,ORGANICO,MX_PRESUPUESTO_TP_ORGANICO,40750
8,MX,PRESUPUESTO,TC,PAGO,MX_PRESUPUESTO_TC_PAGO,40600
9,MX,PRESUPUESTO,TP,PAGO,MX_PRESUPUESTO_TP_PAGO,40600


## 3.2. Obtención de data histórica

In [946]:
def request_df_para_prophet(tf, client, tabla_data, duplicacion, paises, dia_desde, dia_hasta, nat_medio, df_factores_PE, co_sin_fcom = True):
    
    #dic_str_nat_medio = {'ORGANICO': '= "Organico"', 'PAGO': '!= "Organico"'}
    dic_str_nat_medio = {'ORGANICO': '!= "Pago"', 'PAGO': '= "Pago"'} # Cambio 240624...los medios propios forman parte del orgánico

    str_nat_medio = dic_str_nat_medio[nat_medio]
    
    # CO sin FCOM
    str_co = ''
    if co_sin_fcom:
        str_co = 'and not (PAIS = "CO" and FUENTE != "SODIMAC")'
    
    # otras dimensiones (que dependen de la duplicacion)
    
    str_dimensiones, str_numeros = 'PAIS, DATE, FUENTE, CANAL', '1, 2, 3, 4'
    
    # paises          
    paises = list_to_gcp(paises)    
    
    dic_campos = {'F': 'FAMILIA', 'SF': 'SUBFAMILIA', 'LT': campo_last_touch}
    
    for i, c in enumerate(duplicacion.split('-')):
        str_dimensiones += ', ' + dic_campos[c] 
        str_numeros += ', ' + str(i + 5)
    
    # query                                                                                                                                                                                                                                                                                  
    query = f'SELECT {str_dimensiones}, SUM(VENTA_COLOCADA) AS VENTA_COLOCADA, SUM(ORDENES) AS ORDENES, SUM(VISITAS) AS VISITAS FROM `{tabla_data}_{duplicacion}` where PAIS in {paises} {str_co} and FUENTE_DATOS = "REAL" and DATE >= "{dia_desde}" and DATE <= "{dia_hasta}" and NATURALEZA_MEDIO {str_nat_medio} group by {str_numeros}' # main request para entrenamiento
        
    # request y df
    df =  tf.request_GCP_vnew(
            nombre_tabla = "",
            specific_query = query,
            client = client,
            output = True, permitir_fallos = False)

    df['DATE'] = pd.to_datetime(df['DATE'])

    # Ajuste error de marcaje APP PE SODIMAC
    df = df.merge(df_factores_PE, on = ['PAIS', 'DATE', 'CANAL', 'FUENTE'], how = 'left')
    df['FACTOR'] = df['FACTOR'].fillna(1)
    df['VENTA_COLOCADA'] = df['VENTA_COLOCADA'] * df['FACTOR']
    df = df.drop(columns = ['FACTOR'])
    
    return df

In [947]:
campos_agregacion, campos_grano

(['PAIS', 'DATE', 'CANAL', 'FUENTE'], ['FAMILIA', 'TIPO_MEDIO'])

In [948]:
# Que necesito? (data historica)
df_representantes = df_paises[['PAIS']]
df_representantes['AUX'] = 'aux'
df_representantes = df_representantes.merge(df_nat_medio, on = 'AUX', how = 'left')
df_representantes = df_representantes.drop(columns = 'AUX')
df_representantes['DIA_DESDE'], df_representantes['DIA_HASTA'] = dia_desde, dia_hasta
display(df_representantes)

,PAIS,NATURALEZA_MEDIO,DIA_DESDE,DIA_HASTA
0,MX,ORGANICO,2021-04-05,2025-01-22
1,MX,PAGO,2021-04-05,2025-01-22


In [949]:
# Que tengo?

df_representantes_base = pd.DataFrame(columns = ['PAIS', 'DATE', 'NATURALEZA_MEDIO'])
if 'P3_df_all_data.pkl' in os.listdir(dir_data_back):
    df_representantes_base = tf.parquet_act(f'{dir_data_back}P3_df_all_data')
    df_representantes_base['DATE'] = pd.to_datetime(df_representantes_base['DATE'])

df_representantes_base

,PAIS,DATE,FUENTE,CANAL,FAMILIA,TIPO_MEDIO,VENTA_COLOCADA,ORDENES,VISITAS,NATURALEZA_MEDIO
0,AR,2021-04-05,SODIMAC,WEB,06 - Plomeria / Gasfiteria,SEO,40062.9,3.0,1632.0,ORGANICO
1,AR,2021-04-05,SODIMAC,WEB,01 - Madera Y Tableros,SEO,12909.0,1.0,1032.0,ORGANICO
2,AR,2021-04-05,SODIMAC,WEB,25 - Promociones - Soporte Tecnico,SEO,0.0,0.0,57.0,ORGANICO
3,AR,2021-04-05,SODIMAC,WEB,20 - Aseo,SEO,3068.0,2.0,867.0,ORGANICO
4,AR,2021-04-05,SODIMAC,WEB,08 - Accesorios Automoviles,SEO,809.0,1.0,576.0,ORGANICO
...,...,...,...,...,...,...,...,...,...,...
6606226,UY,2025-01-22,SODIMAC,WEB,21 - Temporada,SEM - Non Brand,0.0,0.0,6.0,PAGO
6606227,UY,2025-01-22,SODIMAC,WEB,07 - Electricidad,SEM - Non Brand,0.0,0.0,17.0,PAGO
6606228,UY,2025-01-22,SODIMAC,WEB,19 - Decoracion,SEM - Non Brand,1799.0,1.0,33.0,PAGO
6606229,UY,2025-01-22,SODIMAC,WEB,11 - Casa Inteligente,SEM - Non Brand,0.0,0.0,4.0,PAGO


In [950]:
# copiar lógica en p3
actualizar_generar_data_proyeccion = []

for i in range(len(df_representantes)):
    pais, nat_medio, dia_desde, dia_hasta = df_representantes.loc[i, 'PAIS'], df_representantes.loc[i, 'NATURALEZA_MEDIO'], df_representantes.loc[i, 'DIA_DESDE'], df_representantes.loc[i, 'DIA_HASTA']
    print(pais, nat_medio, dia_desde, dia_hasta)
    
    df_dates = pd.date_range(start = dia_desde, end = dia_hasta, freq = 'D')
    df_dates = pd.DataFrame({'DATE': df_dates})
    df_req['DATE'] = pd.to_datetime(df_req['DATE'])
    df_dates['AUX'] = 'aux'
    
    df_pais = pd.DataFrame({'PAIS': [pais]})
    df_pais['AUX'] = 'aux'
    
    df_req = df_dates.merge(df_pais, on = 'AUX')
    df_req = df_req.merge(df_nat_medio, on = 'AUX')
    df_req = df_req.drop(columns = ['AUX'])
    
    df_req['DUPLICACION'] = representante

    if not ejecutar[3]['Actualizar_tuplas']:
        df_existente = df_representantes_base[['PAIS', 'NATURALEZA_MEDIO', 'DATE']].drop_duplicates().reset_index(drop = True)
        df_existente['EXISTE'] = True
        df_existente['DATE'] = pd.to_datetime(df_existente['DATE'])
        df_req = df_req.merge(df_existente, on = ['PAIS', 'NATURALEZA_MEDIO', 'DATE'], how = 'left')
        df_req['EXISTE'] = df_req['EXISTE'].fillna(False)
    else:
        df_req['EXISTE'] = False
        
    df_req = df_req[~df_req['EXISTE']]

    if len(df_req) == 0: # Están todos los datos actualizados
        #print('actualizado')
        continue
    
    if pais not in actualizar_generar_data_proyeccion:
        actualizar_generar_data_proyeccion.append(pais) # Cualquier actualización, integra al país a la lista
    min_date, max_date, paises_ejecucion = str(df_req['DATE'].min())[:10], str(df_req['DATE'].max())[:10], df_req['PAIS'].unique()
    df_detalle_new = request_df_para_prophet(tf, client, tabla_data, grano, paises_ejecucion, min_date, max_date, nat_medio, df_factores_PE)
    
    df_detalle_new['DATE'] = pd.to_datetime(df_detalle_new['DATE']).dt.date
    df_detalle_new['NATURALEZA_MEDIO'] = nat_medio
    df_representantes_base = pd.concat([df_detalle_new, df_representantes_base]).reset_index(drop = True)
    
df_representantes_base['DATE'] = pd.to_datetime(df_representantes_base['DATE'])
df_representantes_base = df_representantes_base.drop_duplicates(subset = campos_agregacion + ['NATURALEZA_MEDIO'] + campos_grano).reset_index(drop = True)
df_representantes_base = df_representantes_base.sort_values(['PAIS', 'DATE', 'NATURALEZA_MEDIO']).reset_index(drop = True)
tf.parquet_act(f'{dir_data_back}P3_df_all_data', df_representantes_base, 'save') # Se guarda resultdao actualizado

df = df_representantes_base[['PAIS', 'DATE', 'NATURALEZA_MEDIO', 'VENTA_COLOCADA', 'ORDENES', 'VISITAS']].groupby(['PAIS', 'DATE', 'NATURALEZA_MEDIO']).sum().reset_index()
df = df.sort_values(['PAIS', 'NATURALEZA_MEDIO', 'DATE']).reset_index(drop = True)
df

MX ORGANICO 2021-04-05 2025-01-22
MX PAGO 2021-04-05 2025-01-22


,PAIS,DATE,NATURALEZA_MEDIO,VENTA_COLOCADA,ORDENES,VISITAS
0,AR,2021-04-05,ORGANICO,1176848.35,119.0,63094.0
1,AR,2021-04-06,ORGANICO,1341773.94,82.0,60123.0
2,AR,2021-04-07,ORGANICO,956327.85,91.0,55360.0
3,AR,2021-04-08,ORGANICO,1052004.61,80.0,56632.0
4,AR,2021-04-09,ORGANICO,1171658.83,86.0,60850.0
...,...,...,...,...,...,...
19441,UY,2025-01-18,PAGO,204361.21,45.0,35165.0
19442,UY,2025-01-19,PAGO,261921.61,48.0,36277.0
19443,UY,2025-01-20,PAGO,292444.54,88.0,31351.0
19444,UY,2025-01-21,PAGO,516920.02,137.0,29663.0


In [951]:
#if 'CL' in paises:
#    print('ejecutar - 3- Actualizar_tuplas dejar en False')
#    sys.exit('Leer los datos de nuevo') 

## 3.3. Clase Proyeccion_t

In [952]:

class Proyeccion_t_new():

    def __init__(self, df_tupla, all_df_hist, carpeta_input, df_dias_eventos, output_prophet, grado_para_RLM, test_size, n_iteraciones_con_split, n_max_sin_split, learning_rate, dir_proy_t, dir_plots, dir_model, predecir_con_todos_los_datos, dir_prophet_plots, actualizar_data_hist = True):
        self.df_tupla = df_tupla
        
        self.nombre = df_tupla['NOMBRE'][0].replace('/', '')
            
        self.pais = df_tupla['PAIS'][0]
        self.dir_plots = dir_plots
        self.dir_model = dir_model
        
        self.metrica_main = df_tupla['METRICA'][0]

        self.variables_inicial_aleatorio(carpeta_input)
        self.best_global_ecm = float('inf')
        
        self.dic_funcion_error = {}
        self.dic_df_funcion_error = {}
        self.dic_df_variables = {}
        self.dic_df_RLM = {}
        self.dic_best_variables = {}
        self.dic_model_export = {}
        self.dic_metrics = {}
        self.n_iteraciones_actuales = df_tupla['N_EPOCHS_ACTUALES'][0]
        self.mejor_modelo_encontrado = False

        self.metrica = df_tupla['METRICA'][0]
        self.existe_alguna_proyeccion = False
        self.existe_alguna_proyeccion_prophet = False
        self.best_ecm = {}
        self.best_B0, self.best_B1 = {}, {}
        
        self.dia_predecir_desde, self.dia_predecir_hasta = '2000-01-01', '3000-01-01' # Parámetros por defecto
        self.nueva_proyeccion = False
        # self.existe_prediccion = False
        
        self.rescatar(tf, dir_proy_t) # Se rescata (todo lo de arriba) [se actualiza con los valores guardados]
        
        # no recuperar (ocupar valores actuales)
        self.df_dias_eventos_pais = df_dias_eventos[df_dias_eventos['PAIS'] == self.pais].reset_index(drop = True)
        self.create_df_holidays() # crea self.df_holidays
        if actualizar_data_hist:
            self.df_hist = all_df_hist
        self.dir_prophet_plots = dir_prophet_plots
        self.output_prophet = output_prophet 
        self.grado_para_RLM = grado_para_RLM # no recuperar
        self.test_size = test_size
        self.n_iteraciones_con_split = n_iteraciones_con_split
        self.n_max_sin_split = n_max_sin_split
        self.learning_rate = learning_rate
        self.dir_proy_t = dir_proy_t
        self.predecir_con_todos_los_datos = predecir_con_todos_los_datos
        self.modo_proyeccion = self.nombre.split('_')[1]
        if self.modo_proyeccion not in ['LIBRE', 'PRESUPUESTO']:
            sys.exit('Error {self.modo_proyeccion} no es válido en self.modo_proyeccion')
        
    def rescatar(self, tf, dir_proy_t):
        if (f'{self.nombre}.pkl' not in os.listdir(dir_proy_t)):
            return None # Si no se encuentra, no se pueden rescatar los atributos
        
        #print('Rescate de objeto')
        # setattr
        
        print(f'Setattr en: {dir_proy_t}{self.nombre}')
        valor_cargado = tf.pickle_act(f'{dir_proy_t}{self.nombre}') # De lo contrario, se lee el objeto guardado y se rescatan sus atributos
        
        for key, value in vars(valor_cargado).items(): # vars contiene los atributo y sus valores como diccionario (str, obj) vars = {'x': valor de x, 'y': valor de y}
            #print('key', key)
            setattr(self, key, value) # setattr(objeto, atributo, valor) -> objeto.atributo = valor, actúa sobre la clase self, recibe un key (str) y un value (obj) y los asigna a la clase como atributos: self.key = value...es similar a usar un globals(), pero en una clase
        
        return None

    def generar_data_proyeccion(self): # Solo cambia self.df_hist
        dic_metricas_and_ratios = {'VISITAS': ['VISITAS'],
                        'TC': ['ORDENES', 'VISITAS'],
                        'TP': ['VENTA_COLOCADA', 'ORDENES']}
        
        #self.metrica_main = self.df_tupla['METRICA'][0].split('_')[0]
        #print('self.metrica_main', self.metrica_main)
        self.df_hist = self.df_hist[['DATE'] + dic_metricas_and_ratios[self.metrica_main]]
        
        if len(dic_metricas_and_ratios[self.metrica_main]) == 2: # SI la métrica = ratio, se genera
            self.df_hist[self.metrica_main] = np.where(self.df_hist[dic_metricas_and_ratios[self.metrica_main][1]] == 0, 0, self.df_hist[dic_metricas_and_ratios[self.metrica_main][0]] / self.df_hist[dic_metricas_and_ratios[self.metrica_main][1]])
        
            self.df_hist = self.df_hist[['DATE', self.metrica_main]]
        
        #print('Data Lista para entrenar Prophet 0753')
        #display(self.df_hist.head(5))
        #display(self.df_hist.tail(5))
        
        return None

    def variables_inicial_aleatorio(self, carpeta_input):
        
        df_variables_inicial = pd.read_csv(f'{carpeta_input}Prophet_config_inicial_neural.csv', sep = ';', decimal = ',')

        df_variables_inicial_aleatorio = df_variables_inicial[df_variables_inicial['PARAMETRO'].isin(['FOURIER_ORDER_YEAR', 'FOURIER_ORDER_WEEK', 'FOURIER_ORDER_MONTH'])]
        df_variables_inicial_aleatorio[['COTA_MIN', 'COTA_MAX']] = df_variables_inicial_aleatorio[['COTA_MIN', 'COTA_MAX']].astype(int)
        df_variables_inicial_aleatorio['ACTUAL'] = df_variables_inicial_aleatorio.apply(lambda row: random.randint(row['COTA_MIN'], row['COTA_MAX']), axis=1)

        df_variables_inicial_no_aleatorio = df_variables_inicial[~df_variables_inicial['PARAMETRO'].isin(['FOURIER_ORDER_YEAR', 'FOURIER_ORDER_WEEK', 'FOURIER_ORDER_MONTH'])]

        self.df_variables_inicial = pd.concat([df_variables_inicial_no_aleatorio, df_variables_inicial_aleatorio]).reset_index(drop = True)
        
        return None
        
    def create_df_holidays(self):
        self.df_holidays = self.df_dias_eventos_pais[['EVENTO', 'DATE']].rename(columns = {'EVENTO': 'event', 'DATE': 'ds'})
        return None
        

    def create_math_model(self):
        dic_names = {}
        var_list = []
        
        if self.grado_para_RLM not in self.dic_df_variables:
            self.dic_df_variables[self.grado_para_RLM] = self.df_variables_inicial
        df_variables = self.dic_df_variables[self.grado_para_RLM]

        for i in range(len(df_variables)):
            parametro, representacion = df_variables['PARAMETRO'][i], df_variables['REPRESENTACION'][i]
            var_list.append(sp.Symbol(representacion))
            dic_names[sp.Symbol(representacion)] = parametro

        df_funcion_error = pd.DataFrame()
        for i in list(itertools.combinations_with_replacement(var_list, self.grado_para_RLM)): # Producto cruz de las variables, para hacer tuplas de tamaño (grado)
            termino = 1
            for g in i:
                termino *= g

            beta = sp.Symbol(f'Beta({termino})') 
            df_new = pd.DataFrame({'TERMINO': [termino], 'BETA': [beta]})
            df_funcion_error = pd.concat([df_funcion_error, df_new])

        funcion_error = (df_funcion_error['TERMINO'] * df_funcion_error['BETA']).sum()
                
        self.dic_funcion_error[self.grado_para_RLM] = funcion_error
        self.dic_df_funcion_error[self.grado_para_RLM] = df_funcion_error
        
        return df_variables
    
    def calculate_derivatives(self, df_variables):
        funcion_error = self.dic_funcion_error[self.grado_para_RLM]
        df_variables['DERIVADA'] = df_variables['REPRESENTACION'].apply(lambda x: sp.diff(funcion_error, sp.Symbol(x)))
        self.dic_df_variables[self.grado_para_RLM] = df_variables
        return df_variables
    
    def inicializar_RLM(self):
        df_funcion_error = self.dic_df_funcion_error[self.grado_para_RLM]
        campos_para_RLM = list(df_funcion_error['TERMINO'].unique())
        
        if self.grado_para_RLM not in self.dic_df_RLM:
            self.dic_df_RLM[self.grado_para_RLM] = pd.DataFrame(columns = campos_para_RLM + ['ECM_real'])
        return None
        
    def eliminar_outliers(self, df_model, dia_predecir_desde, beta):
    
        #print('0 eliminar_outliers')
        
        #print('dia_predecir_desde', dia_predecir_desde, 'beta', beta)
        dia_predecir_desde_datetime = dt.datetime.strptime(str(dia_predecir_desde)[:10], '%Y-%m-%d')

        #display('df_model eliminar_outliers 0',dia_predecir_desde_datetime)
        #display(df_model.head(3))
        df_model_post_predict = df_model[df_model['ds'] >= dia_predecir_desde_datetime].reset_index(drop = True)
        
        #df_model_extra = df_model[df_model['ds'] < dia_predecir_desde].reset_index(drop = True)

        #display('df_model_extra eliminar_outliers 0 [ELIMINAR (y df_model_extra igual )]')
        #display(df_model_extra)
        
        df_model = df_model[df_model['ds'] < dia_predecir_desde_datetime].reset_index(drop = True)
        
        #display('df_model eliminar_outliers 1')
        #display(df_model)
        
        df_model_sin_evento = df_model.merge(self.df_holidays, how = 'left', on = 'ds')
        df_model_sin_evento['event'] = df_model_sin_evento['event'].fillna('Sin evento')

        df_model_sin_evento = df_model_sin_evento[df_model_sin_evento['event'] == 'Sin evento']
            
        df_model_sin_evento = df_model_sin_evento[df_model_sin_evento['y'] >= df_model_sin_evento['y'].quantile(beta)] # Elimina el 100 * beta % menor de los datos (excluyendo eventos)
        df_model_sin_evento = df_model_sin_evento[df_model_sin_evento['y'] <= df_model_sin_evento['y'].quantile(1 - beta)] # Elimina el 100 * beta % mayor de los datos (excluyendo eventos)
        df_model_sin_evento = df_model_sin_evento[['ds']]
        df_model_sin_evento['Incluir'] = True
        
        df_model = df_model.merge(df_model_sin_evento, how = 'left', on = 'ds')
        df_model['Incluir'] = df_model['Incluir'].fillna(False)

        df_model = df_model.merge(self.df_holidays, how = 'left', on = 'ds')
        df_model['event'] = df_model['event'].fillna('Sin evento')
        
        df_model = df_model[(df_model['Incluir']) | (df_model['event'] != 'Sin evento')].reset_index(drop = True) # Todos los Incluir = False & Sin evento -> Se excluyen
        df_model = df_model[['ds', 'y']]
        
        df_model = pd.concat([df_model, df_model_post_predict]).reset_index(drop = True)
        
        return df_model

    def show_x0(self, df_variables, n_iter):   
        
        if self.output_prophet:
            print(f'# {n_iter}:  ', end = '')
        
        for i in range(len(df_variables)):
            v = df_variables['PARAMETRO'][i]
            tipo = df_variables['TIPO'][i]
            if tipo == 'float':
                actual = float(df_variables['ACTUAL'][i])
                value = "{:.3e}".format(actual)
            else:
                actual = int(df_variables['ACTUAL'][i])
                # value tiene que ser actual, con dos digitos siempre, y sin decimales
                value = "{:02d}".format(actual)
            if self.output_prophet:
                print(f"{v}: {value}", end = ' | ')
        
        if self.output_prophet:
            print(f'BEST_ECM: {"{:.3e}".format(self.best_global_ecm)}', end = ' | ')
            
        return None
    
    def run_prophet(self, df_variables):
        
        # Variables, y sus valores
        dic_variables_names = df_variables[['PARAMETRO', 'ACTUAL']].set_index('PARAMETRO').to_dict()['ACTUAL']
        dic_variables_values = df_variables[['REPRESENTACION', 'ACTUAL']].set_index('REPRESENTACION').to_dict()['ACTUAL']
        dic_variables_tipos = df_variables[['PARAMETRO', 'TIPO']].set_index('PARAMETRO').to_dict()['TIPO']
        
        for v in dic_variables_names:
            if dic_variables_tipos[v] == 'int':
                dic_variables_names[v] = int(dic_variables_names[v])
            
        return dic_variables_names, dic_variables_values
    
    def run_epochs(self, dic_variables_names, confidence_level, n_epochs, dia_predecir_hasta):
        
        #print('RUN EPOCHS')
        # confidence_level: intervalo de confianza para neurprophet
        
        # Parámetros sensibilizables definidos por el manto y gradient descent
        n_lags = int(dic_variables_names['n_lags'])
        changepoints_range = dic_variables_names['CHANGEPOINTS_RANGE']
        fourier_order_year = int(dic_variables_names['FOURIER_ORDER_YEAR'])
        fourier_order_week = int(dic_variables_names['FOURIER_ORDER_WEEK'])
        fourier_order_month = int(dic_variables_names['FOURIER_ORDER_MONTH'])
        
        #print('n_epochs', n_epochs)
        
        #print('ELIMINAR')
        #n_lags, changepoints_range, fourier_order_year, fourier_order_week, fourier_order_month = 0, 0.8, 1, 1, 1
        
        boundaries = round((1 - confidence_level) / 2, 2)
        quantiles = [boundaries, 1 - boundaries]
        
        # Model and prediction (inicialización del objeto NeuralProphet)
        model = NeuralProphet(
            changepoints_range = changepoints_range, # Iterar (entre 0 y 1..en que proporcion de la data de entrenamiento (a la izquierda), pueden existir puntos de cambio)
            yearly_seasonality = fourier_order_year, # Iterar
            weekly_seasonality = fourier_order_week, # Iterar
            daily_seasonality = False, # Siempre False (intraday)
            n_lags = n_lags, # Iterar
            epochs = n_epochs,
            quantiles = quantiles)

        if self.caso_simplificado:
            print('self.caso_simplificado')
            model = NeuralProphet(
                changepoints_range = changepoints_range, # Iterar (entre 0 y 1..en que proporcion de la data de entrenamiento (a la izquierda), pueden existir puntos de cambio)
                yearly_seasonality = False, # Iterar
                weekly_seasonality = False, # Iterar
                daily_seasonality = False, # Siempre False (intraday)
                n_lags = n_lags, # Iterar
                epochs = n_epochs,
                quantiles = quantiles)
        
        
        model.set_plotting_backend("matplotlib") # En que infraestructura se ejecutan los gráficos
        if fourier_order_month > 0: # Extra: No existe por defecto
            if self.caso_simplificado:
                None
            else:
                model.add_seasonality(name = "monthly_seasonality", period = 365.25 / 12, fourier_order = fourier_order_month) # Iterar (365.25 / 12: días promedio en un mes)
        
        # Inclusión de eventos
        
        if self.caso_simplificado:
            df_holidays = self.df_holidays.head(1)
            df_holidays['ds'] = df_holidays['ds'].apply(lambda x: x.replace(year = 2000)) # ds es del tipo datetime64[ns]...cambia el año de la fecha por el año 2000

        else:
            df_holidays = self.df_holidays
                  
        lista_eventos = list(df_holidays['event'].unique()) # Tipos de evento distintos (generalmente Hard Sell & Festividad)
        #print('lista_eventos', lista_eventos)
        model.add_events(lista_eventos)
        
        #display(self.df_model_base_0)
        
        self.df_model_base_0['ds'] = pd.to_datetime(self.df_model_base_0['ds']).dt.date
        df_model_base_0 = self.df_model_base_0.drop_duplicates(subset = ['ds'], keep = 'first').reset_index(drop = True) # Solo en caso de que existan días duplicados

        # Integracion de los eventos en df_model
        try:
            df_model = model.create_df_with_events(df_model_base_0, df_holidays)
        except Exception as error:
            error = str(error)
            df_model_base_0.to_csv('df_model_base_0.csv', index = False, sep = ';', decimal = ',')
            df_holidays.to_csv('df_holidays.csv', index = False, sep = ';', decimal = ',')
            print('Exportaciones: df_model_base_0 & df_holidays')
            sys.exit(error)

        # anclar 1...no retornar df_model
        # df_model = df_model[df_model['y'] > 0].reset_index(drop = True) # asegura no negattiidad
        for c in ['y'] + lista_eventos:
            df_model[c] = df_model[c].astype('float64')
            
        # Train & Test Split

        #df_model = df_model[['ds', 'y']]
        for m in ['VENTA_COLOCADA', 'ORDENES', 'VISITAS', 'PAIS']:
            if m in df_model.columns:
                df_model = df_model.drop(columns = m)
                
        #display('df_model rev 241016')
        #display(df_model.head(2))
        
        #display(df_model)       
        if 'NATURALEZA_MEDIO' in df_model.columns:
            df_model = df_model.drop(columns = 'NATURALEZA_MEDIO')
        df_model['EVENTO'] = df_model[list(set(df_model.columns) - {'y', 'ds'})].sum(axis = 1) # si existe cualquier evento o festividad
        
        df_model_evento = df_model[df_model['EVENTO'] > 0].reset_index(drop = True) # separacion
        df_model_no_evento = df_model[df_model['EVENTO'] == 0].reset_index(drop = True)

        # cambio en nueva versión...para cada evento, se hace un train/test split diferente (estratificado)

        train_df, test_df = train_test_split(df_model_no_evento, test_size = self.test_size) # train / test split (inicial, sin evento)
        #train_df.to_csv('Btrain_df.csv', index = False, sep = ';', decimal = ',')
        #test_df.to_csv('Btest_df.csv', index = False, sep = ';', decimal = ',')
        
        for evento in lista_eventos:
            
            try:
                train_df_evento, test_df_evento = train_test_split(df_model_evento[df_model_evento[evento] > 0], test_size = self.test_size) # train / test split (inicial, con evento)
            except:
                train_df_evento, test_df_evento = df_model_evento, pd.DataFrame()
                
            train_df = pd.concat([train_df, train_df_evento]).reset_index(drop = True)
            test_df = pd.concat([test_df, test_df_evento]).reset_index(drop = True)

        train_df['ds'], test_df['ds'] = pd.to_datetime(train_df['ds']).dt.date, pd.to_datetime(test_df['ds']).dt.date   
        train_df = train_df.sort_values(by = 'ds').reset_index(drop = True).drop_duplicates(subset = ['ds']).drop(columns = 'EVENTO')
        test_df = test_df.sort_values(by = 'ds').reset_index(drop = True).drop_duplicates(subset = ['ds']).drop(columns = 'EVENTO')
        
        #print('ELIMINAR (1)')
        #train_df['y'] = np.random.uniform(0, 1, len(train_df)) # Se reemplaza la data de entrenamiento por valores aleatorios

        #print('Aqui caida')
        #display(train_df)
        
        #model.fit(train_df, progress = None, freq = "D")
        
        #display('train_df (no puede entrenarse)')
        #display(train_df)
        
        entrenamiento = True
        try:
            #print(model)
            #train_df.to_csv('E2train_df.csv', index = False, sep = ';', decimal = ',')
            #display(train_df)
            metrics = model.fit(train_df, progress = None, freq = "D") # Métricas de performance
            #metrics.to_csv('Bmetrics.csv', index = False, sep = ';', decimal = ',')
        except:
            sys.exit('Entra al except 0: No hace fit')
            metrics = pd.DataFrame()
            self.model_default = model
            entrenamiento = False
            #train_df['y'] = np.random.uniform(0, 1, len(train_df)) # Se reemplaza la data de entrenamiento por valores aleatorios
            #display('train_df random')
            #display(train_df)
            #model.fit(train_df, progress = None, freq = "D")
               
        df_train_eval = df_model[['ds', 'y']] # Seguridad de que están todas las fechas
        
        df_test_results, forecast = self.hacer_prediccion(tf, model, df_train_eval, dia_predecir_hasta)

        #display('Resultado 2')
        #display(df_test_results)
        
        #aprobado = False # Si en todo el horizonte de predicción, la métrica es >= 0, entonces es una solución candidata (factible)
        #if df_test_results['y'].min() >= 0:
        #    aprobado = True
        
        aprobado = True
        df_test_results['y'] = np.where(df_test_results['y'] < 0, 0, df_test_results['y']) # Al menos 0

        df_test_results['ds'] = pd.to_datetime(df_test_results['ds']).dt.date
        test_df['ds'] = pd.to_datetime(test_df['ds']).dt.date

        df_test_results = df_test_results[['ds', 'y']].rename(columns = {'y': 'y_test_result'})
        test_df = test_df[['ds', 'y']].rename(columns = {'y': 'y_test_real'})
        df_test_results_all = test_df.merge(df_test_results, on = 'ds', how = 'left')
        
        
        #241128 MAL
        #df_test_results_all.to_csv('df_test_results_all.csv', index = False, sep = ';', decimal = ',')
        #sys.exit('Rev 0')
        
        ecm = ((df_test_results_all['y_test_result'] - df_test_results_all['y_test_real']) ** 2).mean()

        if self.output_prophet:
            print('ECM', "{:.3e}".format(ecm), end = ' | ')
            print('R_ECM', round((ecm / self.best_global_ecm), 3))
        
        if aprobado and (ecm < self.best_global_ecm):
            
            if entrenamiento:
                if self.output_prophet:
                    print(f'Mejor modelo encontrado {self.dir_model}{self.nombre}.np')
                model_save(model, f'{self.dir_model}{self.nombre}.np')
            
            #print('Modelo guardado')
            #sys.exit(f'{self.dir_model}{self.nombre}.np')
            #self.best_model = model
            self.best_global_ecm = ecm
            self.dic_best_variables[self.grado_para_RLM] = self.dic_df_variables[self.grado_para_RLM]
            
            model_export = self.crear_model_export(df_model)
            self.dic_model_export[self.grado_para_RLM] = model_export
            self.dic_metrics[self.grado_para_RLM] = metrics
            self.declarar_mejor_modelo(tf, n_epochs)
            self.mejor_modelo_encontrado = True
            
        return ecm
    
    def predict(self, dia_predecir_desde, dia_predecir_hasta, mostrar_graficos, predecir_aunque_exista):
        print('Si el predict requerido existe, y existe en el mismo desde-hasta y además NO hay un mejor modelo encontrado, entonces no hacer el PREDICT')
        
        if self.predecir_con_todos_los_datos:
            self.dia_predecir_desde = self.df_model_base_0['ds'].max() + dt.timedelta(days = 1) # *: self.df_model_base_0['ds'].max().date() depende del último entrenamiento
        
        #print('Revision predict')
        #print(not self.mejor_modelo_encontrado, self.dia_predecir_hasta == dia_predecir_hasta, self.dia_predecir_hasta,  dia_predecir_hasta)
        #sys.exit('B')
        
        if not predecir_aunque_exista:
            if (not self.mejor_modelo_encontrado) and (self.dia_predecir_hasta == dia_predecir_hasta):
                print('ya existe el predict')
                self.plotear(mostrar_graficos)
                return None # Ya existe exactamente el mismo predict...continuar
        
        #print('self.dia_predecir_desde', self.dia_predecir_desde)
        self.dia_predecir_desde, self.dia_predecir_hasta = dia_predecir_desde, dia_predecir_hasta
        self.mejor_modelo_encontrado = False
        #print('self.dia_predecir_desde 2', self.dia_predecir_desde)
        
        # se lee el mejor modelo
        #model = 
        variables = self.dic_best_variables[self.grado_para_RLM]
        df_model = self.df_model_base_0.copy()
        
        #df_model = df_model[df_model['y'] > 0].reset_index(drop = True)

        #pais = self.nombre.split('_')[0]
                
        #print('variables')
        
        #print('df_model predict')
        #display(df_model)
        #display(variables[['PARAMETRO', 'REPRESENTACION', 'ACTUAL']])
        
        try:
            self.best_model = model_load(f'{self.dir_model}{self.nombre}.np')
        except:
            print('No se puede cargar best model -> Se usa self.model_default')
            self.best_model = self.model_default
            
        forecast, self.forecast_original = self.hacer_prediccion(tf, self.best_model, df_model, dia_predecir_hasta, etapa_prediccion = True)
        forecast['ds'] = pd.to_datetime(forecast['ds']).dt.date
        
        # data real y las métricas asociadas
        df_data_real = self.df_hist.copy()
        df_data_real['TP'] = np.where(df_data_real['ORDENES'] == 0, 0, df_data_real['VENTA_COLOCADA'] / df_data_real['ORDENES'])
        df_data_real['TC'] = np.where(df_data_real['VISITAS'] == 0, 0, df_data_real['ORDENES'] / df_data_real['VISITAS'])
        df_data_real['DATE'] = pd.to_datetime(df_data_real['DATE']).dt.date
        
        if self.nombre == "MX_LIBRE_VISITAS_ORGANICO":
            df_data_real.to_csv(f'A1PRUEBA_0C_df_data_real {self.nombre}.csv', index = False, sep = ';', decimal = ',')
        
        #display(df_data_real)
        #print('self.metrica_main', self.metrica_main)
        df_data_real = df_data_real[['DATE', self.metrica_main]].rename(columns = {self.metrica_main: f'{self.metrica_main}_REAL'})
        
        if self.nombre == "MX_LIBRE_VISITAS_ORGANICO":
            df_data_real.to_csv(f'A2PRUEBA_0C_df_data_real2 {self.nombre}.csv', index = False, sep = ';', decimal = ',')
        
        df_data_real_forecast = df_data_real.rename(columns = {f'{self.metrica_main}_REAL': 'y_real', 'DATE': 'ds'})
        forecast = forecast.merge(df_data_real_forecast, on = 'ds', how = 'left')
        forecast['y'] = forecast['y_real']
        forecast = forecast.drop(columns = ['y_real'])
        
        if self.nombre == "MX_LIBRE_VISITAS_ORGANICO":
            forecast.to_csv(f'A3PRUEBA_0C_forecast {self.nombre}.csv', index = False, sep = ';', decimal = ',')
        
        self.forecast = forecast[forecast['ds'] <= dia_predecir_hasta].reset_index(drop = True) # 0_all_forecast
        df_forecast_predict_resumen = forecast[['ds', 'y', 'yhat1', 'yhat1 5.0%', 'yhat1 95.0%']] # resumen
        df_forecast_predict_resumen = df_forecast_predict_resumen.rename(columns = {'ds': 'DATE', 'yhat1': self.metrica_main, 'yhat1 5.0%': f'{self.metrica_main} 5%', 'yhat1 95.0%': f'{self.metrica_main} 95%'})
        
        #df_forecast_predict_resumen['PAIS'] = self.pais
        #df_forecast_predict_resumen['NATURALEZA_MEDIO'] = self.naturaleza_medio
        #df_forecast_predict_resumen['DUPLICACION'] = 'LT'
        
        #display(df_forecast_predict_resumen)
        
        #df_forecast_predict_resumen = df_forecast_predict_resumen[['DATE', 'PAIS', 'DUPLICACION', 'NATURALEZA_MEDIO', self.metrica_main]]
        df_forecast_predict_resumen['DATE'] = pd.to_datetime(df_forecast_predict_resumen['DATE']).dt.date
        df_forecast_predict_resumen = df_forecast_predict_resumen[df_forecast_predict_resumen['DATE'] >= self.dia_predecir_desde]
        
        if self.nombre == "MX_LIBRE_VISITAS_ORGANICO":
            df_forecast_predict_resumen.to_csv(f'A4PRUEBA_0_df_forecast_predict_resumen {self.nombre}.csv', index = False, sep = ';', decimal = ',')
        
        #display('best_variables')
        
        
        df_forecast_predict_resumen = df_forecast_predict_resumen.merge(df_data_real, on = 'DATE', how = 'left')
        df_forecast_predict_resumen[self.metrica_main] = np.where(df_forecast_predict_resumen[f'{self.metrica_main}_REAL'].isna(), df_forecast_predict_resumen[self.metrica_main], df_forecast_predict_resumen[f'{self.metrica_main}_REAL']) # La métrica se reemplaza por el dato real si existe
        self.df_forecast_predict_resumen = df_forecast_predict_resumen.drop(columns = [f'{self.metrica_main}_REAL']) # 1_predict_inicial
        
        #display('self.df_forecast_predict_resumen')
        #display(self.df_forecast_predict_resumen)
        
        #print('Plotear')
        self.plotear(mostrar_graficos)
        
        #sys.exit('Revisar self.df_forecast_predict_resumen metrica main vs real')
        
        #display('self.forecast')
        #display(self.forecast)
        
        #self.existe_prediccion = True

        return None

    def plotear(self, mostrar_graficos):
        
        # New
        
        print('Plotear')
        df_forecast_plt = self.forecast.copy()
        dia_predecir_desde_str = self.dia_predecir_desde.strftime('%Y-%m-%d')
        #df_forecast_plt['y'] = df_forecast_plt['y'].fillna(df_forecast_plt['yhat1'])
        df_forecast_plt = df_forecast_plt[['ds', 'y', 'yhat1']]
        
        #display('df_forecast_plt', dia_predecir_desde_str, self.dia_predecir_desde)
        #display(df_forecast_plt)
        if self.nombre == "MX_LIBRE_VISITAS_ORGANICO":
            df_forecast_plt.to_csv(f'A5df_forecast_plt {self.nombre}.csv', index = False, sep = ';', decimal = ',')

        try:
            df_forecast_plt['y'] = np.where(df_forecast_plt['ds'] >= dia_predecir_desde_str, np.nan, df_forecast_plt['y'])
        except:
            print('Entra al except 0')
            df_forecast_plt['y'] = np.where(df_forecast_plt['ds'] >= self.dia_predecir_desde, np.nan, df_forecast_plt['y'])
            
        df_forecast_plt['ds'] = pd.to_datetime(df_forecast_plt['ds'])

        plt.figure(figsize = (20, 10))
        plt.plot(df_forecast_plt['ds'], df_forecast_plt['y'], 'o', color = 'black')
        plt.plot(df_forecast_plt['ds'], df_forecast_plt['yhat1'], color = 'blue')

        plt.ylabel(self.metrica_main)
                                                                                                                                      
        plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval = 6)) # muestra el eje x cada 6 meses
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %Y')) # Formatear el eje x para mostrar las fechas cada 6 meses   

        # Rotar las etiquetas del eje x para una mejor legibilidad
        plt.gcf().autofmt_xdate()
        plt.title(self.nombre)

        # al guardar, el titulo no aparece completo
        plt.tight_layout()

        print('Guardar gráfico ', f'{self.dir_prophet_plots}{self.nombre}.jpg')
        plt.savefig(f'{self.dir_prophet_plots}{self.nombre}.jpg')

        if not mostrar_graficos: # No mostrar el grafico, solo guardarlo
            plt.close()

        plt.show()

        # COMPONENTS (No he podido exportarlos!!!)
      
        try:
            self.best_model.plot_components(self.forecast_original)
            self.best_model.plot_parameters()
        except:
            print('Exportacion de componentes fallida')
    
    
        return None

    
    def declarar_mejor_modelo(self, tf, n_epochs):
                
        pais = self.nombre.split('_')[0]
        iteracion = self.n_iteraciones_actuales + n_epochs + 1
        date = dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        metrica = self.nombre.split('_')[1] + '_' + self.nombre.split('_')[2]
        
        df_new = pd.DataFrame({'PAIS': [pais], 'N_ITER': [iteracion], 'DATE': [date], 'METRICA': [metrica], 'NEW BEST ECM': [self.best_global_ecm]})

        try:
            df_all = tf.lectura_segura_Gsheets('Iron Train v3', f'Main_v2', time_sleep = 2)
            df_all = pd.concat([df_all, df_new]).reset_index(drop = True)
            
            df_all['N_ITER'] = df_all['N_ITER'].astype('int')
            df_all['NEW BEST ECM'] = df_all['NEW BEST ECM'].astype('float')
            tf.df_to_sheets(df_all, nombre_archivo = 'Iron Train v3', nombre_hoja = 'Main_v2', output = False, crear_archivo = False, crear_hoja = False)
        
        except:
            None # No se declara si no hay conexión a internet
        # Se lee el archivo 
        return None

    def crear_model_export(self, df_model):
        model_export = {'MIN_DATE': df_model['ds'].min(),
                    'MAX_DATE': df_model['ds'].max(),
                    'MIN_Y': df_model['y'].min(),
                    'MAX_Y': df_model['y'].max()}
    
        return model_export
    
    def hacer_prediccion(self, tf, model, df_model, dia_predecir_hasta, etapa_prediccion = False):
                
        self.metrica = self.df_tupla['METRICA'][0]

        df_model_0 = df_model.copy().reset_index(drop = True)
        ajustar_forecast = True
        if self.metrica == 'VISITAS_PAGO':
            print('Visitas pago no requiere pendiente no negativa!!\n\n\n\n')
            ajustar_forecast = False 
            
        print('2. self.metrica', self.metrica, ajustar_forecast)
        
        df_model_0['ds'] = pd.to_datetime(df_model_0['ds']).dt.date
        df_model_0 = df_model_0.drop_duplicates(subset = ['ds']).reset_index(drop = True)
        df_model_0 = df_model_0[['ds', 'y']]

        if self.caso_simplificado:
            df_holidays = self.df_holidays.head(1)
            df_holidays['ds'] = df_holidays['ds'].apply(lambda x: x.replace(year = 2000)) # ds es del tipo datetime64[ns]...cambia el año de la fecha por el año 2000

        else:
            df_holidays = self.df_holidays
        
        df_model_0 = model.create_df_with_events(df_model_0, df_holidays)
        
        #print('E3', model)
        #df_model_0.to_csv('E3_df_model_0.csv', index = False, sep = ';', decimal = ',')
        
        for c in list(set(df_model_0.columns) - {'ds'}):
            df_model_0[c] = df_model_0[c].astype('float64')

        ultimo_dia_actual = df_model_0['ds'].max()
        # convertir a datetime
        ultimo_dia_actual = dt.datetime.strptime(str(ultimo_dia_actual), '%Y-%m-%d %H:%M:%S').date()
        delta_days = (dia_predecir_hasta - ultimo_dia_actual).days + 1
        df_future = model.make_future_dataframe(df_model_0, df_holidays, n_historic_predictions = True, periods = delta_days)
        
        #df_future.to_csv('PRUEBA_0B_df_future.csv', index = False, sep = ';', decimal = ',')
        #print('ULTIMO DIA ACTUAL', ultimo_dia_actual)
        
        # ELIMINAR (3)
        #display('df_future')
        #display(df_future)
        #forecast_original = model.predict(df_future) 
        #print('E4', model)
        #df_future.to_csv('E4df_future.csv', index = False, sep = ';', decimal = ',')

        #display(df_future)
        
        try:
            print('Predict')
            forecast_original = model.predict(df_future)  # Aqui predict
        except:
            print('Entra al except 1')
            forecast_original = df_future[['ds', 'y']]
            v = forecast_original[forecast_original['y'].notna()]['y'].unique()[0]
            for c in ['yhat1', 'yhat1 5.0%', 'yhat1 95.0%']:
                forecast_original[c] = v
                
            forecast_original['trend'] = 0
            
            for c in list(df_holidays['event'].unique()) + ['additive']:
                forecast_original[f'event_{c}'] = 0
        
        #print('E5', model)
        #forecast_original.to_csv('E5_forecast_original.csv', index = False, sep = ';', decimal = ',')
        
        #sys.exit('R0')
        forecast = forecast_original.copy()
        forecast['ds'] = pd.to_datetime(forecast['ds'])
        #forecast['ds'] = pd.to_datetime(forecast['ds']).dt.date
        
        #print('Ajustar forecast?', self.modo_proyeccion == 'PRESUPUESTO', ajustar_forecast, self.modo_proyeccion)
        #print('En cualquier caso (libre o ppto), entontrar tendencia que hace que el mínimo sea 0')
        #forecast.to_csv('forecast.csv', index = False, sep = ';', decimal = ',')
        #sys.exit('Ajustar forecast')
        #print('3.', self.modo_proyeccion, ajustar_forecast)
        if (self.modo_proyeccion == 'PRESUPUESTO') and ajustar_forecast:
            #forecast = self.corregir_forecast(forecast, min_delta_trend = 0)
            #print('4. Corregir fcst')
            forecast = self.corregir_forecast_v2(forecast, dia_predecir_hasta)
        
        if etapa_prediccion:
            return forecast, forecast_original

        #display(forecast)
        df_model_0 = forecast[['ds', 'yhat1']].rename(columns = {'yhat1': 'y'})
        for c in list(df_holidays['event'].unique()):
            df_model_0 = df_model_0.rename(columns = {f'event_{c}': c})
        
        dia_predecir_hasta_date = pd.to_datetime(dia_predecir_hasta)
        df_model_0 = df_model_0[df_model_0['ds'] <= dia_predecir_hasta_date].reset_index(drop = True)
        #df_model_0.to_csv('B6df_model_0.csv', index = False, sep = ';', decimal = ',')
        return df_model_0, forecast
    
    
    def corregir_forecast_v2(self, forecast, dia_predecir_hasta, factor_min_crecimiento = 1):
        
        #print('5. Entra a corregir forecast v2')
    
        """
        if 'VISITAS' in self.nombre:
            df_multiplicativo = pd.DataFrame() # Para visitas
        else:
            print(self.nombre)
            forecast.to_csv('forecast.csv', index = False, sep = ';', decimal = ',')
            sys.exit('Crear df_multiplicativo')
        """
        
        año_ppto = dia_predecir_hasta.year
        año_referencia = año_ppto - 1
        
        forecast_resumen = forecast[['ds', 'y', 'yhat1', 'trend']]
        #display(forecast_resumen)
        
        forecast_resumen['año'] = forecast_resumen['ds'].dt.year
        forecast_resumen = forecast_resumen[forecast_resumen['año'].isin([año_ppto, año_referencia])].reset_index(drop = True)

        real_ref = forecast_resumen[forecast_resumen['y'].notna()]['y'].sum()
        proy_ref = forecast_resumen[(forecast_resumen['año'] == año_referencia) & (forecast_resumen['y'].isna())]['yhat1'].sum()
        proy_ppto = forecast_resumen[(forecast_resumen['año'] == año_ppto) & (forecast_resumen['y'].isna())]['yhat1'].sum()

        df_proy = forecast_resumen[(forecast_resumen['y'].isna())]
        df_proy['t'] = df_proy.index

        N_ref = df_proy[df_proy['año'] == año_referencia]['t'].sum()
        N_ppto = df_proy[df_proy['año'] == año_ppto]['t'].sum()

        forecast_resumen['yf'] = forecast_resumen['y'].fillna(forecast_resumen['yhat1'])
        y_f_año_referencia = forecast_resumen[forecast_resumen['año'] == año_referencia]['yf'].sum()
        y_f_año_ppto = forecast_resumen[forecast_resumen['año'] == año_ppto]['yf'].sum()
        
        #print('6.')
        #display(forecast_resumen)
        #print('f:', y_f_año_ppto / y_f_año_referencia)
        #sys.exit('Salida')
        
        if y_f_año_ppto / y_f_año_referencia < factor_min_crecimiento: # DE lo contrario, no hay problema...continuar como está
            
            print('Requiere ajuste')
            ajuste = (factor_min_crecimiento * (real_ref + proy_ref) - proy_ppto) / (N_ppto - factor_min_crecimiento * N_ref)
            forecast_resumen['ajuste'] = ajuste * forecast_resumen.index
            forecast_resumen['yhat2'] = forecast_resumen['yhat1'] + forecast_resumen['ajuste']
            forecast_resumen['yf2'] = forecast_resumen['y'].fillna(forecast_resumen['yhat2'])

            y_f_año_referencia2 = forecast_resumen[forecast_resumen['año'] == str(año_referencia)]['yf2'].sum()
            y_f_año_ppto2 =  forecast_resumen[forecast_resumen['año'] == str(año_ppto)]['yf2'].sum()

            if abs(y_f_año_ppto2 / y_f_año_referencia2 - factor_min_crecimiento) > 0.0001:
                sys.exit('problema')
            
            forecast_merge = forecast_resumen[forecast_resumen['y'].isna()][['ds', 'ajuste']]
            forecast = forecast.merge(forecast_merge, on = 'ds', how = 'left')
            forecast['ajuste'] = forecast['ajuste'].fillna(0)
        
        else:
            forecast['ajuste'] = 0

        forecast['yhat_new'] = forecast['yhat1'] + forecast['ajuste']
        forecast['factor'] = forecast['yhat_new'] / forecast['yhat1']
        for c in ['yhat1', 'yhat1 5.0%', 'yhat1 95.0%']:
            forecast[c] = forecast[c] * forecast['factor']

        forecast['trend'] += forecast['ajuste']
        forecast = forecast.drop(columns = ['ajuste', 'factor', 'yhat_new'])
        return forecast
    
        
    def corregir_forecast(self, forecast, min_delta_trend = -99999999999): # Asegura una mínima tendencia para el forecast
        forecast['delta_trend'] = forecast['trend'] - forecast['trend'].shift(1)
        delta_trend = forecast['delta_trend'][len(forecast) - 1] # tendencia al final
        #print('delta_trend', delta_trend, min_delta_trend)

        if delta_trend < min_delta_trend:
            
            #print('Entra a correccion', delta_trend, min_delta_trend)
            
            df_ajuste = forecast[forecast['y'].isna()][['ds', 'trend']].reset_index(drop = True)
            df_ajuste['ajuste'] = (min_delta_trend - delta_trend) * df_ajuste.index
            df_ajuste = df_ajuste[['ds', 'ajuste']]

            forecast = forecast.merge(df_ajuste, on = 'ds', how = 'left')
            forecast['ajuste'] = forecast['ajuste'].fillna(0)
            forecast['yhat_new'] = forecast['yhat1'] + forecast['ajuste']
            forecast['factor'] = forecast['yhat_new'] / forecast['yhat1']
            for c in ['yhat1', 'yhat1 5.0%', 'yhat1 95.0%']:
                forecast[c] = forecast[c] * forecast['factor']
            
            forecast['trend'] += forecast['ajuste']
            forecast = forecast.drop(columns = ['ajuste', 'factor'])
            
        return forecast

    def generate_new_RLM_point(self, ecm, dic_variables_values):
        
        df_RLM = self.dic_df_RLM[self.grado_para_RLM]
        df_funcion_error = self.dic_df_funcion_error[self.grado_para_RLM]
        
        if ecm == 0:
            return df_RLM 
        df_funcion_error = df_funcion_error.reset_index(drop = True)
        df_funcion_error['TERMINO_ACTUAL'] = df_funcion_error['TERMINO'].apply(lambda x: x.subs(dic_variables_values))

        # Se agrega el registro a RLM
        df_new_iter = df_funcion_error[['TERMINO', 'TERMINO_ACTUAL']].set_index('TERMINO').T.reset_index(drop = True)
        df_new_iter['ECM_real'] = ecm

        df_RLM = pd.concat([df_RLM, df_new_iter]).reset_index(drop = True)
        #display(df_RLM)
        self.dic_df_RLM[self.grado_para_RLM] = df_RLM
        
        #sys.exit('generate_new_RLM_point 0')
        
        return None
    
    def save(self):
        #print('Guardar')

        # Se guarda la clase como pkl
        tf.pickle_act(f'{self.dir_proy_t}{self.nombre}', variable = self, mode = 'save') # Se guarda la actualizacion
        return None
        
    def entrenar_prophet(self, beta, dias_proyeccion, confidence_level, n_epochs, iteraciones):
        
        #print('0 Entrenar Prophet 0803')
        #display(self.df_hist.head(1))
        
        [dia_predecir_desde, dia_predecir_hasta] = dias_proyeccion
        # etapa inicializar se hace en el init
        df_variables = self.create_math_model() # df_variables depende de grado_para_RLM
        df_variables = self.calculate_derivatives(df_variables)
        self.inicializar_RLM()
        
        df_model_base_0 = self.df_hist.rename(columns = {'DATE': 'ds', self.metrica_main: 'y'})
        if 'y' not in df_model_base_0.columns:
            display(df_model_base_0)
            print('self.metrica_main', self.metrica_main)
            sys.exit('HHH')
        self.df_model_base_0 = self.eliminar_outliers(df_model_base_0, dia_predecir_desde, beta)
        
        #sys.exit('AAA')
        self.caso_simplificado = False
        
        for n_iter in range(iteraciones):
            self.show_x0(df_variables, n_iter)
            dic_variables_names, dic_variables_values = self.run_prophet(df_variables)
            ecm = self.run_epochs(dic_variables_names, confidence_level, n_epochs, dia_predecir_hasta)
            if not self.caso_simplificado:
                self.generate_new_RLM_point(ecm, dic_variables_values)
                self.run_RLM() # cambia df_funcion_error
                df_variables = self.update_variables_gradient_descent(ecm, df_variables)
            self.n_iteraciones_actuales += n_epochs
            
        self.dic_df_variables[self.grado_para_RLM] = df_variables
        return None
    
    def ejecutar_RLM(self, df, lista_x):
        x = df[lista_x] # asegura el mismo orden en el que se incirán los datos
        y = df[['ECM_real']]
                    
        regr = linear_model.LinearRegression(fit_intercept = False) # Para el intercepto, está la variable X0
        regr.fit(x, y)
                    
        betas = list(regr.coef_[0])
        return betas
        
    def run_RLM(self):    
        
        df_RLM = self.dic_df_RLM[self.grado_para_RLM]
        df_funcion_error = self.dic_df_funcion_error[self.grado_para_RLM]

        df_RLM = df_RLM[df_RLM['ECM_real'].notna()].reset_index(drop = True)
        M = 10 ** 20
        df_RLM['ECM_real'] = np.where(df_RLM['ECM_real'] > (10 ** 10) * M, M, df_RLM['ECM_real'])
        
        lista_x = list(df_funcion_error['TERMINO'].unique())
        
        #display('df_funcion_error', self.n_max_sin_split)
        #display(df_funcion_error)
        #display(df_RLM)
        
        
        if len(df_RLM) < self.n_max_sin_split:
            betas = self.ejecutar_RLM(df_RLM, lista_x)
            df_funcion_error['BETA_ACTUAL'] = betas
            self.dic_df_funcion_error[self.grado_para_RLM] = df_funcion_error
        
        else:

            ecm_rlm = float('inf')
            for i in range(self.n_iteraciones_con_split): # Se elige el mejor entre n_iteraciones_con_split casos aleatorios
                df_RLM_train, df_RLM_test = train_test_split(df_RLM, test_size = self.test_size)
                betas = self.ejecutar_RLM(df_RLM_train, lista_x)
                df_RLM_test['ECM_calculado'] = df_RLM_test[lista_x].apply(lambda x: sum([x[i] * betas[i] for i in range(len(x))]), axis = 1)
                ecm_test = ((df_RLM_test['ECM_calculado'] - df_RLM_test['ECM_real']) ** 2).mean()
                if ecm_test < ecm_rlm:
                    ecm_rlm = ecm_test
                    df_funcion_error['BETA_ACTUAL'] = betas
                    self.dic_df_funcion_error[self.grado_para_RLM] = df_funcion_error  
                        
        return None
    
    def update_variables_gradient_descent(self, ecm, df_variables):
        
        alpha_max = 0.01 # si el gradiente ya no se mueve mucho, se mueve aleatoriamente a otro punto
        
        df_funcion_error = self.dic_df_funcion_error[self.grado_para_RLM]
        # self.n_iteraciones_actuales
        # self.learning_rate

        modo_fuera_rango = 'random'
        # Diccionario completo de parametros y sus valores
        dic_all_values_beta = df_funcion_error[['BETA', 'BETA_ACTUAL']].set_index('BETA').to_dict()['BETA_ACTUAL']
        dic_all_values_x = df_variables[['REPRESENTACION', 'ACTUAL']].set_index('REPRESENTACION').to_dict()['ACTUAL']
        dic_all_values = {**dic_all_values_x, **dic_all_values_beta}
        
        df_variables['DERIVADA_VALUE'] = df_variables['DERIVADA'].apply(lambda x: x.subs(dic_all_values)) # sustitución por vaores reales: evalf
        
        aleatorio = False
        if (df_variables['DERIVADA_VALUE'].min() == 0) and (df_variables['DERIVADA_VALUE'].max() == 0):
            aleatorio = True
        else:
            df_variables['DELTA_CAMBIO_ABS'] = abs(df_variables['DERIVADA_VALUE'] * self.learning_rate / (ecm))
            
            #display('df_variables')
            #display(df_variables)
                    
            if len(df_variables[(df_variables['DELTA_CAMBIO_ABS'] > alpha_max) & (df_variables['COTA_MAX'] > df_variables['COTA_MIN'])]) == 0: # & cota max > cota min simplemente excluye el caso de X0 (y eventualmente otros valores fijos)
                print('Aleatorio por alpha max')
                aleatorio = True
                #display(df_variables[(df_variables['COTA_MAX'] > df_variables['COTA_MIN'])])
            df_variables = df_variables.drop(columns = 'DELTA_CAMBIO_ABS')
        
        #display('df_variables en gradient descent')
        #display(df_variables)
 
        if (ecm != 0) and not aleatorio:
            #df_variables['DELTA_CAMBIO'] = df_variables['DERIVADA_VALUE'] * self.learning_rate / (ecm * (self.n_iteraciones_actuales + 1) ** 0.5) # La división permite ir haciendo el LR más pequeño con el avance
            df_variables['DELTA_CAMBIO'] = df_variables['DERIVADA_VALUE'] * self.learning_rate / (ecm) 
            df_variables['NEW_ACTUAL'] = df_variables['ACTUAL'] - df_variables['DELTA_CAMBIO']
        else:
            df_variables['NEW_ACTUAL'] = df_variables.apply(lambda x: random.uniform(x['COTA_MIN'], x['COTA_MAX']) if x['TIPO'] == 'float' else random.randint(x['COTA_MIN'], x['COTA_MAX']), axis = 1)
        
        # actualizar el valor de actual, si NEW_ACTUAL está entre cota_min y cota_max, mantener, de lo contrario, generar un aleatorio entre cota_min y cota_max
        
        if modo_fuera_rango == 'random':
            df_variables['ACTUAL'] = df_variables.apply(lambda x: x['NEW_ACTUAL'] if (x['NEW_ACTUAL'] >= x['COTA_MIN']) and (x['NEW_ACTUAL'] <= x['COTA_MAX']) else (random.uniform(x['COTA_MIN'], x['COTA_MAX']) if x['TIPO'] == 'float' else random.randint(x['COTA_MIN'], x['COTA_MAX'])), axis = 1)
        else: # cotas
            df_variables['ACTUAL'] = df_variables.apply(lambda x: x['NEW_ACTUAL'] if (x['NEW_ACTUAL'] >= x['COTA_MIN']) and (x['NEW_ACTUAL'] <= x['COTA_MAX']) else (x['COTA_MIN'] if x['NEW_ACTUAL'] < x['COTA_MIN'] else x['COTA_MAX']), axis = 1) # se mantiene directamente en las cotas
        df_variables['ACTUAL'] = np.where(df_variables['TIPO'] == 'int', df_variables['ACTUAL'].apply(lambda x: int(round(x))), df_variables['ACTUAL'])
        df_variables['ACTUAL'] = df_variables['ACTUAL'].astype(float)
        df_variables = df_variables.drop(columns = ['DERIVADA_VALUE', 'NEW_ACTUAL']) 
        
        return df_variables

    def plotear_ECM_evolucion(self, plotear_evolucion_ecm, mostrar):
        if not plotear_evolucion_ecm:
            return None
        
        
        # tamaño de la figura
        plt.figure(figsize = (6, 4))

        # restablece diccionario y variables
        df_RLM = self.dic_df_RLM[self.grado_para_RLM]
        y = df_RLM['ECM_real']
        x = list(range(len(y)))

        # gráfico principal
        plt.plot(x, y, c='lightblue')

        # encontrar y marcar mínimo
        min_y = y.min()
        min_x = y.idxmin()
        plt.plot(min_x, min_y, 'ro', c='salmon')

        # gráfica de la línea de tendencia
        z = np.polyfit(x, y, 1)
        p = np.poly1d(z)
        plt.plot(x, p(x), 'r--', c='goldenrod')

        # ecuación de la recta
        plt.text(0, 0, f'Ecuación ECM = {p[0]:.3f} + {p[1]:.3f} * X', color='black', fontsize=12)

        # etiquetas y título
        plt.xlabel('# Iteraciones')
        plt.ylabel('Test ECM')
        plt.title(self.nombre)

        # guardar el gráfico
        plt.savefig(f'{self.dir_plots}/{self.nombre}.jpg')

        # cerrar la figura para evitar superposiciones con otros gráficos
        
        if not mostrar: # No mostrar el grafico, solo guardarlo
            plt.close()
        plt.show()
        
        """
        if not mostrar: # No mostrar el grafico, solo guardarlo
            plt.close()
        plt.show()
        """
        
        #sys.exit('Rev grafico')

        return None
    
    def detalle_proyeccion_basica(self, df, n_epochs, dia_predecir_hasta, df_all_data, campos_merge, df_tupla, evento, df_dates_evento, ingreso_segundario = False):
        
        df_dates_evento_cuenta = df_dates_evento[['EVENTO']]
        df_dates_evento_cuenta['CUENTA_DIAS'] = 1
        df_dates_evento_cuenta = df_dates_evento_cuenta.groupby('EVENTO').sum().reset_index()
        
        beta = 0.1 # cuanto - / + de los extremos históricos es permitido
        extremos = df[self.metrica].min(), df[self.metrica].max()
        
        min_datos_proyeccion = 4 # al menos "min_datos_proyeccion" datos para RLM con train / test
        proporcion_minima = 0.1
        
        min_datos_proporcion = df_dates_evento_cuenta['CUENTA_DIAS'][0] * proporcion_minima
        
        if (len(df) >= min_datos_proyeccion) and (len(df) >= min_datos_proporcion):
            for n in range(n_epochs):
                df_train, df_test = train_test_split(df, test_size = self.test_size)
                df_train, df_test = df_train.sort_values('DATE').reset_index(drop = True), df_test.sort_values('DATE').reset_index(drop = True)
                B0, B1, ecm_test = RLS(df_train, df_test, extremos, beta = beta, date = 'DATE', y_value = self.metrica)
                if (evento not in self.best_ecm) or (ecm_test < self.best_ecm[evento]):
                    self.best_ecm[evento] = ecm_test
                    self.best_B0[evento], self.best_B1[evento] = B0, B1
            
            df_predict = predecir_RLS(df, self.best_B0[evento], self.best_B1[evento], 'DATE', self.metrica, dia_predecir_hasta, extremos, beta)
                    
        else:
            #print('Si no hay datos -> 0')
            """print('Promedio simple...dejar en el mismo formato que df_predict')"""
            
            #print('campos_merge', campos_merge)
            
            for campo in [campo_last_touch, 'SUBFAMILIA', 'FAMILIA', 'FUENTE', 'CANAL']: # Se elimina el primer campo de estos (ordenados) que sea encontrado
                if campo in df_tupla.columns:
                    # print('campo a eliminar', campo)
                    df_tupla = df_tupla.drop(columns = campo)
                    campos_merge.remove(campo)
                    break
                
            #display(df_all_data)
            if len(campos_merge) == 1:
                sys.exit('B')
            #print('campos_merge', campos_merge)
            #print('self.metrica', self.metrica)
            #print('df_tupla')
            #display(df_tupla)
            
            df_data = inicializar_data(df_all_data, df_tupla, campos_merge = campos_merge)
            
            df_data_metrica = df_data[['DATE', self.metrica]]
            df_data_metrica = df_data_metrica[df_data_metrica[self.metrica].notna()].reset_index(drop = True) # sin denominador, la métrica se considera nula (X/0) y simplemente es un dato no existente en el entrenamiento
            
            #display('df_data_metrica 1')
            #display(df_data_metrica)
            
            #df_data_metrica = df_data_metrica[df_data_metrica['DATE'].isin(df['DATE'].unique())].reset_index(drop = True) # Solo para los días iniciales (evento o no evento)
            #if ingreso_segundario:
            #    sys.exit('Post 18 A')
            df_data_metrica['PAIS'] = self.pais # self.pais

            df_data_metrica = df_data_metrica.merge(df_dias_eventos, on = ['DATE', 'PAIS'], how = 'left') 
            df_data_metrica['EVENTO'] = df_data_metrica['EVENTO'].fillna('')
            
            #print('CORRECCION 1A0...cambiar cuando esté OK')
            df_data_metrica = df_data_metrica[df_data_metrica['EVENTO'] == evento].reset_index(drop = True)
            
            #display('df_data_metrica 2')
            #if eventos:
            #    print('UNICOS EVENTOS', df['DATE'].unique())
            #display(df_data_metrica)
            
            df_predict = self.detalle_proyeccion_basica(df_data_metrica, n_epochs, dia_predecir_hasta, df_all_data, campos_merge, df_tupla, evento, df_dates_evento, ingreso_segundario = True)
            
            #display(df_data_metrica)
            #display(df_output)
            #sys.exit('Post 18 [recien df output debe verse como df_predict]')
            
        # print('AQUI CAMBIAR por listado de días asociados al evento en particular [Correccion 1A0] [X]')
        # df_predict = df_predict[(df_predict['DATE'].isin(df_dates_evento['DATE'].unique())) | (df_predict['DATE'] >= dt.datetime.today())].reset_index(drop = True)
        df_predict = df_predict[(df_predict['DATE'].isin(df_dates_evento['DATE'].unique()))].reset_index(drop = True)

        return df_predict

    # Método proyeccion.proyeccion_basica # [ANCLAR 1]
    def proyeccion_basica(self, df_data_metrica, n_epochs, dia_predecir_hasta, df_all_data, campos_desagregados, dates_extremos):
        
        df_data_metrica['PAIS'] = self.pais # self.pais

        try:
            df_data_metrica = df_data_metrica.merge(self.df_dias_eventos_pais, on = ['DATE', 'PAIS'], how = 'left') # self.df_dias_eventos
        except:
                    
            print(df_data_metrica.info())
            print(self.df_dias_eventos_pais.info())
            
        df_data_metrica['EVENTO'] = df_data_metrica['EVENTO'].fillna('')
        
        #display('df_data_metrica')
        #display(df_data_metrica)
        
        [d_inicial, d_final] = dates_extremos
        # df con todos los días entre d_inicial y d_final
        df_dates = pd.DataFrame({'DATE': pd.date_range(d_inicial, d_final)})
        #df_dates['DATE'] = pd.to_datetime(df_dates['DATE'])
        #df_dias_eventos['DATE'] = pd.to_datetime(df_dias_eventos['DATE'])
        df_dates['PAIS'] = self.pais

        
        df_dates = df_dates.merge(self.df_dias_eventos_pais, on = ['DATE', 'PAIS'], how = 'left')
        df_dates['EVENTO'] = df_dates['EVENTO'].fillna('')

        #display(df_dates)
        #sys.exit(0)
        
        # variables por defecto (iniciales)

        # Separación evento y no evento
        #print('\n CORREGIR DEBERIA SER PARA CADA TIPO DE EVENTO DIFERENTE + min_datos requeridos debe depender del evento...calcular dias totales historicos para cada evento [X] y obtener una proporcion [CORRECCION 1A0]')
        df_output = pd.DataFrame()
        
        for evento in df_dates['EVENTO'].unique():
            campos_merge = ['PAIS', 'CANAL', 'FUENTE'] + campos_desagregados
            df_tupla = self.df_tupla.copy() 
            df_data_metrica_evento = df_data_metrica[df_data_metrica['EVENTO'] == evento].reset_index(drop = True)
            df_dates_evento = df_dates[df_dates['EVENTO'] == evento].reset_index(drop = True)
            df_predict_evento = self.detalle_proyeccion_basica(df_data_metrica_evento, n_epochs, dia_predecir_hasta, df_all_data, campos_merge, df_tupla, evento, df_dates_evento)
            
            #display('df_predict_evento', evento)
            #display(df_predict_evento)
            
            df_output = pd.concat([df_output, df_predict_evento]).sort_values('DATE').reset_index(drop = True)
        
        df_output = df_output[['DATE', self.metrica, 'predict']]
        
        df_output['PAIS'] = self.pais
        #df_output['DATE'], df_dias_eventos['DATE'] = pd.to_datetime(df_output['DATE']).dt.date, pd.to_datetime(df_dias_eventos['DATE']).dt.date
        df_output = df_output.merge(self.df_dias_eventos_pais, on = ['DATE', 'PAIS'], how = 'left') # self.df_dias_eventos
                
        df_output['EVENTO'] = df_output['EVENTO'].fillna('')
        
        self.df_output = df_output.copy()
                
        colores = {'': 'salmon', 'Hard Sell': 'lightblue', 'Festividad': 'green'}
        # plot
        plotear = False

        if plotear:
            plt.figure(figsize = (5, 2.5))
            for i, evento in enumerate(df_output['EVENTO'].unique()):
                df_output_evento = df_output[df_output['EVENTO'] == evento].reset_index(drop = True)
                
                plt.plot(df_output_evento['DATE'], df_output_evento[self.metrica], 'o', color = colores[evento], markersize = 2)
            plt.plot(df_output['DATE'], df_output['predict'], color = 'black')

            plt.title(self.nombre)
            plt.show()
        
        if len(df_output) < 5:
            sys.exit('Revisar 0')
            
        return None
        

## 3.4. Train

In [953]:
dates_extremos = [df['DATE'].min(), dias_proyeccion[1]]
dt_object = dt.datetime.strptime(str(dates_extremos[0])[:10], '%Y-%m-%d')
dias_proyeccion

[datetime.date(2025, 1, 1), datetime.date(2025, 12, 31)]

Se actualiza data proyeccion

In [954]:
actualizar_generar_data_proyeccion

[]

Generar data proyección

In [955]:
# Parámetros
output_prophet = True
grado_para_RLM = 2
test_size = 0.2
n_iteraciones_con_split = 10 # cuando hay split en rlm (manto para hiperparámetros de prophet), se busca el mejor ecm de testeo entre esta cantidad de casos
n_max_sin_split = 5 # si df_RLM tiene al menos n_max_sin_split registros, entonces se hace el split
learning_rate = 10

dir_proy_t = '../Data/proyeccion_t/Organico/'
dir_plots = '../Data/test_ecm/'
dir_model = '../Data/model/'
dir_prophet_plots = '../Data/prophet_plots/'

predecir_con_todos_los_datos = True

paso = 3 # Parámetro
beta = 0.05 # que proporcion de los datos más bajos y más altos, es eliminada antes de la proyección
confidence_level = 0.9 # intervalo de confianza para neural prophet ¿Como afecta? (calcula el intervalo de confianza [min-max] asociado a este confidence level)

In [956]:
#print('Eliminar 1 cuando train OK')
print(actualizar_generar_data_proyeccion)
#sys.exit()
actualizar_generar_data_proyeccion = paises

[]


In [957]:
df_tuplas

,PAIS,MODO_PROYECCION,METRICA,NATURALEZA_MEDIO,NOMBRE,N_EPOCHS_ACTUALES
0,MX,LIBRE,VISITAS,ORGANICO,MX_LIBRE_VISITAS_ORGANICO,40800
1,MX,LIBRE,TC,ORGANICO,MX_LIBRE_TC_ORGANICO,40750
2,MX,LIBRE,TP,ORGANICO,MX_LIBRE_TP_ORGANICO,40750
3,MX,LIBRE,TC,PAGO,MX_LIBRE_TC_PAGO,40750
4,MX,LIBRE,TP,PAGO,MX_LIBRE_TP_PAGO,40800
5,MX,PRESUPUESTO,VISITAS,ORGANICO,MX_PRESUPUESTO_VISITAS_ORGANICO,40650
6,MX,PRESUPUESTO,TC,ORGANICO,MX_PRESUPUESTO_TC_ORGANICO,40750
7,MX,PRESUPUESTO,TP,ORGANICO,MX_PRESUPUESTO_TP_ORGANICO,40750
8,MX,PRESUPUESTO,TC,PAGO,MX_PRESUPUESTO_TC_PAGO,40600
9,MX,PRESUPUESTO,TP,PAGO,MX_PRESUPUESTO_TP_PAGO,40600


In [958]:
for i in range(len(df_tuplas)):
    
    df_tupla = df_tuplas.iloc[i:i + 1].reset_index(drop = True) # el único index que queda es 0
    pais, metrica, nombre, nat_medio = df_tupla['PAIS'][0], df_tupla['METRICA'][0], df_tupla['NOMBRE'][0], df_tupla['NATURALEZA_MEDIO'][0]
    
    if pais not in actualizar_generar_data_proyeccion:
        print(f'{nombre} ya está actualizado')
        continue
    
    df_data_metrica = df[(df['PAIS'] == pais) & (df['NATURALEZA_MEDIO'] == nat_medio)].reset_index(drop = True)
    proyeccion = Proyeccion_t_new(df_tupla, df_data_metrica, carpeta_input, df_dias_eventos, output_prophet, grado_para_RLM, test_size, n_iteraciones_con_split, n_max_sin_split, learning_rate, dir_proy_t, dir_plots, dir_model, predecir_con_todos_los_datos, dir_prophet_plots) 
    
    proyeccion.generar_data_proyeccion()
    proyeccion.save() # guarda el objeto actualizado
    #sys.exit()

Setattr en: ../Data/proyeccion_t/Organico/MX_LIBRE_VISITAS_ORGANICO
Setattr en: ../Data/proyeccion_t/Organico/MX_LIBRE_TC_ORGANICO
Setattr en: ../Data/proyeccion_t/Organico/MX_LIBRE_TP_ORGANICO
Setattr en: ../Data/proyeccion_t/Organico/MX_LIBRE_TC_PAGO
Setattr en: ../Data/proyeccion_t/Organico/MX_LIBRE_TP_PAGO
Setattr en: ../Data/proyeccion_t/Organico/MX_PRESUPUESTO_VISITAS_ORGANICO
Setattr en: ../Data/proyeccion_t/Organico/MX_PRESUPUESTO_TC_ORGANICO
Setattr en: ../Data/proyeccion_t/Organico/MX_PRESUPUESTO_TP_ORGANICO
Setattr en: ../Data/proyeccion_t/Organico/MX_PRESUPUESTO_TC_PAGO
Setattr en: ../Data/proyeccion_t/Organico/MX_PRESUPUESTO_TP_PAGO


Train

In [959]:
#ejecutar[3]['Entrenar'] = False

In [960]:
# new (reemplaza al de abajo)
for i in tqdm.tqdm(range(iteraciones_totales)):
    
    if not ejecutar[3]['Entrenar']:
        break
        
    df_tuplas = df_tuplas.sort_values('N_EPOCHS_ACTUALES').reset_index(drop = True)
    df_tupla = df_tuplas.iloc[0:1].reset_index(drop = True)
    
    pais = df_tupla['PAIS'][0]
    nat_medio = df_tupla['NATURALEZA_MEDIO'][0]
    
    df_data_metrica = df[(df['PAIS'] == pais) & (df['NATURALEZA_MEDIO'] == nat_medio)].reset_index(drop = True) # data asociada al país (orgánica por F-LT [grano])

    n_epochs_actuales = df_tuplas['N_EPOCHS_ACTUALES'][0]

    #display(df_tupla)
    #sys.exit('Revisar self.modo_proyeccion y sus uso')
    proyeccion = Proyeccion_t_new(df_tupla, df_data_metrica, carpeta_input, df_dias_eventos, output_prophet, grado_para_RLM, test_size, n_iteraciones_con_split, n_max_sin_split, learning_rate, dir_proy_t, dir_plots, dir_model, predecir_con_todos_los_datos, dir_prophet_plots, actualizar_data_hist = False)    
    proyeccion.entrenar_prophet(beta, [dates_extremos[0], dias_proyeccion[1]], confidence_level, n_epochs_prophet, iteraciones)
    proyeccion.save() # guarda el objeto actualizado
    
    df_tuplas['N_EPOCHS_ACTUALES'][0] += (n_epochs_prophet * iteraciones)
    
    if ((i + 1) % paso == 0) or (i == iteraciones_totales - 1): # Guardar
        
        
        df_combinaciones_base = pd.DataFrame()
        if 'P3_df_combinaciones.pkl' in os.listdir(dir_data_back):
            df_combinaciones_base = tf.parquet_act(f'{dir_data_back}P3_df_combinaciones')
        
        df_combinaciones_base = pd.concat([df_tuplas, df_combinaciones_base]).reset_index(drop = True)
        df_combinaciones_base = df_combinaciones_base.drop_duplicates(subset = ['NOMBRE']).reset_index(drop = True)
        tf.parquet_act(f'{dir_data_back}P3_df_combinaciones', df_combinaciones_base, 'save')
        
        print('\n Guardado')

0it [00:00, ?it/s]


## 3.3. Evolución ECM

Solo (eventualmente) plotea y guarda los gráficos de evolución ECM

In [961]:
mostrar = True
plotear_evolucion_ecm = True

In [962]:
ejecutar[3]['Evolucion_ECM'] = False

In [963]:
for i in tqdm.tqdm(range(len(df_tuplas))): 
    
    if not ejecutar[3]['Evolucion_ECM']:
        break

    df_tupla_metrica = df_tuplas.iloc[i:i + 1].reset_index(drop = True)  # Elegimos el caso puntual (etiqueta)
    
    pais = df_tupla_metrica['PAIS'][0]
    nat_medio = df_tupla_metrica['NATURALEZA_MEDIO'][0]
    df_data_metrica = df[(df['PAIS'] == pais) & (df['NATURALEZA_MEDIO'] == nat_medio)].reset_index(drop = True) # data asociada al país (orgánica por F-LT [grano])

    proyeccion = Proyeccion_t_new(df_tupla_metrica, df_data_metrica, carpeta_input, df_dias_eventos, output_prophet, grado_para_RLM, test_size, n_iteraciones_con_split, n_max_sin_split, learning_rate, dir_proy_t, dir_plots, dir_model, predecir_con_todos_los_datos, dir_prophet_plots, metrica) 
    print(proyeccion.nombre)
    proyeccion.plotear_ECM_evolucion(plotear_evolucion_ecm, mostrar)

  0%|          | 0/10 [00:00<?, ?it/s]


## 3.4. Predict

In [964]:
dias_proyeccion

[datetime.date(2025, 1, 1), datetime.date(2025, 12, 31)]

In [965]:
[dia_predecir_desde, dia_predecir_hasta] = dias_proyeccion
[dia_predecir_desde, dia_predecir_hasta]

[datetime.date(2025, 1, 1), datetime.date(2025, 12, 31)]

In [966]:
mostrar_graficos = True # parámetro

In [967]:
df_tuplas

,PAIS,MODO_PROYECCION,METRICA,NATURALEZA_MEDIO,NOMBRE,N_EPOCHS_ACTUALES
0,MX,LIBRE,VISITAS,ORGANICO,MX_LIBRE_VISITAS_ORGANICO,40800
1,MX,LIBRE,TC,ORGANICO,MX_LIBRE_TC_ORGANICO,40750
2,MX,LIBRE,TP,ORGANICO,MX_LIBRE_TP_ORGANICO,40750
3,MX,LIBRE,TC,PAGO,MX_LIBRE_TC_PAGO,40750
4,MX,LIBRE,TP,PAGO,MX_LIBRE_TP_PAGO,40800
5,MX,PRESUPUESTO,VISITAS,ORGANICO,MX_PRESUPUESTO_VISITAS_ORGANICO,40650
6,MX,PRESUPUESTO,TC,ORGANICO,MX_PRESUPUESTO_TC_ORGANICO,40750
7,MX,PRESUPUESTO,TP,ORGANICO,MX_PRESUPUESTO_TP_ORGANICO,40750
8,MX,PRESUPUESTO,TC,PAGO,MX_PRESUPUESTO_TC_PAGO,40600
9,MX,PRESUPUESTO,TP,PAGO,MX_PRESUPUESTO_TP_PAGO,40600


In [968]:
predecir_aunque_exista = False # parámetro: en general, dejar como False

In [969]:
for i in tqdm.tqdm(range(len(df_tuplas))):

    if not ejecutar[3]['Predict']:
        break
    
    df_tupla_metrica = df_tuplas.iloc[i:i + 1].reset_index(drop = True)  # Elegimos el caso puntual (etiqueta)
    
    pais = df_tupla_metrica['PAIS'][0]
    nat_medio = df_tupla_metrica['NATURALEZA_MEDIO'][0]
    df_data_metrica = df[(df['PAIS'] == pais) & (df['NATURALEZA_MEDIO'] == nat_medio)].reset_index(drop = True) # data asociada al país (orgánica por F-LT [grano])

    proyeccion = Proyeccion_t_new(df_tupla_metrica, df_data_metrica, carpeta_input, df_dias_eventos, output_prophet, grado_para_RLM, test_size, n_iteraciones_con_split, n_max_sin_split, learning_rate, dir_proy_t, dir_plots, dir_model, predecir_con_todos_los_datos, dir_prophet_plots, metrica) 
    print(proyeccion.nombre)
    #if (not proyeccion.existe_prediccion) or (ejecutar[3]['Predict']): # Solo No se ejecuta la proyeccion cuando ejecutar[3]['Predict'] == False y ya existe una proyeccion
    proyeccion.predict(dia_predecir_desde, dia_predecir_hasta, mostrar_graficos, predecir_aunque_exista)
    proyeccion.save() # guarda el objeto actualizado


  0%|          | 0/10 [00:00<?, ?it/s]


## 3.5. Consolidación de parámetro

In [970]:
df_parametros = pd.DataFrame()

for i in tqdm.tqdm(range(len(df_tuplas))):

    df_tupla_metrica = df_tuplas.iloc[i:i + 1].reset_index(drop = True)  # Elegimos el caso puntual (etiqueta)
    
    pais = df_tupla_metrica['PAIS'][0]
    metrica = df_tupla_metrica['METRICA'][0]
    df_data_metrica = df[(df['PAIS'] == pais) & (df['NATURALEZA_MEDIO'] == nat_medio)].reset_index(drop = True) # data asociada al país (orgánica por F-LT [grano])

    proyeccion = Proyeccion_t_new(df_tupla_metrica, df_data_metrica, carpeta_input, df_dias_eventos, output_prophet, grado_para_RLM, test_size, n_iteraciones_con_split, n_max_sin_split, learning_rate, dir_proy_t, dir_plots, dir_model, predecir_con_todos_los_datos, dir_prophet_plots, metrica) 
    print(proyeccion.nombre)
    
    df_forecast_resumen = proyeccion.forecast[['ds', 'y', 'yhat1', 'yhat1 5.0%', 'yhat1 95.0%']]
    
    #df_data_metrica_0 = df_data_metrica.copy()
    df_data_metrica['DATE'] = pd.to_datetime(df_data_metrica['DATE'])
    df_data_metrica['TP'], df_data_metrica['TC'] = df_data_metrica['VENTA_COLOCADA'] / df_data_metrica['ORDENES'], df_data_metrica['ORDENES'] / df_data_metrica['VISITAS']
    #display(df_data_metrica)
    
    #display(df_forecast_resumen)
    df_forecast_resumen['ds'] = pd.to_datetime(df_forecast_resumen['ds'])
    df_data_metrica_0_aux = df_data_metrica[['DATE', metrica]].rename(columns = {metrica: 'y_aux', 'DATE': 'ds'})
    
    #display(df_forecast_resumen)
    #display(df_data_metrica_0_aux)
    df_forecast_resumen = df_forecast_resumen.merge(df_data_metrica_0_aux, on = 'ds', how = 'left')
    
    #display(df_forecast_resumen)
    #sys.exit()
    
    df_forecast_resumen['y'] = df_forecast_resumen['y'].fillna(df_forecast_resumen['y_aux'])
    df_forecast_resumen = df_forecast_resumen.drop(columns = 'y_aux')
    #display(df_forecast_resumen)
    df_forecast_resumen['NOMBRE'] = proyeccion.nombre
    df_forecast_resumen[['PAIS', 'MODO_PROYECCION', 'METRICA', 'NATURALEZA_MEDIO']] = df_forecast_resumen['NOMBRE'].str.split('_', expand = True)
    df_forecast_resumen = df_forecast_resumen[['NOMBRE'] + ['PAIS', 'MODO_PROYECCION', 'NATURALEZA_MEDIO', 'METRICA'] + ['ds', 'y', 'yhat1', 'yhat1 5.0%', 'yhat1 95.0%']]
    #display(df_forecast_resumen)
    df_parametros = pd.concat([df_forecast_resumen, df_parametros])

  0%|          | 0/10 [00:00<?, ?it/s]

Setattr en: ../Data/proyeccion_t/Organico/MX_LIBRE_VISITAS_ORGANICO


 10%|█         | 1/10 [00:02<00:21,  2.37s/it]

MX_LIBRE_VISITAS_ORGANICO
Setattr en: ../Data/proyeccion_t/Organico/MX_LIBRE_TC_ORGANICO


 20%|██        | 2/10 [00:05<00:22,  2.80s/it]

...

MX_PRESUPUESTO_TP_ORGANICO
Setattr en: ../Data/proyeccion_t/Organico/MX_PRESUPUESTO_TC_PAGO


 90%|█████████ | 9/10 [00:30<00:03,  3.49s/it]

MX_PRESUPUESTO_TC_PAGO
Setattr en: ../Data/proyeccion_t/Organico/MX_PRESUPUESTO_TP_PAGO


100%|██████████| 10/10 [00:33<00:00,  3.39s/it]

MX_PRESUPUESTO_TP_PAGO


## 3.6. Detalle

In [971]:
print('Esto puede apoyar a CL (Foco acá)...hacer a nivel de grano!!')

Esto puede apoyar a CL (Foco acá)...hacer a nivel de grano!!


In [972]:
# renombrar ds como DATE, yhat1 como predict, yhat1 5.0% como predict 5.0%, yhat1 95.0% como predict 95.0%, y como real
df_parametros = df_parametros.rename(columns = {'ds': 'DATE', 'yhat1': 'predict', 'yhat1 5.0%': 'predict 5.0%', 'yhat1 95.0%': 'predict 95.0%', 'y': 'real'})
df_parametros

,NOMBRE,PAIS,MODO_PROYECCION,NATURALEZA_MEDIO,METRICA,DATE,real,predict,predict 5.0%,predict 95.0%
0,MX_PRESUPUESTO_TP_PAGO,MX,PRESUPUESTO,PAGO,TP,2021-04-05,1496.365333,2004.181274,1318.221802,2727.117676
1,MX_PRESUPUESTO_TP_PAGO,MX,PRESUPUESTO,PAGO,TP,2021-04-06,1210.862000,2032.425171,1329.803589,2864.197021
2,MX_PRESUPUESTO_TP_PAGO,MX,PRESUPUESTO,PAGO,TP,2021-04-07,2384.065000,2033.105469,1302.353638,2942.741943
3,MX_PRESUPUESTO_TP_PAGO,MX,PRESUPUESTO,PAGO,TP,2021-04-08,2683.781875,2098.124268,1405.116211,2986.895996
4,MX_PRESUPUESTO_TP_PAGO,MX,PRESUPUESTO,PAGO,TP,2021-04-09,2703.418667,2087.737793,1414.672729,3012.326904
...,...,...,...,...,...,...,...,...,...,...
1821,MX_LIBRE_VISITAS_ORGANICO,MX,LIBRE,ORGANICO,VISITAS,2025-12-27,NaN,39359.500000,29772.962891,69302.835938
1822,MX_LIBRE_VISITAS_ORGANICO,MX,LIBRE,ORGANICO,VISITAS,2025-12-28,NaN,39588.843750,29988.480469,68194.937500
1823,MX_LIBRE_VISITAS_ORGANICO,MX,LIBRE,ORGANICO,VISITAS,2025-12-29,NaN,57082.843750,44862.132812,89184.289062
1824,MX_LIBRE_VISITAS_ORGANICO,MX,LIBRE,ORGANICO,VISITAS,2025-12-30,NaN,50649.281250,41405.011719,76467.992188


In [973]:
tf.parquet_act(f'{dir_data_back}P3_df_parametros_prediccion', df_parametros, 'save') # Se guarda como está hasta ahora

# Para la apertura, se continúa solo con parámetros orgánicos 
df_parametros = df_parametros[df_parametros['NATURALEZA_MEDIO'] == 'ORGANICO'].reset_index(drop = True)
df_parametros = df_parametros.drop(columns = 'NATURALEZA_MEDIO')
df_parametros

,NOMBRE,PAIS,MODO_PROYECCION,METRICA,DATE,real,predict,predict 5.0%,predict 95.0%
0,MX_PRESUPUESTO_TP_ORGANICO,MX,PRESUPUESTO,TP,2021-01-01,NaN,2051.456055,1499.181030,3376.771973
1,MX_PRESUPUESTO_TP_ORGANICO,MX,PRESUPUESTO,TP,2021-01-02,NaN,1881.715820,1293.135132,3216.124512
2,MX_PRESUPUESTO_TP_ORGANICO,MX,PRESUPUESTO,TP,2021-01-03,NaN,1782.404053,1341.511475,3059.372070
3,MX_PRESUPUESTO_TP_ORGANICO,MX,PRESUPUESTO,TP,2021-01-04,NaN,2179.326660,1704.687500,3379.052246
4,MX_PRESUPUESTO_TP_ORGANICO,MX,PRESUPUESTO,TP,2021-01-05,NaN,2140.540527,1567.987183,3509.779297
...,...,...,...,...,...,...,...,...,...
10763,MX_LIBRE_VISITAS_ORGANICO,MX,LIBRE,VISITAS,2025-12-27,NaN,39359.500000,29772.962891,69302.835938
10764,MX_LIBRE_VISITAS_ORGANICO,MX,LIBRE,VISITAS,2025-12-28,NaN,39588.843750,29988.480469,68194.937500
10765,MX_LIBRE_VISITAS_ORGANICO,MX,LIBRE,VISITAS,2025-12-29,NaN,57082.843750,44862.132812,89184.289062
10766,MX_LIBRE_VISITAS_ORGANICO,MX,LIBRE,VISITAS,2025-12-30,NaN,50649.281250,41405.011719,76467.992188


### 3.6.1. Apertura en canal - fuente

In [974]:
df_representantes_base = df_representantes_base[df_representantes_base['NATURALEZA_MEDIO'] == 'ORGANICO'].reset_index(drop = True)
df_representantes_base = df_representantes_base.drop(columns = 'NATURALEZA_MEDIO')
df_representantes_base

,PAIS,DATE,FUENTE,CANAL,FAMILIA,TIPO_MEDIO,VENTA_COLOCADA,ORDENES,VISITAS
0,AR,2021-04-05,SODIMAC,WEB,06 - Plomeria / Gasfiteria,SEO,40062.9,3.0,1632.0
1,AR,2021-04-05,SODIMAC,WEB,01 - Madera Y Tableros,SEO,12909.0,1.0,1032.0
2,AR,2021-04-05,SODIMAC,WEB,25 - Promociones - Soporte Tecnico,SEO,0.0,0.0,57.0
3,AR,2021-04-05,SODIMAC,WEB,20 - Aseo,SEO,3068.0,2.0,867.0
4,AR,2021-04-05,SODIMAC,WEB,08 - Accesorios Automoviles,SEO,809.0,1.0,576.0
...,...,...,...,...,...,...,...,...,...
3455852,UY,2025-01-22,SODIMAC,WEB,05 - Obra Gruesa,Reference Domain,0.0,0.0,6.0
3455853,UY,2025-01-22,SODIMAC,WEB,09 - Herramientas Y Maquinarias,Reference Domain,0.0,0.0,4.0
3455854,UY,2025-01-22,SODIMAC,WEB,14 - Pisos,Reference Domain,0.0,0.0,16.0
3455855,UY,2025-01-22,SODIMAC,WEB,08 - Accesorios Automoviles,Reference Domain,0.0,0.0,4.0


In [975]:
from scipy.optimize import curve_fit # importaciones
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [976]:
agrupacion_detalle_shares = ['FAMILIA'] # parámetro

In [977]:
# Se aproximan las curvas de canal-fuente con la función A * x^n + k
df_detalle_canal_fuente = df_representantes_base[['PAIS', 'FUENTE', 'CANAL', 'DATE'] + agrupacion_detalle_shares + ['VENTA_COLOCADA', 'ORDENES', 'VISITAS']].groupby(['PAIS', 'FUENTE', 'CANAL', 'DATE'] + agrupacion_detalle_shares).sum().reset_index()
df_detalle_canal_fuente = df_detalle_canal_fuente.sort_values(['PAIS', 'FUENTE', 'CANAL', 'DATE'] + agrupacion_detalle_shares).reset_index(drop = True)
df_detalle_canal_fuente

,PAIS,FUENTE,CANAL,DATE,FAMILIA,VENTA_COLOCADA,ORDENES,VISITAS
0,AR,SODIMAC,APP,2021-04-05,01 - Madera Y Tableros,0.0,0.0,27.0
1,AR,SODIMAC,APP,2021-04-05,02 - Puertas/Ventanas/Molduras,0.0,0.0,35.0
2,AR,SODIMAC,APP,2021-04-05,03 - Fierro/Hierro/Acero,0.0,0.0,4.0
3,AR,SODIMAC,APP,2021-04-05,04 - Tabiqueria/Techumbre/Aislacion,0.0,0.0,15.0
4,AR,SODIMAC,APP,2021-04-05,05 - Obra Gruesa,0.0,0.0,20.0
...,...,...,...,...,...,...,...,...
678024,UY,SODIMAC,WEB,2025-01-22,22 - Aire Libre,49459.5,10.0,1688.0
678025,UY,SODIMAC,WEB,2025-01-22,23 - Jardin,9702.3,2.0,818.0
678026,UY,SODIMAC,WEB,2025-01-22,26 - Alquiler De Herramientas,0.0,0.0,16.0
678027,UY,SODIMAC,WEB,2025-01-22,27 - Organizacion,6192.5,4.0,704.0


In [978]:
# Con un curve fitting, se proyectan estos números, se obtiene un share por canal-fuente, y se aplica a los resultados

In [979]:
def curve_fitting(df_proyeccion, dia_predecir_desde, dia_predecir_hasta, title, n_iters = 300, mostrar_grafico = False):

    # crea df_dates, desde d_0 a d_f
    d_0 = df_proyeccion['DATE'].min()
    d_f = dia_predecir_hasta

    metrica = list(set(df_proyeccion.columns) - {'DATE'})[0]

    df_dates = pd.date_range(dt.datetime(2023, 1, 1).date(), d_f) # Curve fitting de shares se trabaja desde el 2023
    df_dates = pd.DataFrame({'DATE': df_dates})
    df_proyeccion = df_dates.merge(df_proyeccion, how = 'left', on = 'DATE')

    df_train = df_proyeccion[df_proyeccion['DATE'] < pd.to_datetime(dia_predecir_desde)]
    # dia_predecir_desde - dt.timedelta(days = 1)
    df_train[metrica] = df_train[metrica].fillna(0)
    df_train['X'] = df_train.index
    df_train = df_train.rename(columns = {metrica: 'Y'})

    df_train = df_train[['X', 'Y']]
    rango = df_train['Y'].max() - df_train['Y'].min()
    #y_inf0 = df_train['Y'].min() - rango
    #y_sup0 = df_train['Y'].max() + rango
    
    #x0_inicial = (df_train['X'].max() - df_train['X'].min()) / 2

    x_data = np.array(df_train['X'])
    y_data = np.array(df_train['Y'])

    # Definimos la función de ajuste
    def model_function(x, A, n, k):
        return A * x ** n + k
    
    def logistics_function(x, k, x0, y_inf, y_sup):
        return y_inf + (y_sup - y_inf) / (1 + np.exp(-k * (x - x0)))

    # crea una regresion lineal entre x_daa e y_data
    rlm = LinearRegression()
    rlm.fit(x_data.reshape(-1, 1), y_data)
    # dame los parametros
    b0_v, b1_v = rlm.intercept_, rlm.coef_[0]
    
    n_0 = 0.5
    A_0, k_0 = estimar_valores_iniciales(df_train, b0_v, b1_v, n_0, n_iters)
    # Utilizamos curve_fit para encontrar los mejores parámetros
    initial_guess = [A_0, n_0, k_0]  # Valores iniciales para A, n y k
    params, covariance = curve_fit(model_function, x_data, y_data, p0 = initial_guess, bounds = ([-float('inf'), 0, -float('inf')], [float('inf'), 1, float('inf')]))
    # Extraemos los valores óptimos de los parámetros
    A_opt, n_opt, k_opt = params
    #print(f"Parámetros óptimos: A = {A_opt}, n = {n_opt}, k = {k_opt}")

    if mostrar_grafico:
        # Visualizamos los resultados del ajuste
        plt.scatter(x_data, y_data, label = 'Datos originales')
        plt.plot(x_data, model_function(x_data, A_opt, n_opt, k_opt), color = 'red', label = 'Ajuste de la curva')
        plt.xlabel('x')
        plt.ylabel('y')
        plt.legend()
        plt.title(title)
        plt.show()

    df_proyeccion['X'] = df_proyeccion.index
    df_proyeccion[f'{metrica}_PREDICT'] = df_proyeccion['X'].apply(lambda x: model_function(x, A_opt, n_opt, k_opt))

    return df_proyeccion[['DATE', f'{metrica}_PREDICT']]


In [980]:
def generar_share(df, metrica, apertura, agrupacion_detalle_shares):
    
    df = df[['PAIS', 'DATE', metrica] + apertura + agrupacion_detalle_shares]
    df['_'.join(apertura)] = df[apertura].apply(lambda x: '_'.join(x), axis = 1)

    #df['FUENTE_CANAL'] = df['FUENTE'] + '_' + df['CANAL']
    df = df.drop(columns = apertura)
    
    # AUX gen share (nuevo 250102)

    df_share_tot = df[['PAIS', 'DATE', metrica] + agrupacion_detalle_shares].groupby(['PAIS', 'DATE'] + agrupacion_detalle_shares).sum().reset_index() # Suma de la métrica en las combinaciones share
    df_share_tot = df_share_tot.rename(columns = {metrica: f'{metrica}_S'})

    df_agr = df[['PAIS', metrica, '_'.join(apertura)] + agrupacion_detalle_shares].groupby(['PAIS', '_'.join(apertura)] + agrupacion_detalle_shares).sum().reset_index() # Valor de la métrica en todo el horizonte (sum)
    df_agr = df_agr.rename(columns = {metrica: f'{metrica}_A'})

    df = df.merge(df_share_tot, on = ['PAIS', 'DATE'] + agrupacion_detalle_shares, how = 'left')
    df = df.merge(df_agr, on = ['PAIS', '_'.join(apertura)] + agrupacion_detalle_shares, how = 'left')
    df[metrica] = np.where(df[f'{metrica}_S'] == 0, df[f'{metrica}_A'], df[metrica])
    df = df.drop(columns = [f'{metrica}_S', f'{metrica}_A'])
    
    ### Hasta aqui nueva integración

    df = df.pivot(index = ['DATE', 'PAIS'] + agrupacion_detalle_shares, columns = '_'.join(apertura))[metrica].rename_axis(columns = None).reset_index()
    df = df.fillna(0)

    cols_metricas = list(set(df.columns) - {'DATE', 'PAIS'} - set(agrupacion_detalle_shares))
    df['S'] = df[cols_metricas].sum(axis = 1)

    for c in cols_metricas:
        df[c] = df[c] / df['S']
        #df[c] = df[c].fillna(1 / len(cols_metricas))
        #df[c] = df[c].fillna(0)
    df = df.drop(columns = 'S')

    df = df.melt(id_vars = ['DATE', 'PAIS'] + agrupacion_detalle_shares, var_name = '_'.join(apertura), value_name = metrica)
    df[apertura] = df['_'.join(apertura)].str.split('_', expand = True)
    df = df[['DATE', 'PAIS'] + apertura + agrupacion_detalle_shares + [metrica]]#.rename(columns = {metrica: f'{metrica}_SHARE'})

    return df

In [981]:
df_detalle_canal_fuente[['PAIS', 'FUENTE', 'CANAL'] + agrupacion_detalle_shares].drop_duplicates().reset_index(drop = True) # Extra

,PAIS,FUENTE,CANAL,FAMILIA
0,AR,SODIMAC,APP,01 - Madera Y Tableros
1,AR,SODIMAC,APP,02 - Puertas/Ventanas/Molduras
2,AR,SODIMAC,APP,03 - Fierro/Hierro/Acero
3,AR,SODIMAC,APP,04 - Tabiqueria/Techumbre/Aislacion
4,AR,SODIMAC,APP,05 - Obra Gruesa
...,...,...,...,...
566,UY,SODIMAC,WEB,23 - Jardin
567,UY,SODIMAC,WEB,26 - Alquiler De Herramientas
568,UY,SODIMAC,WEB,27 - Organizacion
569,UY,SODIMAC,WEB,Unspecified


In [982]:
# aunque = ejecutar[3]['Shares_canal_fuente'] == False, si la tupla no existe, tiene que ejecutarse
df_shares = pd.DataFrame()
if 'P3_parametros_3_share_canal_fuente.pkl' in os.listdir(dir_data_back):
    df_shares = tf.parquet_act(f'{dir_data_back}P3_parametros_3_share_canal_fuente')
df_shares

,DATE,PAIS,FUENTE,CANAL,FAMILIA,VENTA_COLOCADA_PREDICT,ORDENES_PREDICT,VISITAS_PREDICT
0,2025-01-01,AR,SODIMAC,APP,01 - Madera Y Tableros,0.208959,0.181784,0.150148
1,2025-01-01,AR,SODIMAC,APP,02 - Puertas/Ventanas/Molduras,0.160120,0.175838,0.154922
2,2025-01-01,AR,SODIMAC,APP,03 - Fierro/Hierro/Acero,0.163305,0.186325,0.198523
3,2025-01-01,AR,SODIMAC,APP,04 - Tabiqueria/Techumbre/Aislacion,0.157717,0.156801,0.185960
4,2025-01-01,AR,SODIMAC,APP,05 - Obra Gruesa,0.255057,0.235355,0.223224
...,...,...,...,...,...,...,...,...
246137,2025-12-31,MX,SODIMAC,WEB,23 - Jardin,0.723927,0.732045,0.779931
246138,2025-12-31,MX,SODIMAC,WEB,25 - Promociones - Soporte Tecnico,0.500000,0.500000,0.566874
246139,2025-12-31,MX,SODIMAC,WEB,26 - Alquiler De Herramientas,1.000000,1.000000,0.683585
246140,2025-12-31,MX,SODIMAC,WEB,27 - Organizacion,0.729666,0.716253,0.721775


In [983]:
df_detalle_canal_fuente_tuplas = df_detalle_canal_fuente[['PAIS', 'FUENTE', 'CANAL'] + agrupacion_detalle_shares].drop_duplicates().reset_index(drop = True)
df_detalle_canal_fuente_tuplas

,PAIS,FUENTE,CANAL,FAMILIA
0,AR,SODIMAC,APP,01 - Madera Y Tableros
1,AR,SODIMAC,APP,02 - Puertas/Ventanas/Molduras
2,AR,SODIMAC,APP,03 - Fierro/Hierro/Acero
3,AR,SODIMAC,APP,04 - Tabiqueria/Techumbre/Aislacion
4,AR,SODIMAC,APP,05 - Obra Gruesa
...,...,...,...,...
566,UY,SODIMAC,WEB,23 - Jardin
567,UY,SODIMAC,WEB,26 - Alquiler De Herramientas
568,UY,SODIMAC,WEB,27 - Organizacion
569,UY,SODIMAC,WEB,Unspecified


In [984]:
#df_detalle_canal_fuente_tuplas = df_detalle_canal_fuente_tuplas[df_detalle_canal_fuente_tuplas['FAMILIA'].isin(['02 - Puertas/Ventanas/Molduras', '05 - Obra Gruesa', '07 - Electricidad'])].reset_index(drop = True)
#print('Small batch')

In [985]:
ejecutar[3]['Shares_canal_fuente'] = True

In [986]:
ejecutar_todos = False # PARAMETRO: SI es False, ejecuta solo los no existentes

In [987]:
df_detalle_canal_fuente_tuplas = df_detalle_canal_fuente_tuplas[df_detalle_canal_fuente_tuplas['PAIS'].isin(paises)].reset_index(drop = True)

if not ejecutar_todos and ('P3_parametros_3_share_canal_fuente.pkl' in os.listdir(dir_data_back)):
    df_shares_existentes = tf.parquet_act(f'{dir_data_back}P3_parametros_3_share_canal_fuente')
    df_shares_existentes = df_shares_existentes[['PAIS', 'FUENTE', 'CANAL'] + agrupacion_detalle_shares].drop_duplicates().reset_index(drop = True)
    df_shares_existentes['EXISTE'] = True
    df_detalle_canal_fuente_tuplas = df_detalle_canal_fuente_tuplas.merge(df_shares_existentes, on = ['PAIS', 'FUENTE', 'CANAL'] + agrupacion_detalle_shares, how = 'left')
    df_detalle_canal_fuente_tuplas['EXISTE'] = df_detalle_canal_fuente_tuplas['EXISTE'].fillna(False)
    df_detalle_canal_fuente_tuplas = df_detalle_canal_fuente_tuplas[~df_detalle_canal_fuente_tuplas['EXISTE']].reset_index(drop = True)
    df_detalle_canal_fuente_tuplas = df_detalle_canal_fuente_tuplas.drop(columns = 'EXISTE')

# De lo contrario, df_detalle_canal_fuente_tuplas queda completa

df_detalle_canal_fuente = df_detalle_canal_fuente.sort_values(['PAIS', 'FUENTE', 'CANAL'] + agrupacion_detalle_shares + ['DATE']).reset_index(drop = True)

In [988]:
dia_predecir_desde, dia_predecir_hasta

(datetime.date(2025, 1, 1), datetime.date(2025, 12, 31))

In [989]:
for metrica in ['VENTA_COLOCADA', 'ORDENES', 'VISITAS']:
    
    if len(df_detalle_canal_fuente_tuplas) == 0:
        break
    
    if not ejecutar[3]['Shares_canal_fuente']:
        break
    print(metrica)
    
    df_detalle_canal_fuente_metrica = generar_share(df_detalle_canal_fuente, metrica, ['FUENTE', 'CANAL'], agrupacion_detalle_shares)
    display(df_detalle_canal_fuente_metrica)
    
    # OK hasta aqui [anclar 1]

    df_metrica = pd.DataFrame()
    for i in range(len(df_detalle_canal_fuente_tuplas)):
        df_i = df_detalle_canal_fuente_tuplas.iloc[i:i + 1].reset_index(drop = True)
        display(df_i)
        title = ''
        for c in df_detalle_canal_fuente_tuplas.columns:
            title += df_i[c][0] + '_'
        title = title[:-1] + '_' + metrica

        df_proyeccion = df_i.merge(df_detalle_canal_fuente_metrica, on = ['PAIS', 'FUENTE', 'CANAL'] + agrupacion_detalle_shares, how = 'left')
        #display(df_proyeccion)
        df_proyeccion = df_proyeccion.drop(columns = ['PAIS', 'FUENTE', 'CANAL'] + agrupacion_detalle_shares)  
        
        #display(df_proyeccion)

        #t_k = time.time()
        df_proyeccion_resultado = curve_fitting(df_proyeccion, dia_predecir_desde, dia_predecir_hasta, title, mostrar_grafico = False)
        #print('t', time.time() - t_k)
        
        df_proyeccion = df_proyeccion.merge(df_proyeccion_resultado, on = 'DATE', how = 'outer')
        
        df_proyeccion_resultado['AUX'] = 'aux'
        df_i['AUX'] = 'aux'
        df_proyeccion_resultado = df_i.merge(df_proyeccion_resultado, how = 'left', on = 'AUX')
        df_proyeccion_resultado = df_proyeccion_resultado.drop(columns = 'AUX')
        df_metrica = pd.concat([df_metrica, df_proyeccion_resultado])

    df_metrica[f'{metrica}_PREDICT'] = np.where(df_metrica[f'{metrica}_PREDICT'] < 0, 0, df_metrica[f'{metrica}_PREDICT'])
    df_metrica = df_metrica[df_metrica['DATE'] >= pd.to_datetime(dia_predecir_desde)]

    # pivot y melt
    print('pivot y melt')
    df_metrica = generar_share(df_metrica, f'{metrica}_PREDICT', ['FUENTE', 'CANAL'], agrupacion_detalle_shares)

    if metrica == 'VENTA_COLOCADA':
        df_shares = df_metrica
    else:
        df_shares = df_shares.merge(df_metrica, on = ['DATE', 'PAIS', 'FUENTE', 'CANAL'] + agrupacion_detalle_shares, how = 'outer')
        
if not ejecutar[3]['Shares_canal_fuente']: # Si no se ejecuta, se rescata directo
    df_shares = tf.parquet_act(f'{dir_data_back}P3_parametros_3_share_canal_fuente') # Se rescata directo
else:
    df_shares_base = pd.DataFrame()
    if 'P3_parametros_3_share_canal_fuente.pkl' in os.listdir(dir_data_back):
        df_shares_base = tf.parquet_act(f'{dir_data_back}P3_parametros_3_share_canal_fuente')
    df_shares = pd.concat([df_shares, df_shares_base]).reset_index(drop = True)
    df_shares['DATE'] = pd.to_datetime(df_shares['DATE'])
    df_shares = df_shares.drop_duplicates(subset = ['DATE', 'PAIS', 'FUENTE', 'CANAL'] + agrupacion_detalle_shares).reset_index(drop = True)
    tf.parquet_act(f'{dir_data_back}P3_parametros_3_share_canal_fuente', df_shares, 'save')

for c in ['VENTA_COLOCADA_PREDICT', 'ORDENES_PREDICT', 'VISITAS_PREDICT']:
    df_shares[c] = np.where(df_shares[c] < 0, 0, df_shares[c])
    df_shares[c] = df_shares[c].fillna(0)

# Graficar los shares

# 250103

# limpieza paises no andes

df_shares = df_shares[~((df_shares['PAIS'].isin(paises_no_andes)) & (df_shares['FUENTE'] != 'SODIMAC'))].reset_index(drop = True)
df_shares

,DATE,PAIS,FUENTE,CANAL,FAMILIA,VENTA_COLOCADA_PREDICT,ORDENES_PREDICT,VISITAS_PREDICT
0,2025-01-01,AR,SODIMAC,APP,01 - Madera Y Tableros,0.208959,0.181784,0.150148
1,2025-01-01,AR,SODIMAC,APP,02 - Puertas/Ventanas/Molduras,0.160120,0.175838,0.154922
2,2025-01-01,AR,SODIMAC,APP,03 - Fierro/Hierro/Acero,0.163305,0.186325,0.198523
3,2025-01-01,AR,SODIMAC,APP,04 - Tabiqueria/Techumbre/Aislacion,0.157717,0.156801,0.185960
4,2025-01-01,AR,SODIMAC,APP,05 - Obra Gruesa,0.255057,0.235355,0.223224
...,...,...,...,...,...,...,...,...
208121,2025-12-31,MX,SODIMAC,WEB,23 - Jardin,0.723927,0.732045,0.779931
208122,2025-12-31,MX,SODIMAC,WEB,25 - Promociones - Soporte Tecnico,0.500000,0.500000,0.566874
208123,2025-12-31,MX,SODIMAC,WEB,26 - Alquiler De Herramientas,1.000000,1.000000,0.683585
208124,2025-12-31,MX,SODIMAC,WEB,27 - Organizacion,0.729666,0.716253,0.721775


In [990]:
df_shares.PAIS.unique()

array(['AR', 'CO', 'PE', 'UY', 'BR', 'CL', 'MX'], dtype=object)

Graficos de los shares

In [991]:
import matplotlib.dates as mdates

In [992]:
df_tuplas_shares = df_shares[['PAIS'] + agrupacion_detalle_shares].drop_duplicates().reset_index(drop = True)
df_tuplas_shares['SELECCION'] = True
df_tuplas_shares

,PAIS,FAMILIA,SELECCION
0,AR,01 - Madera Y Tableros,True
1,AR,02 - Puertas/Ventanas/Molduras,True
2,AR,03 - Fierro/Hierro/Acero,True
3,AR,04 - Tabiqueria/Techumbre/Aislacion,True
4,AR,05 - Obra Gruesa,True
...,...,...,...
187,MX,23 - Jardin,True
188,MX,25 - Promociones - Soporte Tecnico,True
189,MX,26 - Alquiler De Herramientas,True
190,MX,27 - Organizacion,True


In [993]:
df_shares

,DATE,PAIS,FUENTE,CANAL,FAMILIA,VENTA_COLOCADA_PREDICT,ORDENES_PREDICT,VISITAS_PREDICT
0,2025-01-01,AR,SODIMAC,APP,01 - Madera Y Tableros,0.208959,0.181784,0.150148
1,2025-01-01,AR,SODIMAC,APP,02 - Puertas/Ventanas/Molduras,0.160120,0.175838,0.154922
2,2025-01-01,AR,SODIMAC,APP,03 - Fierro/Hierro/Acero,0.163305,0.186325,0.198523
3,2025-01-01,AR,SODIMAC,APP,04 - Tabiqueria/Techumbre/Aislacion,0.157717,0.156801,0.185960
4,2025-01-01,AR,SODIMAC,APP,05 - Obra Gruesa,0.255057,0.235355,0.223224
...,...,...,...,...,...,...,...,...
208121,2025-12-31,MX,SODIMAC,WEB,23 - Jardin,0.723927,0.732045,0.779931
208122,2025-12-31,MX,SODIMAC,WEB,25 - Promociones - Soporte Tecnico,0.500000,0.500000,0.566874
208123,2025-12-31,MX,SODIMAC,WEB,26 - Alquiler De Herramientas,1.000000,1.000000,0.683585
208124,2025-12-31,MX,SODIMAC,WEB,27 - Organizacion,0.729666,0.716253,0.721775


In [994]:
# Define colores y estilos
color_dict = {'SIS': 'silver', 'ES': 'salmon', 'SODIMAC': 'dodgerblue'}
linestyle_dict = {'APP': '--', 'WEB': '-'}
line_width = 2.5  # Grosor de las líneas

In [995]:
mostrar_graficos = False # parámetro

In [996]:
df_tuplas_shares = df_tuplas_shares[df_tuplas_shares['PAIS'].isin(paises)].reset_index(drop = True)

In [997]:
modo_ejecucion = 'p' # 'c'

In [998]:
for j in range(len(df_tuplas_shares)):
    
    print('\n\n\n Ventas-Ordenes-Visitas tienen que estar bien relacionadas (venta = 0 <-> ordenes = 0)')
    
    if not mostrar_graficos:
        break
    df_seleccion = df_tuplas_shares.iloc[j:j + 1]
    pais = df_seleccion['PAIS'][j]
    df_seleccion = df_shares.merge(df_seleccion, on = ['PAIS'] + agrupacion_detalle_shares, how = 'left')
    df_seleccion['SELECCION'] = df_seleccion['SELECCION'].fillna(False)
    df_seleccion = df_seleccion[df_seleccion['SELECCION']].reset_index(drop = True)
    df_seleccion = df_seleccion.drop(columns = 'SELECCION')
    
    # Hacer subplots
    fig, axes = plt.subplots(1, 3, figsize = (15, 5))
    
    #display(df_seleccion)
    
    for i, metrica in enumerate(['VENTA_COLOCADA', 'ORDENES', 'VISITAS']):
        # Filtrar datos para la métrica actual
        df_pais_metrica = df_seleccion[['DATE', 'FUENTE', 'CANAL'] + [f'{metrica}_PREDICT']].copy()
        df_pais_metrica['FUENTE_CANAL'] = df_pais_metrica['FUENTE'] + '_' + df_pais_metrica['CANAL']
        df_pais_metrica = df_pais_metrica.drop(columns = ['FUENTE', 'CANAL'])
        df_pais_metrica = df_pais_metrica.sort_values('DATE').reset_index(drop = True)
        
        # Configurar título del subplot
        ax = axes[i]
        title = f'{pais} | {metrica}'
        for c in agrupacion_detalle_shares:
            title += f' | {df_seleccion[c][j]}'
        ax.set_title(title)
        

        # Graficar cada combinación de fuente y canal con color y estilo definidos
        for fuente_canal in df_pais_metrica['FUENTE_CANAL'].unique():
            df_aux = df_pais_metrica[df_pais_metrica['FUENTE_CANAL'] == fuente_canal]
            fuente, canal = fuente_canal.split('_')
            
            ax.plot(df_aux['DATE'], df_aux[f'{metrica}_PREDICT'], 
                    label = fuente_canal, 
                    color = color_dict.get(fuente, 'black'), 
                    linestyle = linestyle_dict.get(canal, '-'), 
                    linewidth = line_width)
        
        # Configuración de la leyenda y formato de fecha en el eje x
        ax.legend()
        ax.xaxis.set_major_locator(mdates.MonthLocator(interval = 6))  # Fechas cada 6 meses
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))  # Formato de fecha
        
        # Rotar etiquetas del eje x
        ax.tick_params(axis = 'x', rotation = 45)

    # Ajustar diseño y mostrar
    plt.tight_layout()
    plt.show()
    
    #if modo_ejecucion != 'c':
    #    modo_ejecucion = str(input('Presiona cualquier tecla para continuar pausado  Si presionas la tecla c, se continuará de forma automática hasta terminar'))
    




 Ventas-Ordenes-Visitas tienen que estar bien relacionadas (venta = 0 <-> ordenes = 0)


### 3.6.2. Apertura a nivel granular

In [999]:
# Cada dato real se apertura según el día histórico, y cada dato predict, según el día comparativo (referencia estacional), independiente de la combinación canal - fuente

In [1000]:
# El share es apertura por grano (sin agrupacion_detalle_shares), es decir, como se distribuye cada combinacióon date, pais, fuente, canal + agrupacion_detalle_shares
df_detalle_comparacion = df_representantes_base.rename(columns = {'DATE': 'DATE_COMPARACION'}) # data histórica orgánica
df_dias_predecir['DATE_COMPARACION'], df_detalle_comparacion['DATE_COMPARACION'] = pd.to_datetime(df_dias_predecir['DATE_COMPARACION']), pd.to_datetime(df_detalle_comparacion['DATE_COMPARACION']) 
df_detalle_comparacion = df_dias_predecir.merge(df_detalle_comparacion, on = ['DATE_COMPARACION', 'PAIS'], how = 'left') # se lleva a data futura para los shares
df_detalle_comparacion = df_detalle_comparacion.drop(columns = ['DATE_COMPARACION', 'DIA_SEMANA', 'EVENTO'])
df_detalle_comparacion = df_detalle_comparacion[['DATE', 'PAIS'] + campos_grano + ['VENTA_COLOCADA', 'ORDENES', 'VISITAS']].groupby(['DATE', 'PAIS'] + campos_grano).sum().reset_index()
df_detalle_comparacion['CUENTA'] = 1
df_detalle_comparacion

,DATE,PAIS,FAMILIA,TIPO_MEDIO,VENTA_COLOCADA,ORDENES,VISITAS,CUENTA
0,2025-01-01,MX,01 - Madera Y Tableros,RRSS - Organico,0.000000,0.000000,1.000000,1
1,2025-01-01,MX,01 - Madera Y Tableros,SEO - Brand,0.000000,0.000000,75.000000,1
2,2025-01-01,MX,01 - Madera Y Tableros,SEO - Non Brand,1935.150000,1.000000,228.000000,1
3,2025-01-01,MX,01 - Madera Y Tableros,Typed,215.387688,1.733333,285.733333,1
4,2025-01-01,MX,02 - Puertas/Ventanas/Molduras,Internal,0.000000,0.000000,4.000000,1
...,...,...,...,...,...,...,...,...
66977,2025-12-31,MX,Unspecified,RRSS - Organico,0.000000,0.000000,51.000000,1
66978,2025-12-31,MX,Unspecified,Reference Domain,0.000000,0.000000,12.000000,1
66979,2025-12-31,MX,Unspecified,SEO - Brand,1097.000000,1.000000,2473.000000,1
66980,2025-12-31,MX,Unspecified,SEO - Non Brand,1299.000000,1.000000,2861.000000,1


In [1001]:
agrupacion_detalle_shares

['FAMILIA']

In [1002]:
campos_grano

['FAMILIA', 'TIPO_MEDIO']

In [1003]:
df_detalle_comparacion_agr = df_detalle_comparacion[['DATE', 'PAIS'] + ['VENTA_COLOCADA', 'ORDENES', 'VISITAS', 'CUENTA']].groupby(['DATE', 'PAIS']).sum().reset_index() # data agregada para calcular share del grano
for m in ['VENTA_COLOCADA', 'ORDENES', 'VISITAS']:
    df_detalle_comparacion_agr = df_detalle_comparacion_agr.rename(columns = {m: f'{m}_AGR'})

df_detalle_comparacion = df_detalle_comparacion.drop(columns = 'CUENTA')
df_detalle_comparacion = df_detalle_comparacion.merge(df_detalle_comparacion_agr, on = ['DATE', 'PAIS'], how = 'left') # Cálculo del share para cada grano

for m in ['VENTA_COLOCADA', 'ORDENES', 'VISITAS']:
    df_detalle_comparacion['SHARE_' + m] = df_detalle_comparacion[m] / df_detalle_comparacion[f'{m}_AGR']
    df_detalle_comparacion['SHARE_' + m] = df_detalle_comparacion['SHARE_' + m].fillna(1 / df_detalle_comparacion['CUENTA'])
    df_detalle_comparacion = df_detalle_comparacion.drop(columns = [m, f'{m}_AGR'])
df_detalle_comparacion = df_detalle_comparacion.drop(columns = 'CUENTA')
df_detalle_comparacion

,DATE,PAIS,FAMILIA,TIPO_MEDIO,SHARE_VENTA_COLOCADA,SHARE_ORDENES,SHARE_VISITAS
0,2025-01-01,MX,01 - Madera Y Tableros,RRSS - Organico,0.000000,0.000000,0.000024
1,2025-01-01,MX,01 - Madera Y Tableros,SEO - Brand,0.000000,0.000000,0.001820
2,2025-01-01,MX,01 - Madera Y Tableros,SEO - Non Brand,0.001828,0.012755,0.005532
3,2025-01-01,MX,01 - Madera Y Tableros,Typed,0.000203,0.022109,0.006933
4,2025-01-01,MX,02 - Puertas/Ventanas/Molduras,Internal,0.000000,0.000000,0.000097
...,...,...,...,...,...,...,...
66977,2025-12-31,MX,Unspecified,RRSS - Organico,0.000000,0.000000,0.001246
66978,2025-12-31,MX,Unspecified,Reference Domain,0.000000,0.000000,0.000293
66979,2025-12-31,MX,Unspecified,SEO - Brand,0.002805,0.011142,0.060422
66980,2025-12-31,MX,Unspecified,SEO - Non Brand,0.003322,0.011142,0.069902


### 3.6.3. Jerarquización

In [1004]:
# Llevar la proyeccion agregada, al grano

In [1005]:
df_parametros

,NOMBRE,PAIS,MODO_PROYECCION,METRICA,DATE,real,predict,predict 5.0%,predict 95.0%
0,MX_PRESUPUESTO_TP_ORGANICO,MX,PRESUPUESTO,TP,2021-01-01,NaN,2051.456055,1499.181030,3376.771973
1,MX_PRESUPUESTO_TP_ORGANICO,MX,PRESUPUESTO,TP,2021-01-02,NaN,1881.715820,1293.135132,3216.124512
2,MX_PRESUPUESTO_TP_ORGANICO,MX,PRESUPUESTO,TP,2021-01-03,NaN,1782.404053,1341.511475,3059.372070
3,MX_PRESUPUESTO_TP_ORGANICO,MX,PRESUPUESTO,TP,2021-01-04,NaN,2179.326660,1704.687500,3379.052246
4,MX_PRESUPUESTO_TP_ORGANICO,MX,PRESUPUESTO,TP,2021-01-05,NaN,2140.540527,1567.987183,3509.779297
...,...,...,...,...,...,...,...,...,...
10763,MX_LIBRE_VISITAS_ORGANICO,MX,LIBRE,VISITAS,2025-12-27,NaN,39359.500000,29772.962891,69302.835938
10764,MX_LIBRE_VISITAS_ORGANICO,MX,LIBRE,VISITAS,2025-12-28,NaN,39588.843750,29988.480469,68194.937500
10765,MX_LIBRE_VISITAS_ORGANICO,MX,LIBRE,VISITAS,2025-12-29,NaN,57082.843750,44862.132812,89184.289062
10766,MX_LIBRE_VISITAS_ORGANICO,MX,LIBRE,VISITAS,2025-12-30,NaN,50649.281250,41405.011719,76467.992188


In [1006]:
df_parametros['DATE'], df_representantes_base['DATE'] = pd.to_datetime(df_parametros['DATE']), pd.to_datetime(df_representantes_base['DATE'])

Data Histórica

In [1007]:
df_representantes_base # Para completar la data real

,PAIS,DATE,FUENTE,CANAL,FAMILIA,TIPO_MEDIO,VENTA_COLOCADA,ORDENES,VISITAS
0,AR,2021-04-05,SODIMAC,WEB,06 - Plomeria / Gasfiteria,SEO,40062.9,3.0,1632.0
1,AR,2021-04-05,SODIMAC,WEB,01 - Madera Y Tableros,SEO,12909.0,1.0,1032.0
2,AR,2021-04-05,SODIMAC,WEB,25 - Promociones - Soporte Tecnico,SEO,0.0,0.0,57.0
3,AR,2021-04-05,SODIMAC,WEB,20 - Aseo,SEO,3068.0,2.0,867.0
4,AR,2021-04-05,SODIMAC,WEB,08 - Accesorios Automoviles,SEO,809.0,1.0,576.0
...,...,...,...,...,...,...,...,...,...
3455852,UY,2025-01-22,SODIMAC,WEB,05 - Obra Gruesa,Reference Domain,0.0,0.0,6.0
3455853,UY,2025-01-22,SODIMAC,WEB,09 - Herramientas Y Maquinarias,Reference Domain,0.0,0.0,4.0
3455854,UY,2025-01-22,SODIMAC,WEB,14 - Pisos,Reference Domain,0.0,0.0,16.0
3455855,UY,2025-01-22,SODIMAC,WEB,08 - Accesorios Automoviles,Reference Domain,0.0,0.0,4.0


In [1008]:
df_representantes_base['CUENTA'] = 1
df_detalle_agr = df_representantes_base[['DATE', 'PAIS', 'VENTA_COLOCADA', 'ORDENES', 'VISITAS', 'CUENTA']].groupby(['DATE', 'PAIS']).sum().reset_index() # data agregada para calcular share del grano
df_detalle = df_representantes_base.drop(columns = 'CUENTA')
for m in ['VENTA_COLOCADA', 'ORDENES', 'VISITAS']:
    df_detalle_agr = df_detalle_agr.rename(columns = {m: f'{m}_AGR'})
df_detalle_hist = df_detalle.merge(df_detalle_agr, on = ['DATE', 'PAIS'], how = 'left') # Cálculo del share para cada grano

for m in ['VENTA_COLOCADA', 'ORDENES', 'VISITAS']:
    df_detalle_hist['SHARE_' + m] = df_detalle_hist[m] / df_detalle_hist[f'{m}_AGR']
    df_detalle_hist['SHARE_' + m] = df_detalle_hist['SHARE_' + m].fillna(1 / df_detalle_hist['CUENTA'])
    df_detalle_hist = df_detalle_hist.drop(columns = [m, f'{m}_AGR'])

df_detalle_hist = df_detalle_hist.drop(columns = 'CUENTA')
df_detalle_hist # Por date-pais, como se distribuye en todos los demás (solo para históricos)

,PAIS,DATE,FUENTE,CANAL,FAMILIA,TIPO_MEDIO,SHARE_VENTA_COLOCADA,SHARE_ORDENES,SHARE_VISITAS
0,AR,2021-04-05,SODIMAC,WEB,06 - Plomeria / Gasfiteria,SEO,0.034043,0.025210,0.025866
1,AR,2021-04-05,SODIMAC,WEB,01 - Madera Y Tableros,SEO,0.010969,0.008403,0.016357
2,AR,2021-04-05,SODIMAC,WEB,25 - Promociones - Soporte Tecnico,SEO,0.000000,0.000000,0.000903
3,AR,2021-04-05,SODIMAC,WEB,20 - Aseo,SEO,0.002607,0.016807,0.013741
4,AR,2021-04-05,SODIMAC,WEB,08 - Accesorios Automoviles,SEO,0.000687,0.008403,0.009129
...,...,...,...,...,...,...,...,...,...
3455852,UY,2025-01-22,SODIMAC,WEB,05 - Obra Gruesa,Reference Domain,0.000000,0.000000,0.000272
3455853,UY,2025-01-22,SODIMAC,WEB,09 - Herramientas Y Maquinarias,Reference Domain,0.000000,0.000000,0.000182
3455854,UY,2025-01-22,SODIMAC,WEB,14 - Pisos,Reference Domain,0.000000,0.000000,0.000727
3455855,UY,2025-01-22,SODIMAC,WEB,08 - Accesorios Automoviles,Reference Domain,0.000000,0.000000,0.000182


In [1009]:
df_parametros_historicos = df_parametros[df_parametros['DATE'] < pd.to_datetime(dia_predecir_desde)] # Si no aparece nada en real, es porque el predict no está con toda la data actualizada

for campo in ['real', 'predict', 'predict 5.0%', 'predict 95.0%']:
    print(campo)
    if not ejecutar[3]['Consolidar']:
        break
    df_parametros_historicos_campo = df_parametros_historicos[['PAIS', 'MODO_PROYECCION', 'METRICA', 'DATE', campo]]
    
    df_parametros_historicos_campo = df_parametros_historicos_campo.pivot(index = ['PAIS', 'MODO_PROYECCION', 'DATE'], columns = 'METRICA', values = campo).reset_index()
    df_parametros_historicos_campo['ORDENES'] = df_parametros_historicos_campo['VISITAS'] * df_parametros_historicos_campo['TC']
    df_parametros_historicos_campo['VENTA_COLOCADA'] = df_parametros_historicos_campo['ORDENES'] * df_parametros_historicos_campo['TP']
    df_parametros_historicos_campo = df_parametros_historicos_campo.drop(columns = ['TC', 'TP'])
        
    df_parametros_historicos_campo = df_parametros_historicos_campo.merge(df_detalle_hist, on = ['PAIS', 'DATE',], how = 'left')
    
    for m in ['VENTA_COLOCADA', 'ORDENES', 'VISITAS']:
        df_parametros_historicos_campo[m] = df_parametros_historicos_campo[m] * df_parametros_historicos_campo['SHARE_' + m]
        df_parametros_historicos_campo = df_parametros_historicos_campo.drop(columns = ['SHARE_' + m])
    
    df_parametros_historicos_campo['TC'] = df_parametros_historicos_campo['ORDENES'] / df_parametros_historicos_campo['VISITAS']
    df_parametros_historicos_campo['TP'] = df_parametros_historicos_campo['VENTA_COLOCADA'] / df_parametros_historicos_campo['ORDENES']
    df_parametros_historicos_campo = df_parametros_historicos_campo.drop(columns = ['ORDENES', 'VENTA_COLOCADA'])
    df_parametros_historicos_campo = df_parametros_historicos_campo.melt(id_vars = ['PAIS', 'MODO_PROYECCION', 'FUENTE', 'CANAL', 'DATE'] + campos_grano, var_name = 'METRICA', value_name = campo)
    
    if campo == 'real':
        df_historico = df_parametros_historicos_campo.copy()
    else:
        df_historico = df_historico.merge(df_parametros_historicos_campo, on = ['PAIS', 'MODO_PROYECCION', 'FUENTE', 'CANAL', 'DATE', 'METRICA'] + campos_grano, how = 'outer')

if ejecutar[3]['Consolidar']:
    df_historico.insert(0, 'NOMBRE', df_historico['PAIS'] + '_' + df_historico['MODO_PROYECCION'] + '_' + df_historico['METRICA'] + '_ORGANICO') # como primera columna
    display(df_historico)

real


Data Futura

In [1010]:
#Inputs
display(df_detalle_comparacion.head()) # df_detalle_comparacion: El share es apertura por grano , es decir, como se distribuye cada combinacióon date, pais (usar primero)
display(df_shares.head()) # cada pais-date-agrupacion_detalle_shares, como se distribuye en fuente-canal (usar segundo)

# Ej como está ahora:
# grano = FAMILIA - TIPO_MEDIO
# agrupacion_detalle_shares = FAMILIA

# Esto es:
# df_detalle_comparacion: cada pais-date, como se distribuye en grano (FAMILIA-TIPO_MEDIO)
# df_shares: cada pais-date-FAMILIA, como se distribuye en fuente-canal

,DATE,PAIS,FAMILIA,TIPO_MEDIO,SHARE_VENTA_COLOCADA,SHARE_ORDENES,SHARE_VISITAS
0,2025-01-01,MX,01 - Madera Y Tableros,RRSS - Organico,0.000000,0.000000,0.000024
1,2025-01-01,MX,01 - Madera Y Tableros,SEO - Brand,0.000000,0.000000,0.001820
2,2025-01-01,MX,01 - Madera Y Tableros,SEO - Non Brand,0.001828,0.012755,0.005532
3,2025-01-01,MX,01 - Madera Y Tableros,Typed,0.000203,0.022109,0.006933
4,2025-01-01,MX,02 - Puertas/Ventanas/Molduras,Internal,0.000000,0.000000,0.000097


,DATE,PAIS,FUENTE,CANAL,FAMILIA,VENTA_COLOCADA_PREDICT,ORDENES_PREDICT,VISITAS_PREDICT
0,2025-01-01,AR,SODIMAC,APP,01 - Madera Y Tableros,0.208959,0.181784,0.150148
1,2025-01-01,AR,SODIMAC,APP,02 - Puertas/Ventanas/Molduras,0.160120,0.175838,0.154922
2,2025-01-01,AR,SODIMAC,APP,03 - Fierro/Hierro/Acero,0.163305,0.186325,0.198523
3,2025-01-01,AR,SODIMAC,APP,04 - Tabiqueria/Techumbre/Aislacion,0.157717,0.156801,0.185960
4,2025-01-01,AR,SODIMAC,APP,05 - Obra Gruesa,0.255057,0.235355,0.223224


In [1011]:
df_detalle_comparacion

,DATE,PAIS,FAMILIA,TIPO_MEDIO,SHARE_VENTA_COLOCADA,SHARE_ORDENES,SHARE_VISITAS
0,2025-01-01,MX,01 - Madera Y Tableros,RRSS - Organico,0.000000,0.000000,0.000024
1,2025-01-01,MX,01 - Madera Y Tableros,SEO - Brand,0.000000,0.000000,0.001820
2,2025-01-01,MX,01 - Madera Y Tableros,SEO - Non Brand,0.001828,0.012755,0.005532
3,2025-01-01,MX,01 - Madera Y Tableros,Typed,0.000203,0.022109,0.006933
4,2025-01-01,MX,02 - Puertas/Ventanas/Molduras,Internal,0.000000,0.000000,0.000097
...,...,...,...,...,...,...,...
66977,2025-12-31,MX,Unspecified,RRSS - Organico,0.000000,0.000000,0.001246
66978,2025-12-31,MX,Unspecified,Reference Domain,0.000000,0.000000,0.000293
66979,2025-12-31,MX,Unspecified,SEO - Brand,0.002805,0.011142,0.060422
66980,2025-12-31,MX,Unspecified,SEO - Non Brand,0.003322,0.011142,0.069902


In [1012]:
df_shares

,DATE,PAIS,FUENTE,CANAL,FAMILIA,VENTA_COLOCADA_PREDICT,ORDENES_PREDICT,VISITAS_PREDICT
0,2025-01-01,AR,SODIMAC,APP,01 - Madera Y Tableros,0.208959,0.181784,0.150148
1,2025-01-01,AR,SODIMAC,APP,02 - Puertas/Ventanas/Molduras,0.160120,0.175838,0.154922
2,2025-01-01,AR,SODIMAC,APP,03 - Fierro/Hierro/Acero,0.163305,0.186325,0.198523
3,2025-01-01,AR,SODIMAC,APP,04 - Tabiqueria/Techumbre/Aislacion,0.157717,0.156801,0.185960
4,2025-01-01,AR,SODIMAC,APP,05 - Obra Gruesa,0.255057,0.235355,0.223224
...,...,...,...,...,...,...,...,...
208121,2025-12-31,MX,SODIMAC,WEB,23 - Jardin,0.723927,0.732045,0.779931
208122,2025-12-31,MX,SODIMAC,WEB,25 - Promociones - Soporte Tecnico,0.500000,0.500000,0.566874
208123,2025-12-31,MX,SODIMAC,WEB,26 - Alquiler De Herramientas,1.000000,1.000000,0.683585
208124,2025-12-31,MX,SODIMAC,WEB,27 - Organizacion,0.729666,0.716253,0.721775


In [1013]:
df_shares['DATE'], df_detalle_comparacion['DATE'] = pd.to_datetime(df_shares['DATE']), pd.to_datetime(df_detalle_comparacion['DATE'])
df_shares_detalle = df_detalle_comparacion.merge(df_shares, on = ['DATE', 'PAIS'] + agrupacion_detalle_shares, how = 'left')
for m in ['VENTA_COLOCADA', 'ORDENES', 'VISITAS']:
    df_shares_detalle[f'SHARE_{m}'] = df_shares_detalle[f'SHARE_{m}']  * df_shares_detalle[f'{m}_PREDICT']
    df_shares_detalle = df_shares_detalle.drop(columns = [f'{m}_PREDICT'])
df_shares_detalle

,DATE,PAIS,FAMILIA,TIPO_MEDIO,SHARE_VENTA_COLOCADA,SHARE_ORDENES,SHARE_VISITAS,FUENTE,CANAL
0,2025-01-01,MX,01 - Madera Y Tableros,RRSS - Organico,NaN,NaN,NaN,NaN,NaN
1,2025-01-01,MX,01 - Madera Y Tableros,SEO - Brand,NaN,NaN,NaN,NaN,NaN
2,2025-01-01,MX,01 - Madera Y Tableros,SEO - Non Brand,NaN,NaN,NaN,NaN,NaN
3,2025-01-01,MX,01 - Madera Y Tableros,Typed,NaN,NaN,NaN,NaN,NaN
4,2025-01-01,MX,02 - Puertas/Ventanas/Molduras,Internal,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
132124,2025-12-31,MX,Unspecified,SEO - Brand,0.001686,0.006909,0.043856,SODIMAC,WEB
132125,2025-12-31,MX,Unspecified,SEO - Non Brand,0.001325,0.004233,0.019165,SODIMAC,APP
132126,2025-12-31,MX,Unspecified,SEO - Non Brand,0.001996,0.006909,0.050737,SODIMAC,WEB
132127,2025-12-31,MX,Unspecified,Typed,0.000000,0.000000,0.027818,SODIMAC,APP


In [1014]:
#sys.exit('Revisar df shares')

In [1015]:
print('OK hasta aqui 241105_0825...el bloque de abajo debería estar bien también (validar)')

OK hasta aqui 241105_0825...el bloque de abajo debería estar bien también (validar)


In [1016]:
df_parametros_futuros = df_parametros[df_parametros['DATE'] >= pd.to_datetime(dia_predecir_desde)] 

for campo in ['predict', 'predict 5.0%', 'predict 95.0%']:
    if not ejecutar[3]['Consolidar']:
        break
    df_parametros_futuros_campo = df_parametros_futuros[['PAIS', 'MODO_PROYECCION', 'METRICA', 'DATE', campo]]
    df_parametros_futuros_campo = df_parametros_futuros_campo.pivot(index = ['PAIS', 'MODO_PROYECCION', 'DATE'], columns = 'METRICA', values = campo).reset_index()
    df_parametros_futuros_campo['ORDENES'] = df_parametros_futuros_campo['VISITAS'] * df_parametros_futuros_campo['TC']
    df_parametros_futuros_campo['VENTA_COLOCADA'] = df_parametros_futuros_campo['ORDENES'] * df_parametros_futuros_campo['TP']
    df_parametros_futuros_campo = df_parametros_futuros_campo.drop(columns = ['TC', 'TP'])
    
    df_parametros_futuros_campo = df_parametros_futuros_campo.merge(df_shares_detalle, on = ['PAIS', 'DATE'], how = 'left')
    for m in ['VENTA_COLOCADA', 'ORDENES', 'VISITAS']:
        df_parametros_futuros_campo[m] = df_parametros_futuros_campo[m] * df_parametros_futuros_campo[f'SHARE_{m}']
        df_parametros_futuros_campo = df_parametros_futuros_campo.drop(columns = [f'SHARE_{m}'])
    
    df_parametros_futuros_campo['TC'] = df_parametros_futuros_campo['ORDENES'] / df_parametros_futuros_campo['VISITAS']
    df_parametros_futuros_campo['TP'] = df_parametros_futuros_campo['VENTA_COLOCADA'] / df_parametros_futuros_campo['ORDENES']
    df_parametros_futuros_campo = df_parametros_futuros_campo.drop(columns = ['ORDENES', 'VENTA_COLOCADA'])
    df_parametros_futuros_campo = df_parametros_futuros_campo.melt(id_vars = ['PAIS', 'MODO_PROYECCION', 'FUENTE', 'CANAL', 'DATE'] + campos_grano, var_name = 'METRICA', value_name = campo)
    
    if campo == 'predict':
        df_futuro = df_parametros_futuros_campo.copy()
    else:
        df_futuro = df_futuro.merge(df_parametros_futuros_campo, on = ['PAIS', 'MODO_PROYECCION', 'FUENTE', 'CANAL', 'DATE', 'METRICA'] + campos_grano, how = 'outer')

if ejecutar[3]['Consolidar']:
    df_futuro.insert(0, 'NOMBRE', df_futuro['PAIS'] + '_' + df_futuro['MODO_PROYECCION'] + '_' + df_futuro['METRICA'] + '_ORGANICO') # como primera columna
    display(df_futuro.head())

In [1017]:
if ejecutar[3]['Consolidar']: # Parámetro 3 es histórico + futuro (visión completa)
    df_parametro_3 = pd.concat([df_historico, df_futuro])
    tf.parquet_act(f'{dir_data}Parametros/P3', df_parametro_3, 'save')

# 4. Visitas pago (Inversión)

In [1018]:
f'Tiempo total hasta ahora: {round((time.time() - time_0) / 60, 2)} minutos'

'Tiempo total hasta ahora: 3.3 minutos'

In [1019]:
# curve fitting inversion (crear funcion)

def curve_fitting_inversion(df_data_tupla, best_ecm, n_iters, inicial_guess, n_cross_validation = 1):

    con_inicial_guess = True
    if all(np.isnan(elemento) for elemento in inicial_guess): # Si todos son nan
        con_inicial_guess = False

    df_proyeccion = df_data_tupla[['DATE', 'VISITAS_IN', 'INVERSION']]
    df_proyeccion['t'] = (df_proyeccion['DATE'] - dt.datetime(2021, 1, 1)).dt.days # 1/1/2021 es cualquier dia de referencia
    df_proyeccion[['VISITAS_IN', 'INVERSION']] = df_proyeccion[['VISITAS_IN', 'INVERSION']].fillna(0)
    df_proyeccion = df_proyeccion[['DATE', 't', 'VISITAS_IN', 'INVERSION']]
    df_proyeccion = df_proyeccion[df_proyeccion['INVERSION'] > 0] # Evita casos sin inversión que podrían tener tráfico

    # Función a ajustar
    def func(X, a, c, n):
        i, t = X
        return a * (i ** n) + c * t * (i ** n)

    best_params = inicial_guess # Por defecto, si nunca entra al guess, se mantienen
    for j in range(n_cross_validation):
        
        try:
            df_train, df_test = train_test_split(df_proyeccion, test_size = 0.2)
        except:
            df_train, df_test = df_proyeccion.copy(), df_proyeccion.copy()
        df_train = df_train.sort_values('DATE').reset_index(drop = True)
        df_test = df_test.sort_values('DATE').reset_index(drop = True)

        # Datos de ejemplo
        t = np.array(df_train['t'])
        i = np.array(df_train['INVERSION'])
        V = np.array(df_train['VISITAS_IN'])  # Valores observados de z

        # Combinar x e y en una sola variable
        X = np.vstack((i, t))

        # Ajustar la función}
        
        popt = [0, 0, 0]
        maxfev_aux = 1000
        for i in range(5):
            try:
                if con_inicial_guess:
                    popt, pcov = curve_fit(func, X, V, p0 = inicial_guess, bounds = ([0, 0, 0.3], [float('inf'), float('inf'), 1]), maxfev = maxfev_aux)# A y C >=0 y n in [0, 1]
                else:
                    popt, pcov = curve_fit(func, X, V, bounds = ([0, 0, 0.3], [float('inf'), float('inf'), 1]), maxfev = maxfev_aux)# A y C >=0 y n in [0, 1]
            except:
                maxfev_aux *= 2
                
        # Resultados del ajuste
        a, c, n = popt
        df_test['VISITAS_PRED'] = df_test.apply(lambda x: func([x['INVERSION'], x['t']], a, c, n), axis = 1)
        ecm = np.mean((df_test['VISITAS_IN'] - df_test['VISITAS_PRED']) ** 2)
        if ecm < best_ecm:
            best_ecm = ecm
            best_params = popt
    
    n_iters += n_cross_validation
    return best_params, best_ecm, n_iters


## 4.1. Tuplas & data

In [1020]:
df_data = tf.parquet_act(f'{dir_data_back}P2_pars2')
df_data['DATE'] = pd.to_datetime(df_data['DATE'])

df_data = df_data.drop_duplicates().reset_index(drop = True)
df_combinaciones = df_data[['PAIS', 'CANAL', 'FUENTE'] + campos_grano].drop_duplicates().reset_index(drop = True)
df_combinaciones = df_combinaciones.fillna('None')
df_combinaciones = df_combinaciones[df_combinaciones['PAIS'].isin(paises)].reset_index(drop = True)
df_combinaciones

,PAIS,CANAL,FUENTE,FAMILIA,TIPO_MEDIO
0,MX,WEB,SODIMAC,06 - Plomeria / Gasfiteria,SEM
1,MX,WEB,SODIMAC,16 - Electrohogar,SEM
2,MX,WEB,SODIMAC,22 - Aire Libre,SEM
3,MX,WEB,SODIMAC,20 - Aseo,SEM
4,MX,WEB,SODIMAC,04 - Tabiqueria/Techumbre/Aislacion,Videos
...,...,...,...,...,...
317,MX,WEB,SODIMAC,25 - Promociones - Soporte Tecnico,Remarketing
318,MX,WEB,SODIMAC,25 - Promociones - Soporte Tecnico,SEM - Shopping
319,MX,WEB,SODIMAC,25 - Promociones - Soporte Tecnico,Prospecting
320,MX,WEB,SODIMAC,25 - Promociones - Soporte Tecnico,RRSS - Pago


In [1021]:
if 'P4_df_registros.pkl' in os.listdir(dir_data_back):
    print('SI')
    df_registros_P4 = tf.parquet_act(f'{dir_data_back}P4_df_registros')
    df_combinaciones = df_combinaciones.merge(df_registros_P4, on = ['PAIS', 'CANAL', 'FUENTE'] + campos_grano, how = 'left')
    df_combinaciones['best_ecm'], df_combinaciones['a'], df_combinaciones['c'], df_combinaciones['n'], df_combinaciones['n_iters'] = df_combinaciones['best_ecm'].fillna(float('inf')), df_combinaciones['a'].fillna(np.nan), df_combinaciones['c'].fillna(np.nan), df_combinaciones['n'].fillna(np.nan), df_combinaciones['n_iters'].fillna(0)
else:
    df_combinaciones['best_ecm'], df_combinaciones['a'], df_combinaciones['c'], df_combinaciones['n'], df_combinaciones['n_iters'] = float('inf'), np.nan, np.nan, np.nan, 0

df_combinaciones

SI


,PAIS,CANAL,FUENTE,FAMILIA,TIPO_MEDIO,best_ecm,a,c,n,n_iters,BATCHES
0,MX,WEB,SODIMAC,06 - Plomeria / Gasfiteria,SEM,inf,NaN,NaN,NaN,370,37
1,MX,WEB,SODIMAC,16 - Electrohogar,SEM,inf,NaN,NaN,NaN,370,37
2,MX,WEB,SODIMAC,22 - Aire Libre,SEM,inf,NaN,NaN,NaN,370,37
3,MX,WEB,SODIMAC,20 - Aseo,SEM,inf,NaN,NaN,NaN,370,37
4,MX,WEB,SODIMAC,04 - Tabiqueria/Techumbre/Aislacion,Videos,481.004201,3.272786e-20,1.012829e-02,0.441224,370,37
...,...,...,...,...,...,...,...,...,...,...,...
317,MX,WEB,SODIMAC,25 - Promociones - Soporte Tecnico,Remarketing,0.000662,1.044656e+00,4.690505e-14,0.300000,370,37
318,MX,WEB,SODIMAC,25 - Promociones - Soporte Tecnico,SEM - Shopping,0.216098,2.381429e-22,1.019030e-03,0.585416,370,37
319,MX,WEB,SODIMAC,25 - Promociones - Soporte Tecnico,Prospecting,0.004447,1.222695e+00,2.271000e-16,0.300000,370,37
320,MX,WEB,SODIMAC,25 - Promociones - Soporte Tecnico,RRSS - Pago,0.005876,1.055677e+00,5.440326e-14,0.300000,370,37


In [1022]:
df_data

,PAIS,DATE,CANAL,FUENTE,FAMILIA,TIPO_MEDIO,VENTA_COLOCADA,ORDENES,VISITAS,INVERSION
0,MX,2021-04-05,WEB,SODIMAC,06 - Plomeria / Gasfiteria,SEM,3978.0,2.0,1325.0,743.174459
1,MX,2021-04-05,WEB,SODIMAC,16 - Electrohogar,SEM,1699.0,1.0,465.0,575.869904
2,MX,2021-04-05,WEB,SODIMAC,22 - Aire Libre,SEM,2884.0,2.0,1268.0,2768.317180
3,MX,2021-04-05,WEB,SODIMAC,20 - Aseo,SEM,0.0,0.0,830.0,527.922302
4,MX,2021-04-05,WEB,SODIMAC,04 - Tabiqueria/Techumbre/Aislacion,Videos,0.0,0.0,1.0,0.130521
...,...,...,...,...,...,...,...,...,...,...
3150205,CO,2025-01-22,WEB,SODIMAC,Unspecified,SEM - Non Brand,0.0,1.0,5376.0,61855.776290
3150206,CO,2025-01-22,WEB,SODIMAC,12 - Pintura Y Accesorios,SEM - Non Brand,33900.0,1.0,249.0,19641.120044
3150207,CO,2025-01-22,WEB,SODIMAC,18 - Menaje,SEM - Non Brand,415200.0,3.0,469.0,46788.080642
3150208,CO,2025-01-22,WEB,SODIMAC,05 - Obra Gruesa,SEM - Non Brand,2290000.0,2.0,226.0,19641.184784


Visitas IN

In [1023]:
paises_visitas_in = ['MX', 'CL', 'PE', 'CO', 'BR', 'AR', 'UY']
desde_visitas_in = '2022-01-01'

In [1024]:
df_data_visitas_in = pd.DataFrame()
if 'df_data_visitas_in.pkl' in os.listdir(dir_data_back):
    df_data_visitas_in = tf.parquet_act(f'{dir_data_back}df_data_visitas_in')
df_data_visitas_in

if len(df_data_visitas_in) > 0:
    df_data_visitas_in_existentes = df_data_visitas_in[['DATE', 'PAIS']].drop_duplicates().reset_index(drop = True)
    df_data_visitas_in_existentes['EXISTE'] = True

In [1025]:
campos_beta = ['PAIS', 'DATE', 'FUENTE', 'CANAL'] # Por ahora (ya está declarado)
campos_grano

['FAMILIA', 'TIPO_MEDIO']

In [1026]:
df_medios = tf.lectura_segura_Gsheets('clacomizacion_corp_v0', 'DICCIONARIO_MEDIOS_CORP')
#df_medios = df_medios[['TIPO_MEDIO', 'NATURALEZA_MEDIO']].drop_duplicates().reset_index(drop = True)
df_medios['NATURALEZA_MEDIO'] = np.where(df_medios['NATURALEZA_MEDIO'] == 'Pago', 'PAGO', 'ORGANICO')
df_medios = df_medios.drop(columns = 'LAST_TOUCH_CHANNEL')
df_medios = df_medios.rename(columns = {'LAST_TOUCH_CHANNEL_CORP': 'LAST_TOUCH_CHANNEL'})
df_medios = df_medios.drop_duplicates().reset_index(drop = True)
df_medios.head()

tf.sheets_to_df clacomizacion_corp_v0 DICCIONARIO_MEDIOS_CORP


,LAST_TOUCH_CHANNEL,TIPO_MEDIO,CLASIFICACION_MEDIO,NATURALEZA_MEDIO
0,Afiliados,Afiliados,Propios,ORGANICO
1,SEM - Brand,SEM - Brand,Paid Brand,PAGO
2,Email,Email,Propios,ORGANICO
3,Push App,Push App,Propios,ORGANICO
4,Push Web,Push Web,Propios,ORGANICO


In [1027]:
# df_dates tiene los días desde desde_visitas_in hasta ayer
df_dates = pd.date_range(start = desde_visitas_in, end = dt.datetime.today() - dt.timedelta(1), freq = 'D')
df_dates = pd.DataFrame(df_dates, columns = ['DATE'])
df_dates

,DATE
0,2022-01-01
1,2022-01-02
2,2022-01-03
3,2022-01-04
4,2022-01-05
...,...
1114,2025-01-19
1115,2025-01-20
1116,2025-01-21
1117,2025-01-22


In [1028]:
df_data_visitas_in

,PAIS,DATE,FUENTE,CANAL,FAMILIA,TIPO_MEDIO,VISITAS_IN,COSTO,NATURALEZA_MEDIO
0,CO,2025-01-22,ES,APP,01 - Madera Y Tableros,RRSS - Pago,6,0.0,PAGO
1,CO,2025-01-22,ES,APP,01 - Madera Y Tableros,Remarketing,16,0.0,PAGO
2,CO,2025-01-22,ES,APP,01 - Madera Y Tableros,SEM - Non Brand,2,0.0,PAGO
3,CO,2025-01-22,ES,APP,01 - Madera Y Tableros,SEM - Shopping,16,0.0,PAGO
4,CO,2025-01-22,ES,APP,02 - Puertas/Ventanas/Molduras,Display,2,0.0,PAGO
...,...,...,...,...,...,...,...,...,...
3027566,MX,2025-01-12,SODIMAC,WEB,Unspecified,Prospecting,127,66.846163,PAGO
3027567,MX,2025-01-12,SODIMAC,WEB,Unspecified,RRSS - Pago,1,0.0,PAGO
3027568,MX,2025-01-12,SODIMAC,WEB,Unspecified,Remarketing,647,542.330639,PAGO
3027569,MX,2025-01-12,SODIMAC,WEB,Unspecified,SEM - Brand,6,0.0,PAGO


In [1029]:
paso = 100

In [1030]:
for pais in paises:
    
    df_dates_pais = df_dates.copy()
    df_dates_pais['PAIS'] = pais
    if len(df_data_visitas_in) > 0:
        df_dates_pais = df_dates_pais.merge(df_data_visitas_in_existentes, how = 'left', on = ['DATE', 'PAIS'])
        df_dates_pais['EXISTE'] = df_dates_pais['EXISTE'].fillna(False)
        df_dates_pais = df_dates_pais[df_dates_pais['EXISTE'] != True]
    
    if len(df_dates_pais) == 0: # Si no hay nada, el país está completamente actualizado
        continue
    
    min_date, max_date = str(df_dates_pais['DATE'].min())[:10], str(df_dates_pais['DATE'].max())[:10]
    
    query_detalle = f'PAIS = "{pais}" and DATE >= "{min_date}" and DATE <= "{max_date}"'
    #query_detalle = f'PAIS = "{pais}" and DATE in ("2022-01-01", "2022-01-02", "2024-02-01")'
    
    query = f'SELECT * FROM `sod-corp-plp-beta.ETL_main_2023.VisitasIN_241122_0_v2` where {query_detalle} and duplicacion = "{grano}"'

    # request y df
    df_visitas =  tf.request_GCP_vnew(
            nombre_tabla = "",
            specific_query = query,
            client = client,
            output = True, permitir_fallos = False)

    df_visitas['DATE'] = pd.to_datetime(df_visitas['DATE'])
    df_visitas = df_visitas.merge(df_medios, on = 'LAST_TOUCH_CHANNEL', how = 'left')
    df_visitas = df_visitas[campos_beta + campos_grano + ['VISITAS_IN']].groupby(campos_beta + campos_grano, as_index = False).sum()

    if len(df_visitas) == 0: # SI todavía no está el dato (de todas formas, si no hay nada, nada se cargará y volverá a procesarse...es solo por eficiencia)
        continue
    # request inversión

    query = f'SELECT * FROM `sod-corp-plp-beta.ETL_main_2023.Nexus_Inversion_231128_0930_v4` where {query_detalle}'
        
    # request y df
    df_inversion =  tf.request_GCP_vnew(
            nombre_tabla = "",
            specific_query = query,
            client = client,
            output = True, permitir_fallos = False)

    df_inversion['DATE'] = pd.to_datetime(df_inversion['DATE'])
    df_inversion = df_inversion.rename(columns = {'LAST_TOUCH_CHANNEL_CORP': 'LAST_TOUCH_CHANNEL'})

    df_inversion = df_inversion.merge(df_medios, on = 'LAST_TOUCH_CHANNEL', how = 'left')
    df_inversion = df_inversion[campos_beta + campos_grano + ['COSTO']].groupby(campos_beta + campos_grano, as_index = False).sum()
    df_inversion['FAMILIA'] = np.where(df_inversion['FAMILIA'] == 'A. No clacomizado', 'Unspecified', df_inversion['FAMILIA'])

    # Corrección: Sem se distribuye en los canales según visitas IN
    df_inversion['CANAL'] = np.where(df_inversion['TIPO_MEDIO'].str.contains('SEM'), '', df_inversion['CANAL']) 
    
    ## ALGORÍTMO DE ASIGNACIÓN DE COSTOS
    campos_beta_sin_canal = campos_beta.copy()
    campos_beta_sin_canal.remove('CANAL')

    df_visitas['COSTO'] = 0 # Acumulará toda las asignaciones de costos
    
    df_medios_resumen = df_medios[[campo_last_touch, 'NATURALEZA_MEDIO']].drop_duplicates().reset_index(drop = True)

    for i in tqdm.tqdm(range(len(df_inversion))):
        df_i = df_inversion.iloc[i:i + 1]
        df_i = df_i.rename(columns = {'COSTO': 'COSTO_NEW'})
        #display(df_i)
        lista_merge = []
        for c in campos_grano + ['CANAL']:
            if df_i[c][i] == "":
                continue
            lista_merge.append(c)
        
        df_visitas = df_visitas.merge(df_i[campos_beta_sin_canal + lista_merge + ['COSTO_NEW']], on = campos_beta_sin_canal + lista_merge, how = 'left')
        #print(lista_merge)
        
        # ancla 0

        df_asignacion = df_visitas[df_visitas['COSTO_NEW'].notna()].reset_index(drop = True)
        df_asignacion['COSTO_NEW'] = df_asignacion['COSTO_NEW'] * df_asignacion['VISITAS_IN'] / df_asignacion['VISITAS_IN'].sum()
        df_asignacion = df_asignacion[campos_beta + campos_grano + ['COSTO_NEW']]
        
        #display(df_asignacion)
        
        df_visitas = df_visitas.drop(columns = 'COSTO_NEW')
        df_visitas = df_visitas.merge(df_asignacion, on = campos_beta + campos_grano, how = 'left')
        df_visitas['COSTO_NEW'] = df_visitas['COSTO_NEW'].fillna(0)
        df_visitas['COSTO'] = df_visitas['COSTO'] + df_visitas['COSTO_NEW']
        df_visitas = df_visitas.drop(columns = 'COSTO_NEW')
        
        #display(df_visitas)

    df_visitas = df_visitas.merge(df_medios_resumen, on = campo_last_touch, how = 'left')
    df_visitas = df_visitas[df_visitas['NATURALEZA_MEDIO'] == 'PAGO'].reset_index(drop = True)
    
    display(df_visitas)
    #df_visitas = df_visitas[~((df_visitas['VISITAS_IN'] > 0) & (df_visitas['COSTO'] == 0))]# No se admiten registros con visitas y costo 0
    
    df_data_visitas_in = pd.concat([df_visitas, df_data_visitas_in]).reset_index(drop = True) # Prioridad al nuevo registro
    df_data_visitas_in['DATE'] = pd.to_datetime(df_data_visitas_in['DATE'])
    df_data_visitas_in = df_data_visitas_in.drop_duplicates(subset = campos_beta + campos_grano, keep = 'first').reset_index(drop = True)

    tf.parquet_act(f'{dir_data_back}df_data_visitas_in', df_data_visitas_in, 'save')

SELECT * FROM `sod-corp-plp-beta.ETL_main_2023.VisitasIN_241122_0_v2` where PAIS = "MX" and DATE >= "2025-01-23" and DATE <= "2025-01-23" and duplicacion = "F-LT"
SELECT * FROM `sod-corp-plp-beta.ETL_main_2023.Nexus_Inversion_231128_0930_v4` where PAIS = "MX" and DATE >= "2025-01-23" and DATE <= "2025-01-23"


100%|██████████| 26/26 [00:00<00:00, 127.32it/s]


,PAIS,DATE,FUENTE,CANAL,FAMILIA,TIPO_MEDIO,VISITAS_IN,COSTO,NATURALEZA_MEDIO
0,MX,2025-01-23,SODIMAC,APP,01 - Madera Y Tableros,Display,74,0.0,PAGO
1,MX,2025-01-23,SODIMAC,APP,01 - Madera Y Tableros,SEM - Brand,4,0.0,PAGO
2,MX,2025-01-23,SODIMAC,APP,01 - Madera Y Tableros,SEM - Shopping,124,31.750809,PAGO
3,MX,2025-01-23,SODIMAC,APP,02 - Puertas/Ventanas/Molduras,Display,132,0.0,PAGO
4,MX,2025-01-23,SODIMAC,APP,02 - Puertas/Ventanas/Molduras,SEM - Brand,4,0.0,PAGO
...,...,...,...,...,...,...,...,...,...
173,MX,2025-01-23,SODIMAC,WEB,Unspecified,Display,260,0.0,PAGO
174,MX,2025-01-23,SODIMAC,WEB,Unspecified,Prospecting,488,127.712586,PAGO
175,MX,2025-01-23,SODIMAC,WEB,Unspecified,Remarketing,908,527.905154,PAGO
176,MX,2025-01-23,SODIMAC,WEB,Unspecified,SEM - Brand,10,0.0,PAGO


In [1031]:
df_data = df_data[~df_data['PAIS'].isin(paises_visitas_in)]
df_data = df_data[campos_beta + campos_grano + ['VISITAS', 'INVERSION']].rename(columns = {'VISITAS': 'VISITAS_IN'})

df_data_visitas_in = df_data_visitas_in[campos_beta + campos_grano + ['VISITAS_IN', 'COSTO']].rename(columns = {'COSTO': 'INVERSION'})
df_data = pd.concat([df_data, df_data_visitas_in]).reset_index(drop = True)
df_data = df_data.drop_duplicates(subset = campos_beta + campos_grano, keep = 'first').reset_index(drop = True) # Solo por seguridad
df_data = df_data[~((df_data['VISITAS_IN'] > 0) & (df_data['INVERSION'] == 0))]# No se admiten registros con visitas y costo 0

## 4.2. Entrenamiento

In [1032]:
# Etapa inicial

# Curve fitting con la función

# A * (i ** n) + C * t * (i ** n)

# Que cumpla las condiciones: n pertenece a ]0, 1]
# A >= 0, C >= 0

In [1033]:
df_combinaciones

,PAIS,CANAL,FUENTE,FAMILIA,TIPO_MEDIO,best_ecm,a,c,n,n_iters,BATCHES
0,MX,WEB,SODIMAC,06 - Plomeria / Gasfiteria,SEM,inf,NaN,NaN,NaN,370,37
1,MX,WEB,SODIMAC,16 - Electrohogar,SEM,inf,NaN,NaN,NaN,370,37
2,MX,WEB,SODIMAC,22 - Aire Libre,SEM,inf,NaN,NaN,NaN,370,37
3,MX,WEB,SODIMAC,20 - Aseo,SEM,inf,NaN,NaN,NaN,370,37
4,MX,WEB,SODIMAC,04 - Tabiqueria/Techumbre/Aislacion,Videos,481.004201,3.272786e-20,1.012829e-02,0.441224,370,37
...,...,...,...,...,...,...,...,...,...,...,...
317,MX,WEB,SODIMAC,25 - Promociones - Soporte Tecnico,Remarketing,0.000662,1.044656e+00,4.690505e-14,0.300000,370,37
318,MX,WEB,SODIMAC,25 - Promociones - Soporte Tecnico,SEM - Shopping,0.216098,2.381429e-22,1.019030e-03,0.585416,370,37
319,MX,WEB,SODIMAC,25 - Promociones - Soporte Tecnico,Prospecting,0.004447,1.222695e+00,2.271000e-16,0.300000,370,37
320,MX,WEB,SODIMAC,25 - Promociones - Soporte Tecnico,RRSS - Pago,0.005876,1.055677e+00,5.440326e-14,0.300000,370,37


In [1034]:
min_iteraciones_totales = 40 + 5 * delta_days_iters # Parámetro # 15 *
n_cross_validation = 10 # Parámetro (luego de esto, cambia la tupla):subir a 10 en el futuro (al menos)
paso = 100 # Parámetro

In [1035]:
min_iteraciones_totales

375

In [1036]:
df_combinaciones['BATCHES'] = np.where(min_iteraciones_totales > df_combinaciones['n_iters'], ((min_iteraciones_totales - df_combinaciones['n_iters'] - 1) // (n_cross_validation)) + 1, 0)
total_batches = int(df_combinaciones['BATCHES'].sum())
print('total_batches', total_batches)

total_batches 322


In [1037]:
df_combinaciones

,PAIS,CANAL,FUENTE,FAMILIA,TIPO_MEDIO,best_ecm,a,c,n,n_iters,BATCHES
0,MX,WEB,SODIMAC,06 - Plomeria / Gasfiteria,SEM,inf,NaN,NaN,NaN,370,1
1,MX,WEB,SODIMAC,16 - Electrohogar,SEM,inf,NaN,NaN,NaN,370,1
2,MX,WEB,SODIMAC,22 - Aire Libre,SEM,inf,NaN,NaN,NaN,370,1
3,MX,WEB,SODIMAC,20 - Aseo,SEM,inf,NaN,NaN,NaN,370,1
4,MX,WEB,SODIMAC,04 - Tabiqueria/Techumbre/Aislacion,Videos,481.004201,3.272786e-20,1.012829e-02,0.441224,370,1
...,...,...,...,...,...,...,...,...,...,...,...
317,MX,WEB,SODIMAC,25 - Promociones - Soporte Tecnico,Remarketing,0.000662,1.044656e+00,4.690505e-14,0.300000,370,1
318,MX,WEB,SODIMAC,25 - Promociones - Soporte Tecnico,SEM - Shopping,0.216098,2.381429e-22,1.019030e-03,0.585416,370,1
319,MX,WEB,SODIMAC,25 - Promociones - Soporte Tecnico,Prospecting,0.004447,1.222695e+00,2.271000e-16,0.300000,370,1
320,MX,WEB,SODIMAC,25 - Promociones - Soporte Tecnico,RRSS - Pago,0.005876,1.055677e+00,5.440326e-14,0.300000,370,1


In [1038]:
ejecutar[4]['Entrenar'] = True # ELIMINAR

In [1041]:
for i in tqdm.tqdm(range(total_batches)):
    
    if not ejecutar[4]['Entrenar']:
        break
    
    # Ordenar tabla tuplas creciente en iteraciones
    df_combinaciones = df_combinaciones.sort_values(by = 'n_iters', ascending = True).reset_index(drop = True)
    df_tupla = df_combinaciones.iloc[0:1].reset_index(drop = True)  # Elegimos el caso puntual (etiqueta)

    df_tupla['IN'] = True
    df_data_tupla = df_data[['PAIS', 'CANAL', 'FUENTE', 'DATE'] + campos_grano + ['VISITAS_IN', 'INVERSION']].merge(df_tupla[['PAIS', 'CANAL', 'FUENTE'] + campos_grano + ['IN']], on = ['PAIS', 'CANAL', 'FUENTE'] + campos_grano, how = 'left')
    df_data_tupla['IN'] = df_data_tupla['IN'].fillna(False)
    df_data_tupla = df_data_tupla[df_data_tupla['IN']]
    df_data_tupla = df_data_tupla.drop(columns = 'IN')
    df_data_tupla = df_data_tupla.sort_values('DATE').reset_index(drop = True)
    
    best_ecm = df_tupla['best_ecm'][0]
    inicial_guess = [df_tupla['a'][0], df_tupla['c'][0], df_tupla['n'][0]]
    n_iters = df_tupla['n_iters'][0]
    
    if len(df_data_tupla) == 0:
        best_params = [0, 0, 0]
        best_ecm = float('inf')
        n_iters = min_iteraciones_totales # AL menos por hoy, no seguir procesando. No existirán nuevos datos
        
    elif len(df_data_tupla) == 1:
        best_params = [df_data_tupla['VISITAS_IN'][0] / df_data_tupla['INVERSION'][0], 0, 1] # Aproximación inicial linea directa
        best_ecm = float('inf') 
        n_iters = min_iteraciones_totales # AL menos por hoy, no seguir procesando. No existirán nuevos datos
    
    else:
        best_params, best_ecm, n_iters = curve_fitting_inversion(df_data_tupla, best_ecm, n_iters, inicial_guess, n_cross_validation = n_cross_validation)

    a, c, n = best_params
    df_combinaciones['a'][0], df_combinaciones['c'][0], df_combinaciones['n'][0] = a, c, n
    
    df_combinaciones['best_ecm'][0], df_combinaciones['n_iters'][0] = best_ecm, n_iters
    
    if ((i + 1) % paso == 0) or (i == total_batches - 1): # Guardar
        
        #df_combinaciones_base = pd.DataFrame()
        #if 'P4_df_registros.pkl' in os.listdir(dir_data_back):
        #    df_combinaciones_base = tf.ickle_act(f'{dir_data_back}P4_df_registros')
        
        #df_combinaciones_base = pd.concat([df_combinaciones, df_combinaciones_base]).reset_index(drop = True)
        
        #display(df_combinaciones_base)
        #sys.exit('Revisar este archivo para el drop duplicates 241105')
        #df_combinaciones_base = df_combinaciones_base.drop_duplicates(subset = ['NOMBRE']).reset_index(drop = True)
        tf.parquet_act(f'{dir_data_back}P4_df_registros', df_combinaciones, 'save')


100%|██████████| 322/322 [04:48<00:00,  1.12it/s]


In [ ]:
df_combinaciones

In [ ]:
df_data_t = df_data[(df_data['FAMILIA'] == '23 - Jardin') & (df_data['TIPO_MEDIO'] == 'SEM - Brand') & (df_data['PAIS'] == 'MX')]
df_data_t

In [ ]:
print(f"Proporción de resultados exitosos: {round(100 * len(df_combinaciones[df_combinaciones['best_ecm'] < float('inf')]) / len(df_combinaciones), 2)} %")

In [ ]:
df_combinaciones[df_combinaciones.n_iters > 0] # Solo revisión

sys.exit('No mejora el ecm...empezar de nuevo todas estas proyecciones')

In [ ]:
sys.exit('Revisar los valores de c acá arriba (deberían ser >= 0.3)')

In [506]:
#sys.exit('Cuanto se demora la etapa de abajo si agrego en menos particiones? Será mejor llevarla al código de Horus???')

## 4.3. Consolidación parámetro

In [ ]:
df_combinaciones = tf.parquet_act(f'{dir_data_back}P4_df_registros')
#display(df_combinaciones)

df_data_inv_maxima = df_data[['PAIS', 'CANAL', 'FUENTE'] + campos_grano + ['INVERSION']].groupby(['PAIS', 'CANAL', 'FUENTE'] + campos_grano).max().reset_index()
df_data_inv_maxima = df_data_inv_maxima.rename(columns = {'INVERSION': 'INVERSION_MAXIMA'})
df_combinaciones = df_combinaciones.merge(df_data_inv_maxima, on = ['PAIS', 'CANAL', 'FUENTE'] + campos_grano, how = 'left')
df_combinaciones

In [ ]:
alpha = 1.5 # Parámetro (cuanto por sobre la inv máxima un día, se puede invertir)
particiones = 10 # en cuantos tramos se divide la inversión

df_dates = pd.date_range(dia_predecir_desde, dia_predecir_hasta) # Curve fitting de shares se trabaja desde el 2023
df_dates = pd.DataFrame({'DATE': df_dates})
df_dates['t'] = (df_dates['DATE'] - dt.datetime(2021, 1, 1)).dt.days # 1/1/2021 es cualquier dia de referencia (el mismo que el usado en el curve fitting)

df_particiones = pd.DataFrame({'PARTICIONES': [i / particiones for i in range(0, particiones + 1)]})

df_dates['AUX'] = 'aux'
df_particiones['AUX'] = 'aux'

df_dates_particiones = df_dates.merge(df_particiones, how = 'left', on = 'AUX')

df_dates_particiones

In [ ]:
df_combinaciones['INVERSION_MAXIMA'] *= alpha
df_combinaciones_otros = df_combinaciones[~df_combinaciones['PAIS'].isin(paises)].reset_index(drop = True)
df_combinaciones = df_combinaciones[df_combinaciones['PAIS'].isin(paises)].reset_index(drop = True)
df_combinaciones['AUX'] = 'aux'
df_combinaciones = df_combinaciones.merge(df_dates, how = 'left', on = 'AUX')
df_combinaciones = df_combinaciones.drop(columns = 'AUX')
sys.exit('Revisar df_combinaciones')


df_combinaciones['RANGO'] = particiones * df_combinaciones['PARTICIONES']
df_combinaciones['RANGO'] = df_combinaciones['RANGO'].astype(int)
df_combinaciones

In [ ]:
campos_beta, campos_grano

In [431]:
df_combinaciones.to_csv('df_combinaciones.csv', index = False)

In [400]:
ejecutar[4]['Consolidar'] = True

In [ ]:
if ejecutar[4]['Consolidar']:
    
    sys.exit('Cambiar por nueva lógica de linearización')
    
    df_combinaciones['INVERSION'] = df_combinaciones['INVERSION_MAXIMA'] * df_combinaciones['PARTICIONES']
    df_combinaciones = df_combinaciones.drop(columns = ['INVERSION_MAXIMA', 'PARTICIONES'])

    # Ver Dta/4_inversión (dibujo  con explicación)
    # Se busca más la curva azul que la verde, porque la verde siempre quedará por debajo de la original
    def func(X, a, c, n):
        i, t = X
        return a * (i ** n) + c * t * (i ** n)

    df_combinaciones['VISITAS_PAGO'] = df_combinaciones.apply(lambda x: func([x['INVERSION'], x['t']], x['a'], x['c'], x['n']), axis = 1)
    df_parametro_4 = df_combinaciones[campos_beta + campos_grano + ['RANGO', 'INVERSION', 'VISITAS_PAGO']] # Va desde rango = 0 hasta rango = rango (abajo rango += 1 y filtro de rango <= particiones)
    
    df_parametro_4[['INVERSION', 'VISITAS_PAGO']] = df_parametro_4[['INVERSION', 'VISITAS_PAGO']].fillna(0)
    df_parametro_4 = df_parametro_4.sort_values(by = campos_beta + campos_grano + ['RANGO']).reset_index(drop = True)
    df_parametro_4['I_0'] = df_parametro_4['INVERSION']
    df_parametro_4['I_F'] = df_parametro_4['INVERSION'].shift(-1)
    df_parametro_4['VP0'] = df_parametro_4['VISITAS_PAGO']
    df_parametro_4['VPF'] = df_parametro_4['VISITAS_PAGO'].shift(-1)
    df_parametro_4['RANGO'] = df_parametro_4['RANGO'] + 1
    df_parametro_4 = df_parametro_4[df_parametro_4['RANGO'] <= particiones].reset_index(drop = True)
    sys.exit('ANCLA 1')
    # Si tengo los puntos x = (i0, if), y = (vp0, vpf) entonces la recta que pasa por esos puntos es: A * I + B
    df_parametro_4['A'] = (df_parametro_4['VPF'] - df_parametro_4['VP0']) / (df_parametro_4['I_F'] - df_parametro_4['I_0']) # Pendiente
    df_parametro_4['B'] = df_parametro_4['VP0'] - df_parametro_4['A'] * df_parametro_4['I_0'] # Intersección con el eje y
    
    df_parametro_4['A'] = np.where(np.isnan(df_parametro_4['A']), 0, df_parametro_4['A']) # El fillna no los detecta como nan
    df_parametro_4['B'] = np.where(np.isnan(df_parametro_4['B']), 0, df_parametro_4['B']) 
    
    df_parametro_4 = pd.concat([df_parametro_4, df_combinaciones_otros]).reset_index(drop = True)
    

    #df_parametro_4_all = tf.parquet_act(f'{dir_data}Parametros/P4')
    #df_parametro_4_all = pd.concat([df_parametro_4, df_parametro_4_all])
    #df_parametro_4_all = df_parametro_4_all.drop_duplicates(subset = campos_grano + campos_beta).reset_index(drop = True)
    
    tf.parquet_act(f'{dir_data}Parametros/P4', df_parametro_4, 'save')

In [403]:
# ANCLA 1: Nueva linearización
df_combinaciones.to_csv('df_combinaciones.csv', index = False)

In [186]:
# Para la restricción, revisar: https://chatgpt.com/c/67102780-e244-8000-9602-2fae3a1cb6a0

In [ ]:
sys.exit()

# 5. Metas

In [593]:
# Las metas están disponibles en los archivos
dir_metas = '../Data/metas/'

In [ ]:
for a in os.listdir(dir_metas):
    print(a)
    df = tf.parquet_act(f'{dir_metas}{a[:-4]}')
    display(df.head())

# Sensibilización (por ej, de métricas orgánicas)

In [ ]:
dias_proyeccion = [dt.datetime.today().date(), dt.datetime.today().replace(year = dt.datetime.today().year + 1, month = 12, day = 31).date()] # declarar arriba, al inicio del código
dias_proyeccion

In [ ]:
año_incial, año_final = dias_proyeccion[0].year, dias_proyeccion[1].year
años_planificacion = [i for i in range(año_incial, año_final + 1)]
años_planificacion

In [597]:
archivo_calibracion = 'I3 Calibración métricas Planificación'# Parámetro

In [ ]:
df_tuplas_pais_proyeccion = df_tuplas[['PAIS', 'MODO_PROYECCION']].drop_duplicates().reset_index(drop = True)
df_tuplas_pais_proyeccion

In [599]:
correos_destinatarios = {'MX': {'writer': ['mvaldiviad@sodimac.cl']},
                            'CO': {'writer': ['mvaldiviad@sodimac.cl']},
                            'CL': {'writer': ['mvaldiviad@sodimac.cl']},
                            'BR': {'writer': ['mvaldiviad@sodimac.cl']},
                            'PE': {'writer': ['mvaldiviad@sodimac.cl']},
                            'AR': {'writer': ['mvaldiviad@sodimac.cl']},
                            'UY': {'writer': ['mvaldiviad@sodimac.cl']},}   # Input (Parametro)

In [ ]:
df_prop_organica = tf.parquet_act(f'{dir_metas}1_df_resumen_metas_otros')
df_prop_organica = df_prop_organica[['PAIS', 'PERIODO', 'PORCENTAJE_TRAFICO_ORGANICO']]
df_prop_organica['PORCENTAJE_TRAFICO_ORGANICO'] *= 1 / 100 
df_prop_organica.head()

In [ ]:
df_metas_resumen = tf.parquet_act(f'{dir_metas}0_df_resumen_metas_general')
df_metas_resumen = df_metas_resumen[['PAIS', 'PERIODO', 'VENTA_COLOCADA', 'ORDENES', 'VISITAS']].groupby(['PAIS', 'PERIODO'], as_index = False).sum()
df_metas_resumen = df_metas_resumen.merge(df_prop_organica, on = ['PAIS', 'PERIODO'], how = 'left')
df_metas_resumen = df_metas_resumen[['PAIS', 'PERIODO', 'VENTA_COLOCADA', 'ORDENES', 'VISITAS', 'PORCENTAJE_TRAFICO_ORGANICO']]
display(df_metas_resumen.head())

In [602]:
def cargar_metas_declaradas(modo_proyeccion, pais, df_metas_resumen_pais, correos_destinatarios, archivo_calibracion, años_planificacion, df_factores_duplicidad_pais):
    if modo_proyeccion == 'LIBRE':
        return None
    
    # 1. metas declatadas
    for año_planificacion in años_planificacion:
        nombre_archivo = f'{archivo_calibracion} {pais} {str(año_planificacion)}'
        df_carga = df_metas_resumen_pais[df_metas_resumen_pais['PERIODO'].str.contains(str(año_planificacion))].reset_index(drop = True)
        df_carga = df_carga[['PERIODO', 'VENTA_COLOCADA', 'ORDENES', 'VISITAS', 'PORCENTAJE_TRAFICO_ORGANICO']]
        tf.df_to_sheets(df_carga, correos_destinatarios = correos_destinatarios, nombre_archivo = nombre_archivo, nombre_hoja = '1. metas declaradas', output = False, crear_archivo = False, crear_hoja = False)
    
        # 2. factores lt totales
        df_factores_duplicidad_pais = df_factores_duplicidad_pais[['MES', 'VENTA_COLOCADA', 'ORDENES', 'VISITAS']]  
        tf.df_to_sheets(df_factores_duplicidad_pais, correos_destinatarios = correos_destinatarios, nombre_archivo = nombre_archivo, nombre_hoja = '1.2. factores lt totales', output = False, crear_archivo = False, crear_hoja = False)

    return None

In [ ]:
#sys.exit('Para crear df_factores_duplicidad_pais, tomar la data disponible en P1...recordar que, hay que enviar los datoa al archivo (sheets) con duplicacion LT y no F-LT')

# Aquí está todo, continuar
df_representantes_base = pd.DataFrame(columns = ['DUPLICACION', 'PAIS', 'DATE'])
if 'P1_df_all_data.pkl' in os.listdir(dir_data_back):
    df_representantes_base = tf.parquet_act(f'{dir_data_back}P1_df_all_data')

dia_hasta = df_max_date['DATE'][0].date()

dia_desde = (dia_hasta + dt.timedelta(days = 1)).replace(year = dia_hasta.year - 1)#.strftime('%Y-%m-%d')
dia_desde, dia_hasta

In [ ]:
def relacion_duplicidades(df_representantes_base, d_desagregada, dates, d_agregada = 'TOTALES', otros_campos = []):
    [dia_desde, dia_hasta] = dates
    dia_desde, dia_hasta = pd.to_datetime(dia_desde), pd.to_datetime(dia_hasta)
    df_representantes_base = df_representantes_base[df_representantes_base['DUPLICACION'].isin([d_desagregada, d_agregada])]
    df_representantes_base['DATE'] = pd.to_datetime(df_representantes_base['DATE'])
    df_representantes_base = df_representantes_base[(df_representantes_base['DATE'] >= dia_desde) & (df_representantes_base['DATE'] <= dia_hasta)].reset_index(drop = True)
    df_representantes_base['MES'] = df_representantes_base['DATE'].dt.month
    df_representantes_base = df_representantes_base[['PAIS', 'DUPLICACION', 'MES'] + otros_campos + ['VENTA_COLOCADA', 'ORDENES', 'VISITAS']].groupby(['PAIS', 'DUPLICACION', 'MES'] + otros_campos, as_index = False).sum()

    df_representantes_base_agregada = df_representantes_base[df_representantes_base['DUPLICACION'].isin([d_agregada])][['PAIS', 'MES']  + otros_campos + ['VENTA_COLOCADA', 'ORDENES', 'VISITAS']]
    df_representantes_base_desagregada = df_representantes_base[df_representantes_base['DUPLICACION'].isin([d_desagregada])][['PAIS', 'MES'] + otros_campos + ['VENTA_COLOCADA', 'ORDENES', 'VISITAS']]
    
    df_factores_duplicidad = df_representantes_base_desagregada.merge(df_representantes_base_agregada, on = ['PAIS', 'MES'] + otros_campos, how = 'left', suffixes = ('_DESAGREGADA', '_AGREGADA'))
    
    for m in ['VENTA_COLOCADA', 'ORDENES', 'VISITAS']:
        if m == 'VENTA_COLOCADA':
            df_factores_duplicidad[f'FACTOR_{m}'] = 1
        else:
            df_factores_duplicidad[f'FACTOR_{m}'] = df_factores_duplicidad[f'{m}_DESAGREGADA'] / df_factores_duplicidad[f'{m}_AGREGADA']
            df_factores_duplicidad[f'FACTOR_{m}'] = np.where(df_factores_duplicidad[f'FACTOR_{m}'] < 1, 1, df_factores_duplicidad[f'FACTOR_{m}'])
        df_factores_duplicidad = df_factores_duplicidad.drop(columns = [f'{m}_DESAGREGADA', f'{m}_AGREGADA'])
        df_factores_duplicidad = df_factores_duplicidad.rename(columns = {f'FACTOR_{m}': m})
    
    return df_factores_duplicidad


df_factores_duplicidad = relacion_duplicidades(df_representantes_base, 'LT', [dia_desde, dia_hasta])
df_factores_duplicidad

In [ ]:
años_planificacion

In [ ]:
sys.exit('Revisar variable años_planificacion')

In [ ]:
for i in range(len(df_tuplas_pais_proyeccion)):
    pais, modo_proyeccion = df_tuplas_pais_proyeccion['PAIS'][i], df_tuplas_pais_proyeccion['MODO_PROYECCION'][i]

    print(pais, modo_proyeccion)
    df_metas_resumen_pais = df_metas_resumen[df_metas_resumen['PAIS'] == pais].reset_index(drop = True)    
    df_factores_duplicidad_pais = df_factores_duplicidad[df_factores_duplicidad['PAIS'] == pais].reset_index(drop = True)
    cargar_metas_declaradas(modo_proyeccion, pais, df_metas_resumen_pais, correos_destinatarios, archivo_calibracion, años_planificacion, df_factores_duplicidad_pais)

In [ ]:
df_tuplas_pais_proyeccion

In [ ]:
df_representantes_base

In [ ]:
df_medios = tf.lectura_segura_Gsheets('clacomizacion_corp_v0', 'DICCIONARIO_MEDIOS_CORP')
df_medios = df_medios[['TIPO_MEDIO', 'NATURALEZA_MEDIO']].drop_duplicates().reset_index(drop = True)
df_medios['NATURALEZA_MEDIO'] = np.where(df_medios['NATURALEZA_MEDIO'] == 'Pago', 'PAGO', 'ORGANICO')
df_medios

In [ ]:
df_representantes_base_grano_lt = df_representantes_base[df_representantes_base['DUPLICACION'].isin([grano, 'LT'])]
df_representantes_base_grano_lt = df_representantes_base_grano_lt.merge(df_medios, on = 'TIPO_MEDIO', how = 'left')
df_representantes_base_grano_lt = df_representantes_base_grano_lt[['DUPLICACION', 'PAIS', 'DATE', 'CANAL', 'FUENTE', 'NATURALEZA_MEDIO', 'VENTA_COLOCADA', 'ORDENES', 'VISITAS']].groupby(['DUPLICACION', 'PAIS', 'DATE', 'CANAL', 'FUENTE', 'NATURALEZA_MEDIO'], as_index = False).sum()
print('Llevar a función para dos duplicidades, agr, desagr, campo_extra 8ej: nat_medio para este caso')
df_factores_duplicidad_grano_lt = relacion_duplicidades(df_representantes_base_grano_lt, grano, [dia_desde, dia_hasta], d_agregada = 'LT', otros_campos = ['NATURALEZA_MEDIO'])
df_factores_duplicidad_grano_lt

In [ ]:
df_metricas_aditivas = tf.parquet_act(f'{dir_data_back}P3_df_parametros_prediccion') # Todo esto es duplicidad F-LT
df_metricas_aditivas['y'] = df_metricas_aditivas['real'].fillna(df_metricas_aditivas['predict'])
df_metricas_aditivas = df_metricas_aditivas[['PAIS', 'MODO_PROYECCION', 'NATURALEZA_MEDIO', 'METRICA', 'DATE', 'y']]

df_metricas_aditivas_visitas_pago = df_metricas_aditivas[(df_metricas_aditivas['METRICA'] == 'VISITAS') & (df_metricas_aditivas['NATURALEZA_MEDIO'] == 'ORGANICO')]
df_metricas_aditivas_visitas_pago['NATURALEZA_MEDIO'] = 'PAGO'
df_metricas_aditivas_visitas_pago['y'] = 1

df_metricas_aditivas = pd.concat([df_metricas_aditivas, df_metricas_aditivas_visitas_pago]).reset_index(drop = True)

df_metricas_aditivas = df_metricas_aditivas.pivot(index = ['PAIS', 'MODO_PROYECCION', 'NATURALEZA_MEDIO', 'DATE'], columns = ['METRICA'], values = 'y').reset_index()
df_metricas_aditivas['ORDENES'] = df_metricas_aditivas['VISITAS'] * df_metricas_aditivas['TC']
df_metricas_aditivas['VENTA_COLOCADA'] = df_metricas_aditivas['ORDENES'] * df_metricas_aditivas['TP']
df_metricas_aditivas = df_metricas_aditivas.drop(columns = ['TC', 'TP'])

df_metricas_aditivas['PERIODO'] = df_metricas_aditivas['DATE'].dt.strftime('%Y-%m') # Todo agregado por año-mes
df_metricas_aditivas = df_metricas_aditivas[['PAIS', 'MODO_PROYECCION', 'NATURALEZA_MEDIO', 'PERIODO', 'VENTA_COLOCADA', 'ORDENES', 'VISITAS']].groupby(['PAIS', 'MODO_PROYECCION', 'NATURALEZA_MEDIO', 'PERIODO'], as_index = False).sum()

# Traspaso a duplicidad LT (desde grano)
for m in ['VENTA_COLOCADA', 'ORDENES', 'VISITAS']:
    df_factores_duplicidad_grano_lt = df_factores_duplicidad_grano_lt.rename(columns = {m: f'FACTOR_{m}'})
    
df_metricas_aditivas['MES'] = df_metricas_aditivas['PERIODO'].str[-2:].astype(int) # merge con factores de duplicidad
df_metricas_aditivas = df_metricas_aditivas.merge(df_factores_duplicidad_grano_lt, on = ['PAIS', 'MES', 'NATURALEZA_MEDIO'], how = 'left')

for m in ['VENTA_COLOCADA', 'ORDENES', 'VISITAS']: # ajustes por factor
    df_metricas_aditivas[m] = df_metricas_aditivas[m] / df_metricas_aditivas[f'FACTOR_{m}']
    df_metricas_aditivas = df_metricas_aditivas.drop(columns = [f'FACTOR_{m}'])

df_metricas_aditivas = df_metricas_aditivas.drop(columns = ['MES'])

df_metricas_aditivas['TC'], df_metricas_aditivas['TP'] = df_metricas_aditivas['ORDENES'] / df_metricas_aditivas['VISITAS'], df_metricas_aditivas['VENTA_COLOCADA'] / df_metricas_aditivas['ORDENES']
df_metricas_aditivas = df_metricas_aditivas.drop(columns = ['ORDENES', 'VENTA_COLOCADA'])
df_metricas_aditivas # Recordar que visitas pago no tienen una interpretación en si misma (y no se cargan)

In [ ]:
df_metricas_aditivas_ppto = df_metricas_aditivas[df_metricas_aditivas['MODO_PROYECCION'] == 'PRESUPUESTO'].reset_index(drop = True)
df_metricas_aditivas_ppto = df_metricas_aditivas_ppto[['PAIS', 'NATURALEZA_MEDIO', 'PERIODO', 'VISITAS', 'TC', 'TP']]
df_metricas_aditivas_ppto = df_metricas_aditivas_ppto.melt(id_vars = ['PAIS', 'NATURALEZA_MEDIO', 'PERIODO'], var_name = 'METRICA', value_name = 'VALOR_INICIAL')
df_metricas_aditivas_ppto = df_metricas_aditivas_ppto[~((df_metricas_aditivas_ppto['METRICA'] == 'VISITAS') & (df_metricas_aditivas_ppto['NATURALEZA_MEDIO'] == 'PAGO'))].reset_index(drop = True)
df_metricas_aditivas_ppto

In [ ]:
archivo_calibracion

Carga de metricas_raw

In [ ]:
df_tuplas_pais_proyeccion_ppto = df_tuplas_pais_proyeccion[df_tuplas_pais_proyeccion['MODO_PROYECCION'] == 'PRESUPUESTO'].reset_index(drop = True)

for i in range(len(df_tuplas_pais_proyeccion_ppto)):
    df_tupla = df_tuplas_pais_proyeccion_ppto.iloc[i:i + 1].reset_index(drop = True)
    pais = df_tupla['PAIS'][0]
    
    for año in años_planificacion:
        print(pais, año)
        df_carga = df_metricas_aditivas_ppto[df_metricas_aditivas_ppto['PAIS'] == pais].reset_index(drop = True)
        df_carga = df_carga[df_carga['PERIODO'].str.contains(str(año))].reset_index(drop = True)
        nombre_archivo = f'{archivo_calibracion} {pais} {str(año)}'
        tf.df_to_sheets(df_carga, correos_destinatarios = correos_destinatarios, nombre_archivo = nombre_archivo, nombre_hoja = 'metricas_raw', output = False, crear_archivo = False, crear_hoja = False)


#sys.exit('Continuar acá el 241111: La parte de arriba carga metricas raw en el archivo Calibración métricas Planificación ej_iron3 CL 2025')
#sys.exit('Seguir con las hojas del archivo de calibración (ir describiendolas) y con el proceso de ajuste entre este archivo y el código')
#print('Lo que está acá arriba reemplaza parte de la función # f: ajuste_de_metricas')

Se lee métricas, para contrastar inicial vs final

In [ ]:
df_tuplas_pais_proyeccion_ppto

In [ ]:
dic_sufix = {'VALOR_INICIAL': '0', 'VALOR_FINAL': 'F'}

df_P3_ajustado = pd.DataFrame()

df_factores_ajuste = pd.DataFrame()

for i in range(len(df_tuplas_pais_proyeccion_ppto)):
    df_tupla = df_tuplas_pais_proyeccion_ppto.iloc[i:i + 1].reset_index(drop = True)
    pais = df_tupla['PAIS'][0]
    
    df_pais = pd.DataFrame()
    
    for año in años_planificacion:
        
        print(pais, año)
        nombre_archivo = f'{archivo_calibracion} {pais} {str(año)}'
        df = tf.lectura_segura_Gsheets(nombre_archivo, 'metricas')
        
        #display(df)
        
        #df_visitas_pago = df[df['METRICA'] == 'VISITAS'].reset_index(drop = True)
        #df_visitas_pago['VALOR_INICIAL'], df_visitas_pago['VALOR_FINAL'] = 1, 1
        #df_visitas_pago['NATURALEZA_MEDIO'] = 'PAGO'
        
        #df = pd.concat([df, df_visitas_pago]).reset_index(drop = True)
        
        dic_df_valor = {}
        for valor in ['VALOR_INICIAL', 'VALOR_FINAL']:
            df_valor = df[['PAIS', 'NATURALEZA_MEDIO', 'PERIODO', 'METRICA', valor]]
            df_valor[valor] = df_valor[valor].replace('', 0)
            df_valor[valor] = df_valor[valor].astype(float)
            # pivot en metrica
            df_valor = df_valor.pivot(index = ['PAIS', 'NATURALEZA_MEDIO', 'PERIODO'], columns = 'METRICA', values = valor).reset_index()
            df_valor['ORDENES'] = df_valor['VISITAS'] * df_valor['TC']
            df_valor['VENTA_COLOCADA'] = df_valor['ORDENES'] * df_valor['TP']
            df_valor = df_valor.drop(columns = ['TC', 'TP'])
            
            for m in ['VISITAS', 'ORDENES', 'VENTA_COLOCADA']:
                df_valor = df_valor.rename(columns = {m: f'{m}_{dic_sufix[valor]}'})
            
            dic_df_valor[valor] = df_valor
        
        df_ajuste = dic_df_valor['VALOR_INICIAL'].merge(dic_df_valor['VALOR_FINAL'], on = ['PAIS', 'NATURALEZA_MEDIO', 'PERIODO'], how = 'left')
 
        df_pais = pd.concat([df_pais, df_ajuste]).reset_index(drop = True)
    
    df_factores_ajuste = pd.concat([df_factores_ajuste, df_pais]).reset_index(drop = True)
    
    for m in ['VENTA_COLOCADA', 'ORDENES', 'VISITAS']:
        df_pais[f'FACTOR_{m}'] = df_pais[f'{m}_F'] / df_pais[f'{m}_0']
        df_pais = df_pais.drop(columns = [f'{m}_0', f'{m}_F'])
    
    #display(df_pais.head())
    
    # P3 Orgánico

    df_metricas_aditivas = tf.pickle_act(f'{dir_data}Parametros/P3') # Todo esto es duplicidad F-LT
    df_metricas_aditivas['DATE'] = pd.to_datetime(df_metricas_aditivas['DATE'])
    df_metricas_aditivas = df_metricas_aditivas[df_metricas_aditivas['PAIS'] == pais].reset_index(drop = True)
    df_metricas_aditivas['y'] = df_metricas_aditivas['real'].fillna(df_metricas_aditivas['predict'])
    df_metricas_aditivas = df_metricas_aditivas[['PAIS', 'MODO_PROYECCION', 'FUENTE', 'CANAL', 'DATE', 'FAMILIA', 'TIPO_MEDIO', 'METRICA', 'y']]

    #display(df_metricas_aditivas)

    # pivotea metrica
    df_metricas_aditivas = df_metricas_aditivas.pivot(index = ['PAIS', 'MODO_PROYECCION', 'FUENTE', 'CANAL', 'DATE'] + campos_grano, columns = 'METRICA', values = 'y').reset_index()
    df_metricas_aditivas['ORDENES'] = df_metricas_aditivas['VISITAS'] * df_metricas_aditivas['TC']
    df_metricas_aditivas['VENTA_COLOCADA'] = df_metricas_aditivas['ORDENES'] * df_metricas_aditivas['TP']
    df_metricas_aditivas = df_metricas_aditivas.drop(columns = ['TC', 'TP'])

    df_metricas_aditivas = df_metricas_aditivas.fillna(0)
    #display(df_metricas_aditivas)

    df_pais_organico = df_pais[df_pais['NATURALEZA_MEDIO'] == 'ORGANICO'].reset_index(drop = True)
    df_metricas_aditivas['PERIODO'] = df_metricas_aditivas['DATE'].dt.strftime('%Y-%m') # Todo agregado por año-mes

    df_metricas_aditivas['NATURALEZA_MEDIO'] = 'ORGANICO'
    df_metricas_aditivas = df_metricas_aditivas.merge(df_pais_organico, on = ['PAIS', 'PERIODO', 'NATURALEZA_MEDIO'], how = 'left')
    #display(df_metricas_aditivas)

    for m in ['VENTA_COLOCADA', 'ORDENES', 'VISITAS']:
        df_metricas_aditivas[f'FACTOR_{m}'] = df_metricas_aditivas[f'FACTOR_{m}'].fillna(1)
        df_metricas_aditivas[m] *= df_metricas_aditivas[f'FACTOR_{m}'] # Sensibilización directa (sin auste (1 + delta / T))
        df_metricas_aditivas = df_metricas_aditivas.drop(columns = [f'FACTOR_{m}'])
    
    display(df_metricas_aditivas)
    df_P3_ajustado = pd.concat([df_P3_ajustado, df_metricas_aditivas]).reset_index(drop = True)


In [ ]:
if len(df_tuplas_pais_proyeccion_ppto) > 0:
    # Ajuste final para mantener el formato de P3
    df_P3_ajustado['TC'] = df_P3_ajustado['ORDENES'] / df_P3_ajustado['VISITAS']
    df_P3_ajustado['TP'] = df_P3_ajustado['VENTA_COLOCADA'] / df_P3_ajustado['ORDENES']
    df_P3_ajustado = df_P3_ajustado.drop(columns = ['ORDENES', 'VENTA_COLOCADA'])
    df_P3_ajustado = df_P3_ajustado.melt(id_vars = ['PAIS', 'MODO_PROYECCION', 'NATURALEZA_MEDIO', 'FUENTE', 'CANAL', 'DATE'] + campos_grano, var_name = 'METRICA', value_name = 'VALOR')
    df_P3_ajustado['NOMBRE'] = df_P3_ajustado['PAIS'] + '_' + df_P3_ajustado['MODO_PROYECCION'] + '_' + df_P3_ajustado['METRICA'] + '_' + df_P3_ajustado['NATURALEZA_MEDIO']
    df_P3_ajustado = df_P3_ajustado[['NOMBRE', 'PAIS', 'MODO_PROYECCION', 'FUENTE', 'CANAL', 'DATE'] + campos_grano + ['METRICA', 'VALOR']]
    df_aux = tf.pickle_act(f'{dir_data}Parametros/P3')
    df_P3_ajustado = df_aux.merge(df_P3_ajustado, on = ['NOMBRE', 'PAIS', 'MODO_PROYECCION', 'FUENTE', 'CANAL', 'DATE'] + campos_grano + ['METRICA'], how = 'left')

    for m in ['predict', 'predict 5.0%', 'predict 95.0%']:
        df_P3_ajustado[f'{m}_aux'] = df_P3_ajustado[m]
        
    for m in ['predict', 'predict 5.0%', 'predict 95.0%']:
        
        df_P3_ajustado['predict'] = df_P3_ajustado['predict'].replace(0, np.nan)
        print(m)
        try:
            #df_P3_ajustado_yes[m] = np.where(df_P3_ajustado_yes['real'].isna(), df_P3_ajustado_yes[m] * df_P3_ajustado_yes['VALOR'] / df_P3_ajustado_yes['predict'], df_P3_ajustado_yes[m])
            df_P3_ajustado[f'{m}_new'] = df_P3_ajustado[m] * df_P3_ajustado['VALOR'] / df_P3_ajustado['predict']
            df_P3_ajustado[m] = np.where(df_P3_ajustado['real'].isna(), df_P3_ajustado[f'{m}_new'], df_P3_ajustado[m])
            df_P3_ajustado[m] = df_P3_ajustado[m].fillna(df_P3_ajustado[f'{m}_aux'])
            df_P3_ajustado = df_P3_ajustado.drop(columns = [f'{m}_new', f'{m}_aux'])
            #df_P3_ajustado[m] = df_P3_ajustado.apply(lambda row: (row[m] * row['VALOR'] / row['predict'] if row['real'] is np.nan and row['predict'] != 0 else 0) if row['real'] is np.nan else row[m], axis = 1)
        except:
            #df_P3_ajustado_yes.to_csv('df_P3_ajustado_yes.csv')
            sys.exit('Error por división por 0. Revisar exportación df_P3_ajustado_yes.csv')
            
    #df_P3_ajustado = pd.concat([df_P3_ajustado_yes, df_P3_ajustado_no]).reset_index(drop = True)
    df_P3_ajustado = df_P3_ajustado.drop(columns = 'VALOR')
    tf.parquet_act(f'{dir_data}Parametros/P3_ajustado', df_P3_ajustado, 'save') # Todo esto es duplicidad F-LT
    display(df_P3_ajustado.head())
    display(df_P3_ajustado.tail())

Ajustar los valores en P3_df_parametros_prediccion (solo los predicts, no el real)

In [204]:
if len(df_tuplas_pais_proyeccion_ppto) > 0:
    df_factores_ajuste['TP_0'] = df_factores_ajuste['VENTA_COLOCADA_0'] / df_factores_ajuste['ORDENES_0']
    df_factores_ajuste['TP_F'] = df_factores_ajuste['VENTA_COLOCADA_F'] / df_factores_ajuste['ORDENES_F']
    df_factores_ajuste['TC_0'] = df_factores_ajuste['ORDENES_0'] / df_factores_ajuste['VISITAS_0']
    df_factores_ajuste['TC_F'] = df_factores_ajuste['ORDENES_F'] / df_factores_ajuste['VISITAS_F']
    df_factores_ajuste = df_factores_ajuste.drop(columns = ['ORDENES_0', 'ORDENES_F', 'VENTA_COLOCADA_0', 'VENTA_COLOCADA_F'])

    for m in ['TP', 'TC', 'VISITAS']:
        df_factores_ajuste[f'FACTOR_{m}'] = df_factores_ajuste[f'{m}_F'] / df_factores_ajuste[f'{m}_0']
        df_factores_ajuste = df_factores_ajuste.drop(columns = [f'{m}_0', f'{m}_F'])

    df_factores_ajuste = df_factores_ajuste.melt(id_vars = ['PAIS', 'NATURALEZA_MEDIO', 'PERIODO'], var_name = 'METRICA', value_name = 'FACTOR')
    df_factores_ajuste['METRICA'] = df_factores_ajuste['METRICA'].str.split('_', expand = True)[1]

In [205]:
if len(df_tuplas_pais_proyeccion_ppto) > 0:
    df_parametros_prediccion = tf.parquet_act(f'{dir_data_back}P3_df_parametros_prediccion') # Todo esto es duplicidad F-LT
    df_parametros_prediccion['PERIODO'] = df_parametros_prediccion['DATE'].dt.strftime('%Y-%m')
    df_parametros_prediccion = df_parametros_prediccion.merge(df_factores_ajuste, on = ['PAIS', 'NATURALEZA_MEDIO', 'PERIODO', 'METRICA'], how = 'left')
    df_parametros_prediccion['FACTOR'] = df_parametros_prediccion['FACTOR'].fillna(1)
    for m in ['predict', 'predict 5.0%', 'predict 95.0%']:
        df_parametros_prediccion[m] = df_parametros_prediccion[m] * df_parametros_prediccion['FACTOR']
    df_parametros_prediccion = df_parametros_prediccion.drop(columns = 'FACTOR')
    tf.parquet_act(f'{dir_data_back}P3_df_parametros_prediccion_ajustado', df_parametros_prediccion, 'save')

In [ ]:
# Parámetros OK (ajustados y no ajustados)
# Como presento los resultados que tengo hasta ahora?
# Matriz de transformación de visitas in y visitas totales!!
# Trabajar con un solo país mientras (MX)

print("integración de visitasIN y max entropía (https://chatgpt.com/c/673e3b77-464c-8000-a9d7-5bb500da438d)")
sys.exit('Continuar acá 241120')

Lectura de parámetros para revisión

In [ ]:

titulos = {1: 'relacion de factores de duplicidad', 2: 'tc y tp pago', 3: 'metricas orgánicas', 4: 'inversión y visitas pago'}

for i in range(1, 5):
    if i != 3:
        continue 
    print(i, titulos[i], '\n')
    df_parametro = tf.parquet_act(f'{dir_data}Parametros/P{i}')
    display(df_parametro.head())

In [ ]:
df_parametro[df_parametro['DATE'] == '2024-11-06']

# PPT Resultados para conversar con los paises